In [ ]:
## This file implements neural networks before and after lasso selection for p0017kpresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0017kpresabs_qual.csv')
df.shape

(253, 7080)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    0
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,AAATGAGAATAATAACAATAAAACAGTTAAAAATAATAAACTTCTTTTGAGCATGGCGCTTCCTCCAAATATAATATATTTGTTTTTATTCATCTCTTTT,AAATCGTTGACTACAATATAGTATTCATAGTCGGTTTTTATCGAGTAATTTTTCGAATTCATCAAAAATTCTATAGTAAATAATTGGTCTTCAGCAGTTT,AAATATCTACCTTTTAATTTATGTAACTACTATTAGTATGCATATTCATTAGTTTTACCAGGACCATTAATTACATAAGATGATTTAGACTCTCCTTTTT,AAATAAGTGTGTTAGCTATAAAAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTT,AAAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTT,AAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTTTATTGTTATTATTCTCATTTT,AAACGACCATTCAAAAGAACACGCAAAAGAAACTATAAGAAAAACAAACGGTGCTATTCGGTCAGCTTTAGATGACGCATTATATGATGGACTTATTTTT,AAAATGGTTATAAGAGGGGGGTTATACATGGATTTTTAAAATTATCGCGAAGTCGAGCCCCTCCCCGTTCCCCAAGTATTTTGATCGCTTTTGATTTTTT,AAAAGATTAATTATTTTTAACTAAAAATTAAAGTTCAATTAGTGTTTTGATGAATTTGGTCTTGATAAATTGAAATGATCTAAAAAACGCTATAATTTTT,pheno
0,107,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,2
1,109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,120335,0,0,0,0,0,0,0,1,0,...,1,0,1,1,0,1,0,0,1,2
4,120337,0,0,0,0,0,0,0,1,0,...,1,0,1,1,0,1,0,0,1,2


In [6]:
df['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 7079)

In [9]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA,...,AAATGAGAATAATAACAATAAAACAGTTAAAAATAATAAACTTCTTTTGAGCATGGCGCTTCCTCCAAATATAATATATTTGTTTTTATTCATCTCTTTT,AAATCGTTGACTACAATATAGTATTCATAGTCGGTTTTTATCGAGTAATTTTTCGAATTCATCAAAAATTCTATAGTAAATAATTGGTCTTCAGCAGTTT,AAATATCTACCTTTTAATTTATGTAACTACTATTAGTATGCATATTCATTAGTTTTACCAGGACCATTAATTACATAAGATGATTTAGACTCTCCTTTTT,AAATAAGTGTGTTAGCTATAAAAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTT,AAAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTT,AAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTTTATTGTTATTATTCTCATTTT,AAACGACCATTCAAAAGAACACGCAAAAGAAACTATAAGAAAAACAAACGGTGCTATTCGGTCAGCTTTAGATGACGCATTATATGATGGACTTATTTTT,AAAATGGTTATAAGAGGGGGGTTATACATGGATTTTTAAAATTATCGCGAAGTCGAGCCCCTCCCCGTTCCCCAAGTATTTTGATCGCTTTTGATTTTTT,AAAAGATTAATTATTTTTAACTAAAAATTAAAGTTCAATTAGTGTTTTGATGAATTTGGTCTTGATAAATTGAAATGATCTAAAAAACGCTATAATTTTT,pheno
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,1,0,0,1,2
4,0,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,1,0,0,1,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 7079) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
112,NRS027,1
207,NRS246,1
186,NRS218,2
88,CFBRSa70,2
156,NRS177,1
...,...,...
244,SR3585,0
99,GA50819,2
147,NRS161,2
49,CFBREBSa114,1


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 1ms/step - loss: 1.1120 - accuracy: 0.3898 - val_loss: 1.0156 - val_accuracy: 0.4737
Epoch 2/1000
177/177 [==============================] - 0s 504us/step - loss: 1.0339 - accuracy: 0.4746 - val_loss: 1.0108 - val_accuracy: 0.5263
Epoch 3/1000
177/177 [==============================] - 0s 504us/step - loss: 0.9912 - accuracy: 0.5085 - val_loss: 1.0076 - val_accuracy: 0.5263
Epoch 4/1000
177/177 [==============================] - 0s 591us/step - loss: 0.9711 - accuracy: 0.5367 - val_loss: 1.0087 - val_accuracy: 0.5395
Epoch 5/1000
177/177 [==============================] - ETA: 0s - loss: 0.9829 - accuracy: 0.54 - 0s 502us/step - loss: 0.9384 - accuracy: 0.5593 - val_loss: 1.0029 - val_accuracy: 0.5263
Epoch 6/1000
177/177 [==============================] - 0s 624us/step - loss: 0.9068 - accuracy: 0.5819 - val_loss: 1.0073 - val_accuracy: 0.5000
Epoch 7/1000
177/177 [=================

Epoch 112/1000
177/177 [==============================] - 0s 374us/step - loss: 0.1748 - accuracy: 0.9379 - val_loss: 2.0096 - val_accuracy: 0.4474
Epoch 113/1000
177/177 [==============================] - 0s 323us/step - loss: 0.1733 - accuracy: 0.9153 - val_loss: 2.0118 - val_accuracy: 0.4474
Epoch 114/1000
177/177 [==============================] - 0s 347us/step - loss: 0.1758 - accuracy: 0.9322 - val_loss: 1.9970 - val_accuracy: 0.4474
Epoch 115/1000
177/177 [==============================] - 0s 858us/step - loss: 0.1719 - accuracy: 0.9322 - val_loss: 1.9805 - val_accuracy: 0.4474
Epoch 116/1000
177/177 [==============================] - 0s 688us/step - loss: 0.1793 - accuracy: 0.9379 - val_loss: 2.0223 - val_accuracy: 0.4211
Epoch 117/1000
177/177 [==============================] - 0s 726us/step - loss: 0.1756 - accuracy: 0.9096 - val_loss: 2.1081 - val_accuracy: 0.4079
Epoch 118/1000
177/177 [==============================] - 0s 420us/step - loss: 0.1654 - accuracy: 0.9379 - val_

177/177 [==============================] - 0s 338us/step - loss: 0.1221 - accuracy: 0.9435 - val_loss: 2.8282 - val_accuracy: 0.4342
Epoch 223/1000
177/177 [==============================] - 0s 362us/step - loss: 0.1172 - accuracy: 0.9435 - val_loss: 2.7705 - val_accuracy: 0.3947
Epoch 224/1000
177/177 [==============================] - 0s 429us/step - loss: 0.1188 - accuracy: 0.9435 - val_loss: 2.7786 - val_accuracy: 0.3947
Epoch 225/1000
177/177 [==============================] - 0s 973us/step - loss: 0.1249 - accuracy: 0.9322 - val_loss: 2.8441 - val_accuracy: 0.4079
Epoch 226/1000
177/177 [==============================] - 0s 740us/step - loss: 0.1216 - accuracy: 0.9435 - val_loss: 2.8359 - val_accuracy: 0.4079
Epoch 227/1000
177/177 [==============================] - 0s 438us/step - loss: 0.1210 - accuracy: 0.9379 - val_loss: 2.8458 - val_accuracy: 0.4342
Epoch 228/1000
177/177 [==============================] - 0s 604us/step - loss: 0.1237 - accuracy: 0.9435 - val_loss: 2.8857 - 

177/177 [==============================] - 0s 820us/step - loss: 0.1077 - accuracy: 0.9435 - val_loss: 3.1869 - val_accuracy: 0.3947
Epoch 333/1000
177/177 [==============================] - 0s 764us/step - loss: 0.1214 - accuracy: 0.9379 - val_loss: 3.2150 - val_accuracy: 0.4211
Epoch 334/1000
177/177 [==============================] - 0s 2ms/step - loss: 0.1146 - accuracy: 0.9435 - val_loss: 3.2235 - val_accuracy: 0.4079
Epoch 335/1000
177/177 [==============================] - 0s 1ms/step - loss: 0.1077 - accuracy: 0.9435 - val_loss: 3.2079 - val_accuracy: 0.4079
Epoch 336/1000
177/177 [==============================] - 0s 875us/step - loss: 0.1099 - accuracy: 0.9435 - val_loss: 3.2084 - val_accuracy: 0.4211
Epoch 337/1000
177/177 [==============================] - 0s 563us/step - loss: 0.1102 - accuracy: 0.9322 - val_loss: 3.1799 - val_accuracy: 0.4211
Epoch 338/1000
177/177 [==============================] - 0s 463us/step - loss: 0.1129 - accuracy: 0.9266 - val_loss: 3.1777 - val_

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.144239). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


...

177/177 [==============================] - 0s 394us/step - loss: 0.1029 - accuracy: 0.9379 - val_loss: 3.5480 - val_accuracy: 0.4211
Epoch 701/1000
177/177 [==============================] - 0s 369us/step - loss: 0.1090 - accuracy: 0.9379 - val_loss: 3.5209 - val_accuracy: 0.4342
Epoch 702/1000
177/177 [==============================] - 0s 413us/step - loss: 0.1083 - accuracy: 0.9266 - val_loss: 3.5041 - val_accuracy: 0.4211
Epoch 703/1000
177/177 [==============================] - 0s 645us/step - loss: 0.1095 - accuracy: 0.9322 - val_loss: 3.4984 - val_accuracy: 0.4211
Epoch 704/1000
177/177 [==============================] - 0s 598us/step - loss: 0.1108 - accuracy: 0.9379 - val_loss: 3.4868 - val_accuracy: 0.4474
Epoch 705/1000
177/177 [==============================] - 0s 863us/step - loss: 0.1095 - accuracy: 0.9492 - val_loss: 3.5311 - val_accuracy: 0.4342
Epoch 706/1000
177/177 [==============================] - 0s 416us/step - loss: 0.1122 - accuracy: 0.9435 - val_loss: 3.5364 - 

177/177 [==============================] - 0s 422us/step - loss: 0.1171 - accuracy: 0.9209 - val_loss: 3.5461 - val_accuracy: 0.4211
Epoch 811/1000
177/177 [==============================] - 0s 444us/step - loss: 0.1105 - accuracy: 0.9435 - val_loss: 3.5794 - val_accuracy: 0.4474
Epoch 812/1000
177/177 [==============================] - 0s 410us/step - loss: 0.1087 - accuracy: 0.9379 - val_loss: 3.5740 - val_accuracy: 0.4342
Epoch 813/1000
177/177 [==============================] - 0s 405us/step - loss: 0.1083 - accuracy: 0.9379 - val_loss: 3.5172 - val_accuracy: 0.4342
Epoch 814/1000
177/177 [==============================] - 0s 397us/step - loss: 0.1184 - accuracy: 0.9379 - val_loss: 3.5666 - val_accuracy: 0.4342
Epoch 815/1000
177/177 [==============================] - 0s 384us/step - loss: 0.1110 - accuracy: 0.9379 - val_loss: 3.5976 - val_accuracy: 0.4342
Epoch 816/1000
177/177 [==============================] - 0s 415us/step - loss: 0.1073 - accuracy: 0.9435 - val_loss: 3.6020 - 

177/177 [==============================] - 0s 481us/step - loss: 0.1012 - accuracy: 0.9492 - val_loss: 3.7136 - val_accuracy: 0.4342
Epoch 921/1000
177/177 [==============================] - 0s 2ms/step - loss: 0.1210 - accuracy: 0.9379 - val_loss: 3.7425 - val_accuracy: 0.4211
Epoch 922/1000
177/177 [==============================] - 0s 845us/step - loss: 0.1162 - accuracy: 0.9379 - val_loss: 3.6742 - val_accuracy: 0.4211
Epoch 923/1000
177/177 [==============================] - 0s 648us/step - loss: 0.1048 - accuracy: 0.9435 - val_loss: 3.7068 - val_accuracy: 0.4079
Epoch 924/1000
177/177 [==============================] - 0s 2ms/step - loss: 0.1157 - accuracy: 0.9435 - val_loss: 3.7707 - val_accuracy: 0.4079
Epoch 925/1000
177/177 [==============================] - 0s 415us/step - loss: 0.1111 - accuracy: 0.9379 - val_loss: 3.7920 - val_accuracy: 0.4211
Epoch 926/1000
177/177 [==============================] - 0s 621us/step - loss: 0.1056 - accuracy: 0.9379 - val_loss: 3.7249 - val_

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.113515). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 506us/step - loss: 0.1124 - accuracy: 0.9435 - val_loss: 3.7082 - val_accuracy: 0.4079
Epoch 953/1000
177/177 [==============================] - 0s 569us/step - loss: 0.1115 - accuracy: 0.9266 - val_loss: 3.6264 - val_accuracy: 0.4342
Epoch 954/1000
177/177 [==============================] - 0s 926us/step - loss: 0.1133 - accuracy: 0.9379 - val_loss: 3.6144 - val_accuracy: 0.4342
Epoch 955/1000
177/177 [==============================] - 0s 503us/step - loss: 0.1015 - accuracy: 0.9435 - val_loss: 3.6640 - val_accuracy: 0.4211
Epoch 956/1000
177/177 [==============================] - 0s 907us/step - loss: 0.1094 - accuracy: 0.9435 - val_loss: 3.7693 - val_accuracy: 0.4211
Epoch 957/1000
177/177 [==============================] - 0s 616us/step - loss: 0.1080 - accuracy: 0.9435 - val_loss: 3.7428 - val_accuracy: 0.4342
Epoch 958/1000
177/177 [==============================] - 0s 422us/step - loss: 0.1095 - accuracy: 0.9322 - val_loss: 3.6838 - 

In [70]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 155us/step
test accuracy: 47.37%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 0, 1, 2, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 2, 1, 1, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0,
       0, 2, 1, 0, 1, 2, 2, 1, 0, 0, 0, 2, 0, 1, 1, 0, 1, 0, 0, 2, 1, 2,
       0, 0, 1, 2, 2, 2, 0, 0, 0, 0])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
112,NRS027,1,0
207,NRS246,1,0
186,NRS218,2,1
88,CFBRSa70,2,0
156,NRS177,1,1
...,...,...,...
244,SR3585,0,2
99,GA50819,2,0
147,NRS161,2,0
49,CFBREBSa114,1,0


In [23]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.759813,0.001304,2.388830e-01
1,0.947819,0.009591,4.259091e-02
2,0.008693,0.989390,1.916300e-03
3,0.813774,0.001482,1.847438e-01
4,0.000916,0.998926,1.573724e-04
...,...,...,...
71,0.005992,0.014274,9.797335e-01
72,0.999994,0.000005,1.491994e-06
73,0.999998,0.000002,9.895054e-10
74,0.813774,0.001482,1.847438e-01


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p17.csv", index = False,
         header=None)

In [75]:
hist1 = model1.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 410us/step - loss: 0.1166 - accuracy: 0.9379 - val_loss: 3.6037 - val_accuracy: 0.4605
Epoch 2/1000
177/177 [==============================] - 0s 469us/step - loss: 0.1086 - accuracy: 0.9379 - val_loss: 3.5524 - val_accuracy: 0.4737
Epoch 3/1000
177/177 [==============================] - 0s 391us/step - loss: 0.1159 - accuracy: 0.9435 - val_loss: 3.5938 - val_accuracy: 0.4474
Epoch 4/1000
177/177 [==============================] - 0s 367us/step - loss: 0.1060 - accuracy: 0.9435 - val_loss: 3.5173 - val_accuracy: 0.4605
Epoch 5/1000
177/177 [==============================] - 0s 338us/step - loss: 0.1104 - accuracy: 0.9266 - val_loss: 3.5271 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 415us/step - loss: 0.1088 - accuracy: 0.9435 - val_loss: 3.5526 - val_accuracy: 0.4605
Epoch 7/1000
177/177 [==============================] - 0s 349us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 396us/step - loss: 0.1153 - accuracy: 0.9379 - val_loss: 3.7440 - val_accuracy: 0.4737
Epoch 113/1000
177/177 [==============================] - 0s 477us/step - loss: 0.1130 - accuracy: 0.9153 - val_loss: 3.6726 - val_accuracy: 0.4737
Epoch 114/1000
177/177 [==============================] - 0s 396us/step - loss: 0.0982 - accuracy: 0.9492 - val_loss: 3.6040 - val_accuracy: 0.4605
Epoch 115/1000
177/177 [==============================] - 0s 385us/step - loss: 0.1149 - accuracy: 0.9379 - val_loss: 3.6118 - val_accuracy: 0.4605
Epoch 116/1000
177/177 [==============================] - 0s 386us/step - loss: 0.1136 - accuracy: 0.9435 - val_loss: 3.7150 - val_accuracy: 0.4737
Epoch 117/1000
177/177 [==============================] - 0s 392us/step - loss: 0.1073 - accuracy: 0.9379 - val_loss: 3.7261 - val_accuracy: 0.4737
Epoch 118/1000
177/177 [==============================] - 0s 386us/step - loss: 0.1082 - accuracy: 0.9322 - val_

177/177 [==============================] - 0s 393us/step - loss: 0.1022 - accuracy: 0.9435 - val_loss: 3.7257 - val_accuracy: 0.4737
Epoch 223/1000
177/177 [==============================] - 0s 359us/step - loss: 0.1051 - accuracy: 0.9322 - val_loss: 3.7330 - val_accuracy: 0.4605
Epoch 224/1000
177/177 [==============================] - 0s 404us/step - loss: 0.1071 - accuracy: 0.9435 - val_loss: 3.7610 - val_accuracy: 0.4605
Epoch 225/1000
177/177 [==============================] - 0s 410us/step - loss: 0.1150 - accuracy: 0.9379 - val_loss: 3.7024 - val_accuracy: 0.4605
Epoch 226/1000
177/177 [==============================] - 0s 377us/step - loss: 0.1111 - accuracy: 0.9379 - val_loss: 3.7235 - val_accuracy: 0.4868
Epoch 227/1000
177/177 [==============================] - 0s 418us/step - loss: 0.1046 - accuracy: 0.9379 - val_loss: 3.7599 - val_accuracy: 0.4605
Epoch 228/1000
177/177 [==============================] - 0s 422us/step - loss: 0.1031 - accuracy: 0.9435 - val_loss: 3.7817 - 

177/177 [==============================] - 0s 808us/step - loss: 0.1047 - accuracy: 0.9435 - val_loss: 3.7910 - val_accuracy: 0.4605
Epoch 333/1000
177/177 [==============================] - 0s 523us/step - loss: 0.1089 - accuracy: 0.9435 - val_loss: 3.7812 - val_accuracy: 0.4605
Epoch 334/1000
177/177 [==============================] - 0s 627us/step - loss: 0.1070 - accuracy: 0.9379 - val_loss: 3.7130 - val_accuracy: 0.4737
Epoch 335/1000
177/177 [==============================] - 0s 561us/step - loss: 0.1133 - accuracy: 0.9266 - val_loss: 3.7300 - val_accuracy: 0.4605
Epoch 336/1000
177/177 [==============================] - 0s 548us/step - loss: 0.1087 - accuracy: 0.9322 - val_loss: 3.7422 - val_accuracy: 0.4737
Epoch 337/1000
177/177 [==============================] - 0s 518us/step - loss: 0.1070 - accuracy: 0.9379 - val_loss: 3.7817 - val_accuracy: 0.4605
Epoch 338/1000
177/177 [==============================] - 0s 466us/step - loss: 0.1075 - accuracy: 0.9379 - val_loss: 3.7852 - 

177/177 [==============================] - 0s 427us/step - loss: 0.1139 - accuracy: 0.9379 - val_loss: 3.7996 - val_accuracy: 0.4737
Epoch 443/1000
177/177 [==============================] - 0s 403us/step - loss: 0.1096 - accuracy: 0.9435 - val_loss: 3.7985 - val_accuracy: 0.4868
Epoch 444/1000
177/177 [==============================] - 0s 396us/step - loss: 0.1015 - accuracy: 0.9435 - val_loss: 3.8474 - val_accuracy: 0.4737
Epoch 445/1000
177/177 [==============================] - 0s 348us/step - loss: 0.1067 - accuracy: 0.9379 - val_loss: 3.9069 - val_accuracy: 0.4605
Epoch 446/1000
177/177 [==============================] - 0s 346us/step - loss: 0.1116 - accuracy: 0.9435 - val_loss: 3.8221 - val_accuracy: 0.4605
Epoch 447/1000
177/177 [==============================] - 0s 357us/step - loss: 0.1086 - accuracy: 0.9379 - val_loss: 3.7648 - val_accuracy: 0.4868
Epoch 448/1000
177/177 [==============================] - 0s 385us/step - loss: 0.1075 - accuracy: 0.9435 - val_loss: 3.7819 - 

177/177 [==============================] - 0s 366us/step - loss: 0.1071 - accuracy: 0.9379 - val_loss: 3.8778 - val_accuracy: 0.4474
Epoch 553/1000
177/177 [==============================] - 0s 363us/step - loss: 0.1112 - accuracy: 0.9379 - val_loss: 3.9764 - val_accuracy: 0.4605
Epoch 554/1000
177/177 [==============================] - 0s 359us/step - loss: 0.1078 - accuracy: 0.9435 - val_loss: 4.1864 - val_accuracy: 0.4342
Epoch 555/1000
177/177 [==============================] - 0s 343us/step - loss: 0.1378 - accuracy: 0.9266 - val_loss: 3.9680 - val_accuracy: 0.4605
Epoch 556/1000
177/177 [==============================] - 0s 374us/step - loss: 0.1135 - accuracy: 0.9379 - val_loss: 3.9908 - val_accuracy: 0.4342
Epoch 557/1000
177/177 [==============================] - 0s 367us/step - loss: 0.1078 - accuracy: 0.9435 - val_loss: 3.9659 - val_accuracy: 0.4605
Epoch 558/1000
177/177 [==============================] - 0s 366us/step - loss: 0.1137 - accuracy: 0.9435 - val_loss: 3.9232 - 

177/177 [==============================] - 0s 374us/step - loss: 0.1022 - accuracy: 0.9492 - val_loss: 3.9808 - val_accuracy: 0.4605
Epoch 663/1000
177/177 [==============================] - 0s 380us/step - loss: 0.1080 - accuracy: 0.9379 - val_loss: 3.9794 - val_accuracy: 0.4474
Epoch 664/1000
177/177 [==============================] - 0s 340us/step - loss: 0.1056 - accuracy: 0.9435 - val_loss: 4.0009 - val_accuracy: 0.4737
Epoch 665/1000
177/177 [==============================] - 0s 354us/step - loss: 0.1046 - accuracy: 0.9435 - val_loss: 4.0045 - val_accuracy: 0.4868
Epoch 666/1000
177/177 [==============================] - 0s 359us/step - loss: 0.1112 - accuracy: 0.9379 - val_loss: 4.0676 - val_accuracy: 0.4737
Epoch 667/1000
177/177 [==============================] - 0s 353us/step - loss: 0.1047 - accuracy: 0.9379 - val_loss: 4.0191 - val_accuracy: 0.4605
Epoch 668/1000
177/177 [==============================] - 0s 352us/step - loss: 0.1049 - accuracy: 0.9322 - val_loss: 3.9803 - 

177/177 [==============================] - 0s 436us/step - loss: 0.1085 - accuracy: 0.9266 - val_loss: 3.7423 - val_accuracy: 0.4605
Epoch 773/1000
177/177 [==============================] - 0s 455us/step - loss: 0.1066 - accuracy: 0.9379 - val_loss: 3.8121 - val_accuracy: 0.4474
Epoch 774/1000
177/177 [==============================] - 0s 442us/step - loss: 0.1055 - accuracy: 0.9435 - val_loss: 3.9141 - val_accuracy: 0.4342
Epoch 775/1000
177/177 [==============================] - 0s 383us/step - loss: 0.1094 - accuracy: 0.9379 - val_loss: 3.9159 - val_accuracy: 0.4342
Epoch 776/1000
177/177 [==============================] - 0s 409us/step - loss: 0.1125 - accuracy: 0.9435 - val_loss: 3.7607 - val_accuracy: 0.4474
Epoch 777/1000
177/177 [==============================] - 0s 791us/step - loss: 0.1076 - accuracy: 0.9435 - val_loss: 3.7737 - val_accuracy: 0.4605
Epoch 778/1000
177/177 [==============================] - 0s 667us/step - loss: 0.1120 - accuracy: 0.9435 - val_loss: 3.8773 - 

177/177 [==============================] - 0s 373us/step - loss: 0.1048 - accuracy: 0.9322 - val_loss: 4.0529 - val_accuracy: 0.4474
Epoch 883/1000
177/177 [==============================] - 0s 336us/step - loss: 0.1065 - accuracy: 0.9435 - val_loss: 4.0829 - val_accuracy: 0.4474
Epoch 884/1000
177/177 [==============================] - 0s 365us/step - loss: 0.1072 - accuracy: 0.9435 - val_loss: 4.1604 - val_accuracy: 0.4474
Epoch 885/1000
177/177 [==============================] - 0s 364us/step - loss: 0.1089 - accuracy: 0.9435 - val_loss: 4.1027 - val_accuracy: 0.4737
Epoch 886/1000
177/177 [==============================] - 0s 356us/step - loss: 0.1041 - accuracy: 0.9379 - val_loss: 4.0895 - val_accuracy: 0.4605
Epoch 887/1000
177/177 [==============================] - 0s 372us/step - loss: 0.1075 - accuracy: 0.9435 - val_loss: 4.0905 - val_accuracy: 0.4605
Epoch 888/1000
177/177 [==============================] - 0s 347us/step - loss: 0.1056 - accuracy: 0.9379 - val_loss: 4.1435 - 

177/177 [==============================] - 0s 348us/step - loss: 0.1025 - accuracy: 0.9435 - val_loss: 4.1221 - val_accuracy: 0.4605
Epoch 993/1000
177/177 [==============================] - 0s 374us/step - loss: 0.1071 - accuracy: 0.9379 - val_loss: 4.1381 - val_accuracy: 0.4605
Epoch 994/1000
177/177 [==============================] - 0s 358us/step - loss: 0.1034 - accuracy: 0.9435 - val_loss: 4.1467 - val_accuracy: 0.4605
Epoch 995/1000
177/177 [==============================] - 0s 351us/step - loss: 0.1038 - accuracy: 0.9435 - val_loss: 4.0906 - val_accuracy: 0.4605
Epoch 996/1000
177/177 [==============================] - 0s 359us/step - loss: 0.1063 - accuracy: 0.9379 - val_loss: 4.0863 - val_accuracy: 0.4605
Epoch 997/1000
177/177 [==============================] - 0s 359us/step - loss: 0.1065 - accuracy: 0.9435 - val_loss: 4.1208 - val_accuracy: 0.4605
Epoch 998/1000
177/177 [==============================] - 0s 357us/step - loss: 0.1119 - accuracy: 0.9435 - val_loss: 4.1601 - 

In [76]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 93.87%


In [17]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [18]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [19]:
y_prob = df_proba[df_proba['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[7.59812530e-01, 1.30435520e-03, 2.38883020e-01],
       [9.47818500e-01, 9.59060500e-03, 4.25909120e-02],
       [8.69330200e-03, 9.89390430e-01, 1.91630020e-03],
       [8.13774000e-01, 1.48225110e-03, 1.84743800e-01],
       [9.16057700e-04, 9.98926460e-01, 1.57372440e-04],
       [4.28411250e-02, 9.57129360e-01, 2.95791960e-05],
       [9.68521200e-01, 1.50188270e-04, 3.13286220e-02],
       [8.53036100e-01, 1.12316280e-03, 1.45840730e-01],
       [9.49329000e-01, 4.69951030e-02, 3.67594020e-03],
       [8.00499040e-05, 6.93903100e-02, 9.30529600e-01],
       [4.31731700e-01, 5.66080100e-01, 2.18832170e-03],
       [4.41110400e-03, 9.94466840e-01, 1.12205280e-03],
       [4.33804730e-06, 3.36855750e-07, 9.99995350e-01],
       [9.78957400e-06, 3.54735860e-03, 9.96442850e-01],
       [9.55464240e-01, 4.04707340e-02, 4.06513200e-03],
       [9.99955650e-01, 4.39769650e-05, 3.30461460e-07],
       [4.92761270e-06, 9.99984260e-01, 1.08302450e-05],
       [6.83702850e-03, 1.31683

In [20]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.5804759614283423

In [24]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [25]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.5804759614283423

In [26]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [27]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [28]:
dat2

,id,test
178,NRS210,2
104,Grady1,0
79,CFBRSa29,2
66,CFBRSa03,0
9,217,1
...,...,...
25,BCH-SA-10,1
94,GA15,2
244,SR3585,0
227,NRS387,2


In [29]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [31]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [32]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model2.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 2ms/step - loss: 1.1335 - accuracy: 0.3390 - val_loss: 1.0969 - val_accuracy: 0.3421
Epoch 2/1000
177/177 [==============================] - 0s 589us/step - loss: 1.0412 - accuracy: 0.4463 - val_loss: 1.0453 - val_accuracy: 0.5000
Epoch 3/1000
177/177 [==============================] - 0s 747us/step - loss: 0.9807 - accuracy: 0.5424 - val_loss: 1.0248 - val_accuracy: 0.4737
Epoch 4/1000
177/177 [==============================] - 0s 724us/step - loss: 0.9577 - accuracy: 0.5989 - val_loss: 1.0773 - val_accuracy: 0.4342
Epoch 5/1000
177/177 [==============================] - 0s 480us/step - loss: 0.9809 - accuracy: 0.5763 - val_loss: 1.1440 - val_accuracy: 0.4211
Epoch 6/1000
177/177 [==============================] - 0s 848us/step - loss: 0.9102 - accuracy: 0.6384 - val_loss: 1.0576 - val_accuracy: 0.4474
Epoch 7/1000
177/177 [==============================] - 0s 1ms/step - loss: 0.874

Epoch 112/1000
177/177 [==============================] - 0s 487us/step - loss: 0.2164 - accuracy: 0.8983 - val_loss: 2.3169 - val_accuracy: 0.4211
Epoch 113/1000
177/177 [==============================] - 0s 418us/step - loss: 0.2125 - accuracy: 0.9096 - val_loss: 2.3373 - val_accuracy: 0.4211
Epoch 114/1000
177/177 [==============================] - 0s 414us/step - loss: 0.2113 - accuracy: 0.8814 - val_loss: 2.3760 - val_accuracy: 0.4211
Epoch 115/1000
177/177 [==============================] - 0s 425us/step - loss: 0.2050 - accuracy: 0.9209 - val_loss: 2.4095 - val_accuracy: 0.4079
Epoch 116/1000
177/177 [==============================] - 0s 334us/step - loss: 0.2023 - accuracy: 0.9322 - val_loss: 2.3687 - val_accuracy: 0.3947
Epoch 117/1000
177/177 [==============================] - 0s 361us/step - loss: 0.2234 - accuracy: 0.9153 - val_loss: 2.3664 - val_accuracy: 0.4079
Epoch 118/1000
177/177 [==============================] - 0s 379us/step - loss: 0.2052 - accuracy: 0.9153 - val_

177/177 [==============================] - 0s 549us/step - loss: 0.1610 - accuracy: 0.9322 - val_loss: 3.1298 - val_accuracy: 0.4079
Epoch 223/1000
177/177 [==============================] - 0s 505us/step - loss: 0.1540 - accuracy: 0.9266 - val_loss: 3.1557 - val_accuracy: 0.4211
Epoch 224/1000
177/177 [==============================] - 0s 408us/step - loss: 0.1551 - accuracy: 0.9266 - val_loss: 3.1626 - val_accuracy: 0.4342
Epoch 225/1000
177/177 [==============================] - 0s 441us/step - loss: 0.1460 - accuracy: 0.9266 - val_loss: 3.2162 - val_accuracy: 0.3947
Epoch 226/1000
177/177 [==============================] - 0s 405us/step - loss: 0.1587 - accuracy: 0.9266 - val_loss: 3.1531 - val_accuracy: 0.4211
Epoch 227/1000
177/177 [==============================] - 0s 424us/step - loss: 0.1511 - accuracy: 0.9266 - val_loss: 3.1575 - val_accuracy: 0.4079
Epoch 228/1000
177/177 [==============================] - 0s 415us/step - loss: 0.1542 - accuracy: 0.9266 - val_loss: 3.2536 - 

177/177 [==============================] - 0s 380us/step - loss: 0.1670 - accuracy: 0.9379 - val_loss: 3.5807 - val_accuracy: 0.3947
Epoch 333/1000
177/177 [==============================] - 0s 373us/step - loss: 0.1446 - accuracy: 0.9266 - val_loss: 3.5611 - val_accuracy: 0.3947
Epoch 334/1000
177/177 [==============================] - 0s 351us/step - loss: 0.1410 - accuracy: 0.9266 - val_loss: 3.4857 - val_accuracy: 0.4211
Epoch 335/1000
177/177 [==============================] - 0s 361us/step - loss: 0.1432 - accuracy: 0.9322 - val_loss: 3.5621 - val_accuracy: 0.4342
Epoch 336/1000
177/177 [==============================] - 0s 382us/step - loss: 0.1381 - accuracy: 0.9209 - val_loss: 3.5942 - val_accuracy: 0.4079
Epoch 337/1000
177/177 [==============================] - 0s 392us/step - loss: 0.1430 - accuracy: 0.9266 - val_loss: 3.5703 - val_accuracy: 0.3947
Epoch 338/1000
177/177 [==============================] - 0s 592us/step - loss: 0.1442 - accuracy: 0.9209 - val_loss: 3.5401 - 

177/177 [==============================] - 0s 1ms/step - loss: 0.1378 - accuracy: 0.9379 - val_loss: 3.6974 - val_accuracy: 0.4342
Epoch 443/1000
177/177 [==============================] - 0s 1ms/step - loss: 0.1329 - accuracy: 0.9209 - val_loss: 3.7741 - val_accuracy: 0.4079
Epoch 444/1000
177/177 [==============================] - 0s 914us/step - loss: 0.1371 - accuracy: 0.9379 - val_loss: 3.8755 - val_accuracy: 0.4079
Epoch 445/1000
177/177 [==============================] - 0s 1ms/step - loss: 0.1268 - accuracy: 0.9322 - val_loss: 3.8649 - val_accuracy: 0.4079
Epoch 446/1000
177/177 [==============================] - 0s 524us/step - loss: 0.1526 - accuracy: 0.9266 - val_loss: 3.7671 - val_accuracy: 0.4211
Epoch 447/1000
177/177 [==============================] - 0s 504us/step - loss: 0.1398 - accuracy: 0.9209 - val_loss: 3.7851 - val_accuracy: 0.3947
Epoch 448/1000
177/177 [==============================] - 0s 754us/step - loss: 0.1623 - accuracy: 0.9266 - val_loss: 3.8174 - val_ac

177/177 [==============================] - 0s 742us/step - loss: 0.1242 - accuracy: 0.9266 - val_loss: 3.9365 - val_accuracy: 0.3816
Epoch 553/1000
177/177 [==============================] - 0s 456us/step - loss: 0.1380 - accuracy: 0.9379 - val_loss: 3.8660 - val_accuracy: 0.4211
Epoch 554/1000
177/177 [==============================] - 0s 438us/step - loss: 0.1532 - accuracy: 0.9266 - val_loss: 3.8826 - val_accuracy: 0.4079
Epoch 555/1000
177/177 [==============================] - 0s 347us/step - loss: 0.1273 - accuracy: 0.9266 - val_loss: 3.9339 - val_accuracy: 0.4079
Epoch 556/1000
177/177 [==============================] - 0s 623us/step - loss: 0.1462 - accuracy: 0.9266 - val_loss: 3.8445 - val_accuracy: 0.4211
Epoch 557/1000
177/177 [==============================] - 0s 861us/step - loss: 0.1399 - accuracy: 0.9209 - val_loss: 3.9138 - val_accuracy: 0.3684
Epoch 558/1000
177/177 [==============================] - 0s 676us/step - loss: 0.1404 - accuracy: 0.9435 - val_loss: 3.8308 - 

177/177 [==============================] - 0s 424us/step - loss: 0.1307 - accuracy: 0.9266 - val_loss: 4.0053 - val_accuracy: 0.3947
Epoch 663/1000
177/177 [==============================] - 0s 615us/step - loss: 0.1297 - accuracy: 0.9379 - val_loss: 3.9986 - val_accuracy: 0.3947
Epoch 664/1000
177/177 [==============================] - 0s 934us/step - loss: 0.1277 - accuracy: 0.9322 - val_loss: 3.9524 - val_accuracy: 0.3947
Epoch 665/1000
177/177 [==============================] - 0s 556us/step - loss: 0.1290 - accuracy: 0.9379 - val_loss: 3.9471 - val_accuracy: 0.4079
Epoch 666/1000
177/177 [==============================] - 0s 601us/step - loss: 0.1294 - accuracy: 0.9209 - val_loss: 3.9952 - val_accuracy: 0.3947
Epoch 667/1000
177/177 [==============================] - 0s 487us/step - loss: 0.1400 - accuracy: 0.9266 - val_loss: 4.0550 - val_accuracy: 0.3684
Epoch 668/1000
177/177 [==============================] - 0s 686us/step - loss: 0.1327 - accuracy: 0.9322 - val_loss: 3.9749 - 

177/177 [==============================] - 0s 589us/step - loss: 0.1294 - accuracy: 0.9266 - val_loss: 4.1448 - val_accuracy: 0.4079
Epoch 773/1000
177/177 [==============================] - 0s 513us/step - loss: 0.1291 - accuracy: 0.9266 - val_loss: 4.1640 - val_accuracy: 0.4211
Epoch 774/1000
177/177 [==============================] - 0s 692us/step - loss: 0.1255 - accuracy: 0.9322 - val_loss: 4.1851 - val_accuracy: 0.3947
Epoch 775/1000
177/177 [==============================] - 0s 552us/step - loss: 0.1279 - accuracy: 0.9379 - val_loss: 4.2025 - val_accuracy: 0.3816
Epoch 776/1000
177/177 [==============================] - 0s 522us/step - loss: 0.1354 - accuracy: 0.9209 - val_loss: 4.2160 - val_accuracy: 0.3816
Epoch 777/1000
177/177 [==============================] - 0s 517us/step - loss: 0.1389 - accuracy: 0.9379 - val_loss: 4.1655 - val_accuracy: 0.3947
Epoch 778/1000
177/177 [==============================] - 0s 587us/step - loss: 0.1196 - accuracy: 0.9379 - val_loss: 4.2320 - 

177/177 [==============================] - 0s 591us/step - loss: 0.1359 - accuracy: 0.9266 - val_loss: 4.0315 - val_accuracy: 0.3684
Epoch 883/1000
177/177 [==============================] - 0s 438us/step - loss: 0.1235 - accuracy: 0.9379 - val_loss: 4.0262 - val_accuracy: 0.3816
Epoch 884/1000
177/177 [==============================] - 0s 709us/step - loss: 0.1340 - accuracy: 0.9322 - val_loss: 4.0614 - val_accuracy: 0.3816
Epoch 885/1000
177/177 [==============================] - 0s 467us/step - loss: 0.1330 - accuracy: 0.9153 - val_loss: 4.0965 - val_accuracy: 0.3947
Epoch 886/1000
177/177 [==============================] - 0s 426us/step - loss: 0.1305 - accuracy: 0.9266 - val_loss: 4.0538 - val_accuracy: 0.4079
Epoch 887/1000
177/177 [==============================] - 0s 773us/step - loss: 0.1263 - accuracy: 0.9379 - val_loss: 4.0401 - val_accuracy: 0.3947
Epoch 888/1000
177/177 [==============================] - 0s 733us/step - loss: 0.1280 - accuracy: 0.9322 - val_loss: 4.0804 - 

177/177 [==============================] - 0s 410us/step - loss: 0.1641 - accuracy: 0.9266 - val_loss: 3.6119 - val_accuracy: 0.4211
Epoch 993/1000
177/177 [==============================] - 0s 519us/step - loss: 0.1597 - accuracy: 0.9266 - val_loss: 3.6338 - val_accuracy: 0.3553
Epoch 994/1000
177/177 [==============================] - 0s 416us/step - loss: 0.1594 - accuracy: 0.9322 - val_loss: 3.5942 - val_accuracy: 0.4079
Epoch 995/1000
177/177 [==============================] - 0s 406us/step - loss: 0.1635 - accuracy: 0.9153 - val_loss: 3.6179 - val_accuracy: 0.4079
Epoch 996/1000
177/177 [==============================] - 0s 429us/step - loss: 0.1524 - accuracy: 0.9322 - val_loss: 3.6526 - val_accuracy: 0.3816
Epoch 997/1000
177/177 [==============================] - 0s 427us/step - loss: 0.1519 - accuracy: 0.9379 - val_loss: 3.6127 - val_accuracy: 0.3816
Epoch 998/1000
177/177 [==============================] - 0s 405us/step - loss: 0.1493 - accuracy: 0.9379 - val_loss: 3.6065 - 

In [96]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 174us/step
test accuracy: 40.79%


In [34]:
pred2 = model2.predict_classes(X_test)
pred2

array([0, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 2, 0, 0, 2, 0, 1, 0, 2, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 2, 1, 2, 0, 0, 2, 2, 0, 0, 1, 0, 1, 2, 2, 0,
       0, 0, 0, 0, 0, 1, 2, 1, 2, 1, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 1, 2,
       0, 0, 2, 2, 0, 1, 1, 1, 0, 1])

In [35]:
dat2['pred'] = pred2
dat2

,id,test,pred
178,NRS210,2,0
104,Grady1,0,0
79,CFBRSa29,2,0
66,CFBRSa03,0,0
9,217,1,0
...,...,...,...
25,BCH-SA-10,1,1
94,GA15,2,1
244,SR3585,0,1
227,NRS387,2,0


In [36]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.999887,0.000112,8.851192e-07
1,0.625329,0.369782,4.889403e-03
2,0.999098,0.000269,6.335157e-04
3,0.647337,0.331796,2.086646e-02
4,0.613342,0.381903,4.754707e-03
...,...,...,...
71,0.343739,0.652274,3.987430e-03
72,0.490054,0.506295,3.651080e-03
73,0.122960,0.736978,1.400619e-01
74,0.525483,0.082825,3.916920e-01


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p17.csv", index = False,
         header=None)

In [100]:
hist2 = model2.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 453us/step - loss: 0.1283 - accuracy: 0.9266 - val_loss: 4.0972 - val_accuracy: 0.4342
Epoch 2/1000
177/177 [==============================] - 0s 535us/step - loss: 0.1349 - accuracy: 0.9096 - val_loss: 4.0961 - val_accuracy: 0.3947
Epoch 3/1000
177/177 [==============================] - 0s 513us/step - loss: 0.1406 - accuracy: 0.9266 - val_loss: 4.0832 - val_accuracy: 0.4342
Epoch 4/1000
177/177 [==============================] - 0s 439us/step - loss: 0.1325 - accuracy: 0.9379 - val_loss: 4.1128 - val_accuracy: 0.3947
Epoch 5/1000
177/177 [==============================] - 0s 403us/step - loss: 0.1290 - accuracy: 0.9322 - val_loss: 4.0958 - val_accuracy: 0.4079
Epoch 6/1000
177/177 [==============================] - 0s 424us/step - loss: 0.1324 - accuracy: 0.9379 - val_loss: 4.0865 - val_accuracy: 0.4342
Epoch 7/1000
177/177 [==============================] - 0s 421us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 369us/step - loss: 0.1344 - accuracy: 0.9266 - val_loss: 4.3091 - val_accuracy: 0.3947
Epoch 113/1000
177/177 [==============================] - 0s 373us/step - loss: 0.1291 - accuracy: 0.9266 - val_loss: 4.2422 - val_accuracy: 0.4211
Epoch 114/1000
177/177 [==============================] - 0s 718us/step - loss: 0.1331 - accuracy: 0.9379 - val_loss: 4.2300 - val_accuracy: 0.4342
Epoch 115/1000
177/177 [==============================] - 0s 447us/step - loss: 0.1256 - accuracy: 0.9379 - val_loss: 4.2187 - val_accuracy: 0.3947
Epoch 116/1000
177/177 [==============================] - 0s 421us/step - loss: 0.1257 - accuracy: 0.9322 - val_loss: 4.2334 - val_accuracy: 0.3684
Epoch 117/1000
177/177 [==============================] - 0s 548us/step - loss: 0.1271 - accuracy: 0.9322 - val_loss: 4.2390 - val_accuracy: 0.3947
Epoch 118/1000
177/177 [==============================] - 0s 610us/step - loss: 0.1264 - accuracy: 0.9322 - val_

177/177 [==============================] - 0s 537us/step - loss: 0.1342 - accuracy: 0.9379 - val_loss: 4.3106 - val_accuracy: 0.3947
Epoch 223/1000
177/177 [==============================] - 0s 423us/step - loss: 0.1312 - accuracy: 0.9322 - val_loss: 4.3211 - val_accuracy: 0.4079
Epoch 224/1000
177/177 [==============================] - 0s 393us/step - loss: 0.1264 - accuracy: 0.9266 - val_loss: 4.3312 - val_accuracy: 0.4079
Epoch 225/1000
177/177 [==============================] - 0s 590us/step - loss: 0.1265 - accuracy: 0.9266 - val_loss: 4.3178 - val_accuracy: 0.3947
Epoch 226/1000
177/177 [==============================] - 0s 393us/step - loss: 0.1308 - accuracy: 0.9322 - val_loss: 4.3174 - val_accuracy: 0.4342
Epoch 227/1000
177/177 [==============================] - 0s 586us/step - loss: 0.1331 - accuracy: 0.9153 - val_loss: 4.3428 - val_accuracy: 0.4342
Epoch 228/1000
177/177 [==============================] - 0s 409us/step - loss: 0.1307 - accuracy: 0.9322 - val_loss: 4.3868 - 

177/177 [==============================] - 0s 442us/step - loss: 0.1309 - accuracy: 0.9379 - val_loss: 4.5205 - val_accuracy: 0.4079
Epoch 333/1000
177/177 [==============================] - 0s 412us/step - loss: 0.1283 - accuracy: 0.9435 - val_loss: 4.5370 - val_accuracy: 0.3947
Epoch 334/1000
177/177 [==============================] - 0s 411us/step - loss: 0.1300 - accuracy: 0.9209 - val_loss: 4.5239 - val_accuracy: 0.4079
Epoch 335/1000
177/177 [==============================] - 0s 400us/step - loss: 0.1455 - accuracy: 0.9266 - val_loss: 4.5040 - val_accuracy: 0.3947
Epoch 336/1000
177/177 [==============================] - 0s 365us/step - loss: 0.1452 - accuracy: 0.9209 - val_loss: 4.4544 - val_accuracy: 0.3684
Epoch 337/1000
177/177 [==============================] - 0s 385us/step - loss: 0.1400 - accuracy: 0.9379 - val_loss: 4.2699 - val_accuracy: 0.3816
Epoch 338/1000
177/177 [==============================] - 0s 433us/step - loss: 0.1378 - accuracy: 0.9322 - val_loss: 4.3101 - 

177/177 [==============================] - 0s 340us/step - loss: 0.1288 - accuracy: 0.9266 - val_loss: 4.7007 - val_accuracy: 0.3947
Epoch 443/1000
177/177 [==============================] - 0s 351us/step - loss: 0.1241 - accuracy: 0.9209 - val_loss: 4.7155 - val_accuracy: 0.3816
Epoch 444/1000
177/177 [==============================] - 0s 391us/step - loss: 0.1264 - accuracy: 0.9266 - val_loss: 4.7132 - val_accuracy: 0.3816
Epoch 445/1000
177/177 [==============================] - 0s 432us/step - loss: 0.1225 - accuracy: 0.9266 - val_loss: 4.7166 - val_accuracy: 0.4079
Epoch 446/1000
177/177 [==============================] - 0s 449us/step - loss: 0.1236 - accuracy: 0.9379 - val_loss: 4.7230 - val_accuracy: 0.4079
Epoch 447/1000
177/177 [==============================] - 0s 444us/step - loss: 0.1230 - accuracy: 0.9379 - val_loss: 4.7324 - val_accuracy: 0.3947
Epoch 448/1000
177/177 [==============================] - 0s 343us/step - loss: 0.1217 - accuracy: 0.9322 - val_loss: 4.7474 - 

177/177 [==============================] - 0s 426us/step - loss: 0.1243 - accuracy: 0.9379 - val_loss: 4.8343 - val_accuracy: 0.3816
Epoch 553/1000
177/177 [==============================] - 0s 356us/step - loss: 0.1260 - accuracy: 0.9322 - val_loss: 4.8346 - val_accuracy: 0.4079
Epoch 554/1000
177/177 [==============================] - 0s 336us/step - loss: 0.1264 - accuracy: 0.9209 - val_loss: 4.8416 - val_accuracy: 0.3816
Epoch 555/1000
177/177 [==============================] - 0s 331us/step - loss: 0.1231 - accuracy: 0.9266 - val_loss: 4.8432 - val_accuracy: 0.3947
Epoch 556/1000
177/177 [==============================] - 0s 372us/step - loss: 0.1230 - accuracy: 0.9379 - val_loss: 4.8473 - val_accuracy: 0.4079
Epoch 557/1000
177/177 [==============================] - 0s 364us/step - loss: 0.1247 - accuracy: 0.9266 - val_loss: 4.8551 - val_accuracy: 0.4079
Epoch 558/1000
177/177 [==============================] - 0s 381us/step - loss: 0.1283 - accuracy: 0.9096 - val_loss: 4.8602 - 

177/177 [==============================] - 0s 400us/step - loss: 0.1645 - accuracy: 0.9096 - val_loss: 4.7493 - val_accuracy: 0.4079
Epoch 663/1000
177/177 [==============================] - 0s 414us/step - loss: 0.1595 - accuracy: 0.9209 - val_loss: 4.7044 - val_accuracy: 0.4211
Epoch 664/1000
177/177 [==============================] - 0s 385us/step - loss: 0.1820 - accuracy: 0.8983 - val_loss: 4.4873 - val_accuracy: 0.4342
Epoch 665/1000
177/177 [==============================] - 0s 393us/step - loss: 0.1739 - accuracy: 0.9096 - val_loss: 4.5646 - val_accuracy: 0.4342
Epoch 666/1000
177/177 [==============================] - 0s 348us/step - loss: 0.1765 - accuracy: 0.9096 - val_loss: 4.7490 - val_accuracy: 0.4342
Epoch 667/1000
177/177 [==============================] - 0s 369us/step - loss: 0.1812 - accuracy: 0.9153 - val_loss: 4.6194 - val_accuracy: 0.4079
Epoch 668/1000
177/177 [==============================] - 0s 351us/step - loss: 0.2469 - accuracy: 0.8927 - val_loss: 4.6265 - 

177/177 [==============================] - 0s 377us/step - loss: 0.1706 - accuracy: 0.9266 - val_loss: 5.1361 - val_accuracy: 0.3421
Epoch 773/1000
177/177 [==============================] - 0s 385us/step - loss: 0.1594 - accuracy: 0.9153 - val_loss: 5.0297 - val_accuracy: 0.4079
Epoch 774/1000
177/177 [==============================] - 0s 338us/step - loss: 0.1695 - accuracy: 0.9153 - val_loss: 4.9904 - val_accuracy: 0.4342
Epoch 775/1000
177/177 [==============================] - 0s 376us/step - loss: 0.1851 - accuracy: 0.9153 - val_loss: 4.9801 - val_accuracy: 0.4342
Epoch 776/1000
177/177 [==============================] - 0s 354us/step - loss: 0.1594 - accuracy: 0.9096 - val_loss: 5.0172 - val_accuracy: 0.3947
Epoch 777/1000
177/177 [==============================] - 0s 449us/step - loss: 0.1545 - accuracy: 0.9209 - val_loss: 5.0725 - val_accuracy: 0.3947
Epoch 778/1000
177/177 [==============================] - 0s 366us/step - loss: 0.1583 - accuracy: 0.9266 - val_loss: 5.0838 - 

177/177 [==============================] - 0s 349us/step - loss: 0.1466 - accuracy: 0.9266 - val_loss: 5.1267 - val_accuracy: 0.3684
Epoch 883/1000
177/177 [==============================] - 0s 354us/step - loss: 0.1492 - accuracy: 0.9209 - val_loss: 5.1349 - val_accuracy: 0.3816
Epoch 884/1000
177/177 [==============================] - 0s 374us/step - loss: 0.1729 - accuracy: 0.9153 - val_loss: 5.1043 - val_accuracy: 0.4079
Epoch 885/1000
177/177 [==============================] - 0s 375us/step - loss: 0.2121 - accuracy: 0.9153 - val_loss: 5.0640 - val_accuracy: 0.4079
Epoch 886/1000
177/177 [==============================] - 0s 337us/step - loss: 0.1835 - accuracy: 0.9040 - val_loss: 5.1261 - val_accuracy: 0.3684
Epoch 887/1000
177/177 [==============================] - 0s 370us/step - loss: 0.1606 - accuracy: 0.9266 - val_loss: 5.2207 - val_accuracy: 0.3553
Epoch 888/1000
177/177 [==============================] - 0s 366us/step - loss: 0.1696 - accuracy: 0.8927 - val_loss: 5.2069 - 

177/177 [==============================] - 0s 358us/step - loss: 0.1415 - accuracy: 0.9322 - val_loss: 5.4041 - val_accuracy: 0.3553
Epoch 993/1000
177/177 [==============================] - 0s 366us/step - loss: 0.1427 - accuracy: 0.9322 - val_loss: 5.4168 - val_accuracy: 0.3684
Epoch 994/1000
177/177 [==============================] - 0s 347us/step - loss: 0.1436 - accuracy: 0.9153 - val_loss: 5.4181 - val_accuracy: 0.3684
Epoch 995/1000
177/177 [==============================] - 0s 373us/step - loss: 0.1392 - accuracy: 0.9379 - val_loss: 5.4293 - val_accuracy: 0.3421
Epoch 996/1000
177/177 [==============================] - 0s 356us/step - loss: 0.1430 - accuracy: 0.9266 - val_loss: 5.4470 - val_accuracy: 0.3421
Epoch 997/1000
177/177 [==============================] - 0s 370us/step - loss: 0.1440 - accuracy: 0.9322 - val_loss: 5.4266 - val_accuracy: 0.3421
Epoch 998/1000
177/177 [==============================] - 0s 360us/step - loss: 0.1418 - accuracy: 0.9322 - val_loss: 5.4343 - 

In [101]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 92.70%


In [30]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [31]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [32]:
y_prob2 = df_proba2[df_proba2['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.99886900e-01, 1.12307134e-04, 8.85119160e-07],
       [6.25329100e-01, 3.69781550e-01, 4.88940350e-03],
       [9.99097700e-01, 2.68757460e-04, 6.33515650e-04],
       [6.47337500e-01, 3.31795960e-01, 2.08664600e-02],
       [6.13342200e-01, 3.81903080e-01, 4.75470700e-03],
       [1.14109650e-06, 3.26310870e-06, 9.99995600e-01],
       [9.98627900e-01, 1.53496410e-04, 1.21868300e-03],
       [3.21461400e-01, 6.77547750e-01, 9.90918400e-04],
       [3.57540820e-13, 4.83921900e-08, 1.00000000e+00],
       [5.27881560e-01, 8.41802700e-02, 3.87938170e-01],
       [7.30024040e-01, 2.03981500e-01, 6.59944600e-02],
       [1.60616200e-02, 1.81242390e-03, 9.82126000e-01],
       [9.78584000e-01, 2.05800200e-02, 8.35929100e-04],
       [9.89230450e-01, 4.53780800e-03, 6.23174340e-03],
       [4.26795860e-02, 5.77596130e-04, 9.56742800e-01],
       [9.77096740e-01, 2.28959490e-02, 7.36804260e-06],
       [6.26219100e-05, 9.99937060e-01, 3.59197430e-07],
       [5.15714200e-01, 4.81643

In [35]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.584633179871275

In [36]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.584633179871275

In [37]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [38]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [39]:
dat3

,id,test
231,NY360,2
92,EUH25,2
91,EUH15,2
203,NRS241,0
242,SR2852,2
...,...,...
26,BCH-SA-11,0
111,NRS022,1
96,GA27,2
129,NRS102,1


In [40]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [41]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model3.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 988us/step - loss: 1.1454 - accuracy: 0.4463 - val_loss: 1.3051 - val_accuracy: 0.3816
Epoch 2/1000
177/177 [==============================] - 0s 409us/step - loss: 1.0933 - accuracy: 0.5537 - val_loss: 1.1927 - val_accuracy: 0.3421
Epoch 3/1000
177/177 [==============================] - 0s 674us/step - loss: 0.9714 - accuracy: 0.5424 - val_loss: 1.1744 - val_accuracy: 0.4342
Epoch 4/1000
177/177 [==============================] - 0s 683us/step - loss: 0.8973 - accuracy: 0.6158 - val_loss: 1.1803 - val_accuracy: 0.4211
Epoch 5/1000
177/177 [==============================] - 0s 537us/step - loss: 0.8731 - accuracy: 0.6158 - val_loss: 1.1901 - val_accuracy: 0.4342
Epoch 6/1000
177/177 [==============================] - 0s 547us/step - loss: 0.8222 - accuracy: 0.6497 - val_loss: 1.2001 - val_accuracy: 0.4079
Epoch 7/1000
177/177 [==============================] - 0s 869us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 354us/step - loss: 0.2503 - accuracy: 0.9153 - val_loss: 2.2876 - val_accuracy: 0.4079
Epoch 113/1000
177/177 [==============================] - 0s 440us/step - loss: 0.2791 - accuracy: 0.9435 - val_loss: 2.2609 - val_accuracy: 0.3947
Epoch 114/1000
177/177 [==============================] - 0s 362us/step - loss: 0.2801 - accuracy: 0.9266 - val_loss: 2.3822 - val_accuracy: 0.4079
Epoch 115/1000
177/177 [==============================] - 0s 595us/step - loss: 0.2723 - accuracy: 0.9266 - val_loss: 2.3008 - val_accuracy: 0.4079
Epoch 116/1000
177/177 [==============================] - 0s 641us/step - loss: 0.2366 - accuracy: 0.9379 - val_loss: 2.2887 - val_accuracy: 0.4342
Epoch 117/1000
177/177 [==============================] - 0s 521us/step - loss: 0.2369 - accuracy: 0.9209 - val_loss: 2.3400 - val_accuracy: 0.4342
Epoch 118/1000
177/177 [==============================] - 0s 456us/step - loss: 0.2464 - accuracy: 0.9096 - val_

177/177 [==============================] - 0s 587us/step - loss: 0.1518 - accuracy: 0.9435 - val_loss: 2.8500 - val_accuracy: 0.3947
Epoch 223/1000
177/177 [==============================] - 0s 623us/step - loss: 0.1566 - accuracy: 0.9379 - val_loss: 2.8169 - val_accuracy: 0.4211
Epoch 224/1000
177/177 [==============================] - 0s 332us/step - loss: 0.1531 - accuracy: 0.9322 - val_loss: 2.8488 - val_accuracy: 0.3947
Epoch 225/1000
177/177 [==============================] - 0s 423us/step - loss: 0.1572 - accuracy: 0.9435 - val_loss: 2.8610 - val_accuracy: 0.3947
Epoch 226/1000
177/177 [==============================] - 0s 382us/step - loss: 0.1531 - accuracy: 0.9435 - val_loss: 2.8757 - val_accuracy: 0.3816
Epoch 227/1000
177/177 [==============================] - 0s 854us/step - loss: 0.1501 - accuracy: 0.9435 - val_loss: 2.8452 - val_accuracy: 0.3947
Epoch 228/1000
177/177 [==============================] - 0s 369us/step - loss: 0.1534 - accuracy: 0.9379 - val_loss: 2.8232 - 

177/177 [==============================] - 0s 424us/step - loss: 0.1334 - accuracy: 0.9379 - val_loss: 3.2951 - val_accuracy: 0.3816
Epoch 333/1000
177/177 [==============================] - 0s 861us/step - loss: 0.1428 - accuracy: 0.9548 - val_loss: 3.3065 - val_accuracy: 0.3816
Epoch 334/1000
177/177 [==============================] - 0s 758us/step - loss: 0.1441 - accuracy: 0.9266 - val_loss: 3.3468 - val_accuracy: 0.3947
Epoch 335/1000
177/177 [==============================] - 0s 389us/step - loss: 0.1359 - accuracy: 0.9266 - val_loss: 3.3036 - val_accuracy: 0.3816
Epoch 336/1000
177/177 [==============================] - 0s 436us/step - loss: 0.1268 - accuracy: 0.9548 - val_loss: 3.3293 - val_accuracy: 0.3816
Epoch 337/1000
177/177 [==============================] - 0s 458us/step - loss: 0.1213 - accuracy: 0.9492 - val_loss: 3.3303 - val_accuracy: 0.3816
Epoch 338/1000
177/177 [==============================] - 0s 463us/step - loss: 0.1234 - accuracy: 0.9379 - val_loss: 3.2965 - 

177/177 [==============================] - 0s 423us/step - loss: 0.1290 - accuracy: 0.9435 - val_loss: 3.3922 - val_accuracy: 0.3684
Epoch 443/1000
177/177 [==============================] - 0s 654us/step - loss: 0.1111 - accuracy: 0.9435 - val_loss: 3.4764 - val_accuracy: 0.3947
Epoch 444/1000
177/177 [==============================] - 0s 368us/step - loss: 0.1266 - accuracy: 0.9322 - val_loss: 3.5025 - val_accuracy: 0.3947
Epoch 445/1000
177/177 [==============================] - 0s 458us/step - loss: 0.1079 - accuracy: 0.9435 - val_loss: 3.3996 - val_accuracy: 0.3684
Epoch 446/1000
177/177 [==============================] - 0s 426us/step - loss: 0.1272 - accuracy: 0.9492 - val_loss: 3.3671 - val_accuracy: 0.3816
Epoch 447/1000
177/177 [==============================] - 0s 430us/step - loss: 0.1419 - accuracy: 0.9379 - val_loss: 3.5144 - val_accuracy: 0.3684
Epoch 448/1000
177/177 [==============================] - 0s 411us/step - loss: 0.1201 - accuracy: 0.9492 - val_loss: 3.4648 - 

177/177 [==============================] - 0s 379us/step - loss: 0.1022 - accuracy: 0.9492 - val_loss: 3.6503 - val_accuracy: 0.3816
Epoch 553/1000
177/177 [==============================] - 0s 401us/step - loss: 0.0985 - accuracy: 0.9492 - val_loss: 3.6321 - val_accuracy: 0.3947
Epoch 554/1000
177/177 [==============================] - 0s 410us/step - loss: 0.1154 - accuracy: 0.9379 - val_loss: 3.6401 - val_accuracy: 0.4079
Epoch 555/1000
177/177 [==============================] - 0s 515us/step - loss: 0.1046 - accuracy: 0.9435 - val_loss: 3.6755 - val_accuracy: 0.3947
Epoch 556/1000
177/177 [==============================] - 0s 548us/step - loss: 0.1086 - accuracy: 0.9492 - val_loss: 3.6790 - val_accuracy: 0.3816
Epoch 557/1000
177/177 [==============================] - 0s 513us/step - loss: 0.0986 - accuracy: 0.9492 - val_loss: 3.6704 - val_accuracy: 0.3816
Epoch 558/1000
177/177 [==============================] - 0s 533us/step - loss: 0.1082 - accuracy: 0.9322 - val_loss: 3.6329 - 

177/177 [==============================] - 0s 559us/step - loss: 0.0907 - accuracy: 0.9605 - val_loss: 3.8887 - val_accuracy: 0.3684
Epoch 663/1000
177/177 [==============================] - 0s 588us/step - loss: 0.1170 - accuracy: 0.9379 - val_loss: 3.8674 - val_accuracy: 0.3947
Epoch 664/1000
177/177 [==============================] - 0s 671us/step - loss: 0.0949 - accuracy: 0.9548 - val_loss: 3.8618 - val_accuracy: 0.3816
Epoch 665/1000
177/177 [==============================] - 0s 413us/step - loss: 0.0985 - accuracy: 0.9492 - val_loss: 3.7800 - val_accuracy: 0.3816
Epoch 666/1000
177/177 [==============================] - 0s 389us/step - loss: 0.0988 - accuracy: 0.9435 - val_loss: 3.8285 - val_accuracy: 0.3816
Epoch 667/1000
177/177 [==============================] - 0s 426us/step - loss: 0.0939 - accuracy: 0.9435 - val_loss: 3.8528 - val_accuracy: 0.3684
Epoch 668/1000
177/177 [==============================] - 0s 443us/step - loss: 0.0999 - accuracy: 0.9492 - val_loss: 3.8838 - 

177/177 [==============================] - 0s 605us/step - loss: 0.1036 - accuracy: 0.9435 - val_loss: 4.0845 - val_accuracy: 0.3684
Epoch 773/1000
177/177 [==============================] - 0s 592us/step - loss: 0.1491 - accuracy: 0.9435 - val_loss: 4.1076 - val_accuracy: 0.3816
Epoch 774/1000
177/177 [==============================] - 0s 391us/step - loss: 0.1053 - accuracy: 0.9435 - val_loss: 4.0062 - val_accuracy: 0.3684
Epoch 775/1000
177/177 [==============================] - 0s 458us/step - loss: 0.1360 - accuracy: 0.9435 - val_loss: 3.8803 - val_accuracy: 0.3816
Epoch 776/1000
177/177 [==============================] - 0s 452us/step - loss: 0.1014 - accuracy: 0.9435 - val_loss: 3.9204 - val_accuracy: 0.3816
Epoch 777/1000
177/177 [==============================] - 0s 477us/step - loss: 0.1071 - accuracy: 0.9379 - val_loss: 4.0813 - val_accuracy: 0.3816
Epoch 778/1000
177/177 [==============================] - 0s 458us/step - loss: 0.1227 - accuracy: 0.9548 - val_loss: 4.1451 - 

177/177 [==============================] - 0s 505us/step - loss: 0.0921 - accuracy: 0.9379 - val_loss: 4.1762 - val_accuracy: 0.3684
Epoch 883/1000
177/177 [==============================] - 0s 380us/step - loss: 0.0937 - accuracy: 0.9492 - val_loss: 4.1854 - val_accuracy: 0.3684
Epoch 884/1000
177/177 [==============================] - 0s 376us/step - loss: 0.1021 - accuracy: 0.9379 - val_loss: 4.1895 - val_accuracy: 0.3684
Epoch 885/1000
177/177 [==============================] - 0s 350us/step - loss: 0.1036 - accuracy: 0.9492 - val_loss: 4.2098 - val_accuracy: 0.3816
Epoch 886/1000
177/177 [==============================] - 0s 360us/step - loss: 0.0896 - accuracy: 0.9548 - val_loss: 4.2088 - val_accuracy: 0.3816
Epoch 887/1000
177/177 [==============================] - 0s 352us/step - loss: 0.0985 - accuracy: 0.9379 - val_loss: 4.1468 - val_accuracy: 0.3816
Epoch 888/1000
177/177 [==============================] - 0s 370us/step - loss: 0.0969 - accuracy: 0.9492 - val_loss: 4.1254 - 

177/177 [==============================] - 0s 470us/step - loss: 0.0974 - accuracy: 0.9548 - val_loss: 4.3274 - val_accuracy: 0.3816
Epoch 993/1000
177/177 [==============================] - 0s 463us/step - loss: 0.0920 - accuracy: 0.9492 - val_loss: 4.3175 - val_accuracy: 0.3816
Epoch 994/1000
177/177 [==============================] - 0s 425us/step - loss: 0.0869 - accuracy: 0.9548 - val_loss: 4.3459 - val_accuracy: 0.3816
Epoch 995/1000
177/177 [==============================] - 0s 1ms/step - loss: 0.0909 - accuracy: 0.9492 - val_loss: 4.3301 - val_accuracy: 0.3684
Epoch 996/1000
177/177 [==============================] - 0s 876us/step - loss: 0.0884 - accuracy: 0.9435 - val_loss: 4.2947 - val_accuracy: 0.3684
Epoch 997/1000
177/177 [==============================] - 0s 483us/step - loss: 0.0892 - accuracy: 0.9548 - val_loss: 4.3246 - val_accuracy: 0.3684
Epoch 998/1000
177/177 [==============================] - 0s 528us/step - loss: 0.0860 - accuracy: 0.9435 - val_loss: 4.3018 - va

In [137]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 143us/step
test accuracy: 38.16%


In [47]:
pred3 = model3.predict_classes(X_test)
pred3

array([2, 0, 2, 0, 2, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 1, 0, 0, 2,
       0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 2, 0, 1, 1, 2, 0, 2,
       0, 0, 0, 1, 0, 1, 0, 1, 2, 0, 0, 1, 2, 1, 1, 0, 1, 2, 0, 0, 1, 2,
       2, 0, 2, 1, 1, 1, 2, 2, 0, 0])

In [48]:
dat3['pred'] = pred3
dat3

,id,test,pred
231,NY360,2,2
92,EUH25,2,0
91,EUH15,2,2
203,NRS241,0,0
242,SR2852,2,2
...,...,...,...
26,BCH-SA-11,0,1
111,NRS022,1,2
96,GA27,2,2
129,NRS102,1,0


In [49]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,2.165526e-02,4.848140e-01,0.493531
1,9.986388e-01,1.245148e-03,0.000116
2,9.227520e-04,1.424882e-02,0.984828
3,8.374333e-01,1.614128e-01,0.001154
4,3.976981e-09,5.145955e-10,1.000000
...,...,...,...
71,8.656543e-04,9.905151e-01,0.008619
72,4.509500e-01,2.885678e-03,0.546164
73,1.693363e-08,1.278872e-05,0.999987
74,8.764913e-01,1.235003e-01,0.000008


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p17.csv", index = False,
         header=None)

In [141]:
hist3 = model3.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 371us/step - loss: 0.0971 - accuracy: 0.9379 - val_loss: 3.8061 - val_accuracy: 0.3684
Epoch 2/1000
177/177 [==============================] - 0s 428us/step - loss: 0.0941 - accuracy: 0.9435 - val_loss: 3.8078 - val_accuracy: 0.3816
Epoch 3/1000
177/177 [==============================] - 0s 359us/step - loss: 0.0915 - accuracy: 0.9548 - val_loss: 3.8207 - val_accuracy: 0.4079
Epoch 4/1000
177/177 [==============================] - 0s 363us/step - loss: 0.0985 - accuracy: 0.9435 - val_loss: 3.8898 - val_accuracy: 0.3947
Epoch 5/1000
177/177 [==============================] - 0s 379us/step - loss: 0.0889 - accuracy: 0.9548 - val_loss: 3.8529 - val_accuracy: 0.3816
Epoch 6/1000
177/177 [==============================] - 0s 330us/step - loss: 0.0951 - accuracy: 0.9492 - val_loss: 3.7904 - val_accuracy: 0.3816
Epoch 7/1000
177/177 [==============================] - 0s 376us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 380us/step - loss: 0.0883 - accuracy: 0.9492 - val_loss: 3.9712 - val_accuracy: 0.3947
Epoch 113/1000
177/177 [==============================] - 0s 343us/step - loss: 0.0943 - accuracy: 0.9492 - val_loss: 3.9981 - val_accuracy: 0.3816
Epoch 114/1000
177/177 [==============================] - 0s 430us/step - loss: 0.0950 - accuracy: 0.9492 - val_loss: 3.9693 - val_accuracy: 0.3816
Epoch 115/1000
177/177 [==============================] - 0s 369us/step - loss: 0.1067 - accuracy: 0.9379 - val_loss: 3.9294 - val_accuracy: 0.3816
Epoch 116/1000
177/177 [==============================] - 0s 339us/step - loss: 0.0871 - accuracy: 0.9492 - val_loss: 3.9312 - val_accuracy: 0.3816
Epoch 117/1000
177/177 [==============================] - 0s 417us/step - loss: 0.0922 - accuracy: 0.9492 - val_loss: 3.9823 - val_accuracy: 0.3816
Epoch 118/1000
177/177 [==============================] - 0s 530us/step - loss: 0.0938 - accuracy: 0.9548 - val_

177/177 [==============================] - 0s 356us/step - loss: 0.0923 - accuracy: 0.9492 - val_loss: 3.9280 - val_accuracy: 0.3816
Epoch 223/1000
177/177 [==============================] - 0s 370us/step - loss: 0.0898 - accuracy: 0.9492 - val_loss: 3.9989 - val_accuracy: 0.3816
Epoch 224/1000
177/177 [==============================] - 0s 359us/step - loss: 0.0928 - accuracy: 0.9492 - val_loss: 4.0471 - val_accuracy: 0.3816
Epoch 225/1000
177/177 [==============================] - 0s 368us/step - loss: 0.1095 - accuracy: 0.9435 - val_loss: 4.0228 - val_accuracy: 0.3816
Epoch 226/1000
177/177 [==============================] - 0s 348us/step - loss: 0.0882 - accuracy: 0.9492 - val_loss: 3.9999 - val_accuracy: 0.3816
Epoch 227/1000
177/177 [==============================] - 0s 402us/step - loss: 0.1101 - accuracy: 0.9322 - val_loss: 3.9558 - val_accuracy: 0.3947
Epoch 228/1000
177/177 [==============================] - 0s 397us/step - loss: 0.0992 - accuracy: 0.9379 - val_loss: 3.9726 - 

177/177 [==============================] - 0s 364us/step - loss: 0.1031 - accuracy: 0.9492 - val_loss: 4.1173 - val_accuracy: 0.3553
Epoch 333/1000
177/177 [==============================] - 0s 434us/step - loss: 0.0959 - accuracy: 0.9379 - val_loss: 4.1447 - val_accuracy: 0.3684
Epoch 334/1000
177/177 [==============================] - 0s 409us/step - loss: 0.1108 - accuracy: 0.9379 - val_loss: 4.1022 - val_accuracy: 0.3553
Epoch 335/1000
177/177 [==============================] - 0s 433us/step - loss: 0.0894 - accuracy: 0.9379 - val_loss: 4.1254 - val_accuracy: 0.3553
Epoch 336/1000
177/177 [==============================] - 0s 373us/step - loss: 0.0913 - accuracy: 0.9492 - val_loss: 4.1355 - val_accuracy: 0.3684
Epoch 337/1000
177/177 [==============================] - 0s 351us/step - loss: 0.0824 - accuracy: 0.9605 - val_loss: 4.1459 - val_accuracy: 0.3684
Epoch 338/1000
177/177 [==============================] - 0s 347us/step - loss: 0.0889 - accuracy: 0.9492 - val_loss: 4.1204 - 

177/177 [==============================] - 0s 362us/step - loss: 0.0966 - accuracy: 0.9548 - val_loss: 4.1705 - val_accuracy: 0.3684
Epoch 443/1000
177/177 [==============================] - 0s 353us/step - loss: 0.0884 - accuracy: 0.9492 - val_loss: 4.2049 - val_accuracy: 0.3684
Epoch 444/1000
177/177 [==============================] - 0s 407us/step - loss: 0.0926 - accuracy: 0.9435 - val_loss: 4.2829 - val_accuracy: 0.3684
Epoch 445/1000
177/177 [==============================] - 0s 399us/step - loss: 0.0881 - accuracy: 0.9492 - val_loss: 4.2636 - val_accuracy: 0.3553
Epoch 446/1000
177/177 [==============================] - 0s 371us/step - loss: 0.0847 - accuracy: 0.9605 - val_loss: 4.1624 - val_accuracy: 0.3553
Epoch 447/1000
177/177 [==============================] - 0s 356us/step - loss: 0.0861 - accuracy: 0.9435 - val_loss: 4.1471 - val_accuracy: 0.3684
Epoch 448/1000
177/177 [==============================] - 0s 335us/step - loss: 0.0827 - accuracy: 0.9548 - val_loss: 4.2156 - 

177/177 [==============================] - 0s 380us/step - loss: 0.0910 - accuracy: 0.9435 - val_loss: 4.2965 - val_accuracy: 0.3553
Epoch 553/1000
177/177 [==============================] - 0s 377us/step - loss: 0.0877 - accuracy: 0.9548 - val_loss: 4.2646 - val_accuracy: 0.3684
Epoch 554/1000
177/177 [==============================] - 0s 323us/step - loss: 0.0936 - accuracy: 0.9492 - val_loss: 4.3289 - val_accuracy: 0.3684
Epoch 555/1000
177/177 [==============================] - 0s 440us/step - loss: 0.0961 - accuracy: 0.9492 - val_loss: 4.3370 - val_accuracy: 0.3553
Epoch 556/1000
177/177 [==============================] - 0s 497us/step - loss: 0.0843 - accuracy: 0.9492 - val_loss: 4.3586 - val_accuracy: 0.3553
Epoch 557/1000
177/177 [==============================] - 0s 598us/step - loss: 0.0947 - accuracy: 0.9492 - val_loss: 4.3222 - val_accuracy: 0.3684
Epoch 558/1000
177/177 [==============================] - 0s 672us/step - loss: 0.0966 - accuracy: 0.9548 - val_loss: 4.2841 - 

177/177 [==============================] - 0s 362us/step - loss: 0.0869 - accuracy: 0.9492 - val_loss: 4.4937 - val_accuracy: 0.3553
Epoch 663/1000
177/177 [==============================] - 0s 359us/step - loss: 0.0774 - accuracy: 0.9605 - val_loss: 4.4723 - val_accuracy: 0.3553
Epoch 664/1000
177/177 [==============================] - 0s 363us/step - loss: 0.0855 - accuracy: 0.9548 - val_loss: 4.4235 - val_accuracy: 0.3684
Epoch 665/1000
177/177 [==============================] - 0s 351us/step - loss: 0.0914 - accuracy: 0.9492 - val_loss: 4.3733 - val_accuracy: 0.3553
Epoch 666/1000
177/177 [==============================] - 0s 352us/step - loss: 0.0839 - accuracy: 0.9492 - val_loss: 4.4227 - val_accuracy: 0.3553
Epoch 667/1000
177/177 [==============================] - 0s 344us/step - loss: 0.0904 - accuracy: 0.9492 - val_loss: 4.5024 - val_accuracy: 0.3553
Epoch 668/1000
177/177 [==============================] - 0s 348us/step - loss: 0.0826 - accuracy: 0.9548 - val_loss: 4.5046 - 

177/177 [==============================] - 0s 443us/step - loss: 0.0826 - accuracy: 0.9492 - val_loss: 4.4811 - val_accuracy: 0.3553
Epoch 773/1000
177/177 [==============================] - 0s 389us/step - loss: 0.0770 - accuracy: 0.9492 - val_loss: 4.5450 - val_accuracy: 0.3553
Epoch 774/1000
177/177 [==============================] - 0s 365us/step - loss: 0.0819 - accuracy: 0.9548 - val_loss: 4.6033 - val_accuracy: 0.3553
Epoch 775/1000
177/177 [==============================] - 0s 371us/step - loss: 0.0826 - accuracy: 0.9492 - val_loss: 4.5530 - val_accuracy: 0.3553
Epoch 776/1000
177/177 [==============================] - 0s 373us/step - loss: 0.0799 - accuracy: 0.9435 - val_loss: 4.5022 - val_accuracy: 0.3553
Epoch 777/1000
177/177 [==============================] - 0s 392us/step - loss: 0.0819 - accuracy: 0.9492 - val_loss: 4.4822 - val_accuracy: 0.3553
Epoch 778/1000
177/177 [==============================] - 0s 399us/step - loss: 0.0796 - accuracy: 0.9435 - val_loss: 4.5219 - 

177/177 [==============================] - 0s 368us/step - loss: 0.0819 - accuracy: 0.9548 - val_loss: 4.5978 - val_accuracy: 0.3553
Epoch 883/1000
177/177 [==============================] - 0s 346us/step - loss: 0.0806 - accuracy: 0.9548 - val_loss: 4.5996 - val_accuracy: 0.3684
Epoch 884/1000
177/177 [==============================] - 0s 330us/step - loss: 0.0796 - accuracy: 0.9548 - val_loss: 4.6150 - val_accuracy: 0.3684
Epoch 885/1000
177/177 [==============================] - 0s 360us/step - loss: 0.0786 - accuracy: 0.9605 - val_loss: 4.6436 - val_accuracy: 0.3553
Epoch 886/1000
177/177 [==============================] - 0s 375us/step - loss: 0.0796 - accuracy: 0.9492 - val_loss: 4.6254 - val_accuracy: 0.3553
Epoch 887/1000
177/177 [==============================] - 0s 440us/step - loss: 0.0837 - accuracy: 0.9435 - val_loss: 4.6188 - val_accuracy: 0.3553
Epoch 888/1000
177/177 [==============================] - 0s 349us/step - loss: 0.0819 - accuracy: 0.9492 - val_loss: 4.5724 - 

177/177 [==============================] - 0s 331us/step - loss: 0.0815 - accuracy: 0.9492 - val_loss: 4.6436 - val_accuracy: 0.3553
Epoch 993/1000
177/177 [==============================] - 0s 325us/step - loss: 0.0824 - accuracy: 0.9492 - val_loss: 4.6299 - val_accuracy: 0.3553
Epoch 994/1000
177/177 [==============================] - 0s 419us/step - loss: 0.0910 - accuracy: 0.9492 - val_loss: 4.6561 - val_accuracy: 0.3553
Epoch 995/1000
177/177 [==============================] - 0s 364us/step - loss: 0.0808 - accuracy: 0.9492 - val_loss: 4.6554 - val_accuracy: 0.3553
Epoch 996/1000
177/177 [==============================] - 0s 428us/step - loss: 0.0809 - accuracy: 0.9548 - val_loss: 4.6787 - val_accuracy: 0.3684
Epoch 997/1000
177/177 [==============================] - 0s 379us/step - loss: 0.0834 - accuracy: 0.9548 - val_loss: 4.7332 - val_accuracy: 0.3553
Epoch 998/1000
177/177 [==============================] - 0s 347us/step - loss: 0.0793 - accuracy: 0.9492 - val_loss: 4.7027 - 

In [142]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 94.90%


In [42]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [43]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [44]:
y_prob3 = df_proba3[df_proba3['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[2.16552600e-02, 4.84814000e-01, 4.93530720e-01],
       [9.98638800e-01, 1.24514820e-03, 1.16102700e-04],
       [9.22751960e-04, 1.42488240e-02, 9.84828400e-01],
       [8.37433340e-01, 1.61412790e-01, 1.15391890e-03],
       [3.97698050e-09, 5.14595500e-10, 1.00000000e+00],
       [9.01291850e-01, 1.92286820e-07, 9.87079900e-02],
       [3.25990000e-01, 6.55076200e-01, 1.89337620e-02],
       [1.73334870e-06, 2.38342380e-07, 9.99998000e-01],
       [8.72527360e-01, 1.47177640e-03, 1.26000850e-01],
       [9.99970100e-01, 5.34381340e-07, 2.94912240e-05],
       [3.24331430e-01, 6.52845100e-01, 2.28235140e-02],
       [9.92592100e-01, 7.40791440e-03, 2.62094240e-09],
       [8.79312040e-01, 1.19599484e-01, 1.08843720e-03],
       [8.72527360e-01, 1.47177640e-03, 1.26000850e-01],
       [9.90747900e-01, 8.39727650e-05, 9.16815550e-03],
       [1.72630510e-02, 9.60304560e-01, 2.24323940e-02],
       [2.56626160e-01, 7.42894600e-01, 4.79310580e-04],
       [5.69714400e-06, 1.21308

In [45]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.5564402528688243

In [46]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.5564402528688243

In [47]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [48]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [49]:
dat4

,id,test
236,SR1129,2
31,CA105,2
155,NRS175,1
92,EUH25,2
122,NRS070,2
...,...,...
235,SR1065,0
24,BCH-SA-09,0
104,Grady1,0
213,NRS254,2


In [50]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [51]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [58]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model4.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 2ms/step - loss: 1.2431 - accuracy: 0.4068 - val_loss: 1.2522 - val_accuracy: 0.3684
Epoch 2/1000
177/177 [==============================] - 0s 545us/step - loss: 1.0380 - accuracy: 0.5198 - val_loss: 1.1039 - val_accuracy: 0.4211
Epoch 3/1000
177/177 [==============================] - 0s 380us/step - loss: 0.9344 - accuracy: 0.5876 - val_loss: 1.1541 - val_accuracy: 0.3947
Epoch 4/1000
177/177 [==============================] - 0s 439us/step - loss: 0.9002 - accuracy: 0.5876 - val_loss: 1.2015 - val_accuracy: 0.4079
Epoch 5/1000
177/177 [==============================] - 0s 592us/step - loss: 0.8770 - accuracy: 0.6328 - val_loss: 1.2435 - val_accuracy: 0.4079
Epoch 6/1000
177/177 [==============================] - 0s 795us/step - loss: 0.8352 - accuracy: 0.6384 - val_loss: 1.3072 - val_accuracy: 0.4211
Epoch 7/1000
177/177 [==============================] - 0s 394us/step - loss: 0.8

Epoch 112/1000
177/177 [==============================] - 0s 705us/step - loss: 0.2074 - accuracy: 0.9153 - val_loss: 3.3705 - val_accuracy: 0.3684
Epoch 113/1000
177/177 [==============================] - 0s 516us/step - loss: 0.2206 - accuracy: 0.9209 - val_loss: 3.2105 - val_accuracy: 0.4211
Epoch 114/1000
177/177 [==============================] - 0s 383us/step - loss: 0.2303 - accuracy: 0.8927 - val_loss: 3.2534 - val_accuracy: 0.3421
Epoch 115/1000
177/177 [==============================] - 0s 429us/step - loss: 0.2774 - accuracy: 0.8927 - val_loss: 3.3098 - val_accuracy: 0.3816
Epoch 116/1000
177/177 [==============================] - 0s 556us/step - loss: 0.2649 - accuracy: 0.8927 - val_loss: 3.2380 - val_accuracy: 0.3684
Epoch 117/1000
177/177 [==============================] - 0s 714us/step - loss: 0.2411 - accuracy: 0.8927 - val_loss: 3.3285 - val_accuracy: 0.3684
Epoch 118/1000
177/177 [==============================] - 0s 344us/step - loss: 0.2469 - accuracy: 0.8870 - val_

177/177 [==============================] - 0s 402us/step - loss: 0.1428 - accuracy: 0.9435 - val_loss: 4.0712 - val_accuracy: 0.3553
Epoch 223/1000
177/177 [==============================] - 0s 750us/step - loss: 0.1596 - accuracy: 0.9209 - val_loss: 4.0750 - val_accuracy: 0.3684
Epoch 224/1000
177/177 [==============================] - 0s 550us/step - loss: 0.1575 - accuracy: 0.9266 - val_loss: 4.1155 - val_accuracy: 0.3421
Epoch 225/1000
177/177 [==============================] - 0s 464us/step - loss: 0.1643 - accuracy: 0.9379 - val_loss: 4.0783 - val_accuracy: 0.3947
Epoch 226/1000
177/177 [==============================] - 0s 409us/step - loss: 0.1518 - accuracy: 0.9322 - val_loss: 4.1648 - val_accuracy: 0.3947
Epoch 227/1000
177/177 [==============================] - 0s 407us/step - loss: 0.1475 - accuracy: 0.9266 - val_loss: 4.1391 - val_accuracy: 0.3684
Epoch 228/1000
177/177 [==============================] - 0s 674us/step - loss: 0.1524 - accuracy: 0.9322 - val_loss: 4.0824 - 

177/177 [==============================] - 0s 629us/step - loss: 0.1624 - accuracy: 0.9266 - val_loss: 4.3945 - val_accuracy: 0.3684
Epoch 333/1000
177/177 [==============================] - 0s 628us/step - loss: 0.1463 - accuracy: 0.9266 - val_loss: 4.4228 - val_accuracy: 0.3553
Epoch 334/1000
177/177 [==============================] - 0s 658us/step - loss: 0.1563 - accuracy: 0.9322 - val_loss: 4.4152 - val_accuracy: 0.3553
Epoch 335/1000
177/177 [==============================] - 0s 670us/step - loss: 0.1634 - accuracy: 0.9266 - val_loss: 4.3965 - val_accuracy: 0.3947
Epoch 336/1000
177/177 [==============================] - 0s 673us/step - loss: 0.1603 - accuracy: 0.9153 - val_loss: 4.3366 - val_accuracy: 0.3553
Epoch 337/1000
177/177 [==============================] - 0s 694us/step - loss: 0.1547 - accuracy: 0.9266 - val_loss: 4.4494 - val_accuracy: 0.3684
Epoch 338/1000
177/177 [==============================] - 0s 602us/step - loss: 0.1552 - accuracy: 0.9322 - val_loss: 4.4764 - 

177/177 [==============================] - 0s 488us/step - loss: 0.1777 - accuracy: 0.9209 - val_loss: 4.7385 - val_accuracy: 0.4079
Epoch 443/1000
177/177 [==============================] - 0s 515us/step - loss: 0.1608 - accuracy: 0.9266 - val_loss: 4.5388 - val_accuracy: 0.3947
Epoch 444/1000
177/177 [==============================] - 0s 464us/step - loss: 0.1426 - accuracy: 0.9209 - val_loss: 4.7270 - val_accuracy: 0.4211
Epoch 445/1000
177/177 [==============================] - 0s 873us/step - loss: 0.1597 - accuracy: 0.9322 - val_loss: 4.7118 - val_accuracy: 0.3553
Epoch 446/1000
177/177 [==============================] - 0s 488us/step - loss: 0.1474 - accuracy: 0.9322 - val_loss: 4.6726 - val_accuracy: 0.3816
Epoch 447/1000
177/177 [==============================] - 0s 432us/step - loss: 0.1478 - accuracy: 0.9266 - val_loss: 4.6846 - val_accuracy: 0.3684
Epoch 448/1000
177/177 [==============================] - 0s 467us/step - loss: 0.1475 - accuracy: 0.9322 - val_loss: 4.6536 - 

177/177 [==============================] - 0s 919us/step - loss: 0.1499 - accuracy: 0.9153 - val_loss: 4.3525 - val_accuracy: 0.3421
Epoch 553/1000
177/177 [==============================] - 0s 409us/step - loss: 0.1520 - accuracy: 0.9209 - val_loss: 4.3197 - val_accuracy: 0.3553
Epoch 554/1000
177/177 [==============================] - 0s 775us/step - loss: 0.1468 - accuracy: 0.9322 - val_loss: 4.3650 - val_accuracy: 0.3421
Epoch 555/1000
177/177 [==============================] - 0s 461us/step - loss: 0.1439 - accuracy: 0.9266 - val_loss: 4.3500 - val_accuracy: 0.3553
Epoch 556/1000
177/177 [==============================] - 0s 1ms/step - loss: 0.1475 - accuracy: 0.9209 - val_loss: 4.3296 - val_accuracy: 0.3421
Epoch 557/1000
177/177 [==============================] - 0s 879us/step - loss: 0.1422 - accuracy: 0.9209 - val_loss: 4.3904 - val_accuracy: 0.3421
Epoch 558/1000
177/177 [==============================] - 0s 975us/step - loss: 0.1401 - accuracy: 0.9322 - val_loss: 4.3636 - va

177/177 [==============================] - 0s 523us/step - loss: 0.1589 - accuracy: 0.9322 - val_loss: 4.2712 - val_accuracy: 0.3816
Epoch 663/1000
177/177 [==============================] - 0s 525us/step - loss: 0.1494 - accuracy: 0.9322 - val_loss: 4.4628 - val_accuracy: 0.3553
Epoch 664/1000
177/177 [==============================] - 0s 427us/step - loss: 0.1688 - accuracy: 0.9322 - val_loss: 4.2840 - val_accuracy: 0.3947
Epoch 665/1000
177/177 [==============================] - 0s 356us/step - loss: 0.1778 - accuracy: 0.9153 - val_loss: 4.1590 - val_accuracy: 0.3816
Epoch 666/1000
177/177 [==============================] - 0s 377us/step - loss: 0.1873 - accuracy: 0.9266 - val_loss: 4.6065 - val_accuracy: 0.3421
Epoch 667/1000
177/177 [==============================] - 0s 381us/step - loss: 0.2043 - accuracy: 0.9096 - val_loss: 4.1910 - val_accuracy: 0.4211
Epoch 668/1000
177/177 [==============================] - 0s 453us/step - loss: 0.1823 - accuracy: 0.9266 - val_loss: 4.2309 - 

177/177 [==============================] - 0s 604us/step - loss: 0.1499 - accuracy: 0.9266 - val_loss: 4.3867 - val_accuracy: 0.3816
Epoch 773/1000
177/177 [==============================] - 0s 651us/step - loss: 0.1595 - accuracy: 0.9266 - val_loss: 4.3946 - val_accuracy: 0.3553
Epoch 774/1000
177/177 [==============================] - 0s 534us/step - loss: 0.1502 - accuracy: 0.9322 - val_loss: 4.4453 - val_accuracy: 0.3816
Epoch 775/1000
177/177 [==============================] - 0s 473us/step - loss: 0.1551 - accuracy: 0.9322 - val_loss: 4.3409 - val_accuracy: 0.4079
Epoch 776/1000
177/177 [==============================] - 0s 415us/step - loss: 0.1488 - accuracy: 0.9266 - val_loss: 4.3363 - val_accuracy: 0.3947
Epoch 777/1000
177/177 [==============================] - 0s 408us/step - loss: 0.1486 - accuracy: 0.9322 - val_loss: 4.4370 - val_accuracy: 0.3684
Epoch 778/1000
177/177 [==============================] - 0s 384us/step - loss: 0.1485 - accuracy: 0.9322 - val_loss: 4.4914 - 

177/177 [==============================] - 0s 373us/step - loss: 0.1582 - accuracy: 0.9209 - val_loss: 4.3753 - val_accuracy: 0.3947
Epoch 883/1000
177/177 [==============================] - 0s 366us/step - loss: 0.1550 - accuracy: 0.9266 - val_loss: 4.5357 - val_accuracy: 0.3684
Epoch 884/1000
177/177 [==============================] - 0s 415us/step - loss: 0.1565 - accuracy: 0.9322 - val_loss: 4.5959 - val_accuracy: 0.3684
Epoch 885/1000
177/177 [==============================] - 0s 343us/step - loss: 0.1575 - accuracy: 0.9322 - val_loss: 4.5335 - val_accuracy: 0.3947
Epoch 886/1000
177/177 [==============================] - 0s 370us/step - loss: 0.1500 - accuracy: 0.9322 - val_loss: 4.4837 - val_accuracy: 0.3684
Epoch 887/1000
177/177 [==============================] - 0s 416us/step - loss: 0.1646 - accuracy: 0.9266 - val_loss: 4.5236 - val_accuracy: 0.3684
Epoch 888/1000
177/177 [==============================] - 0s 364us/step - loss: 0.1405 - accuracy: 0.9266 - val_loss: 4.4297 - 

177/177 [==============================] - 0s 432us/step - loss: 0.1462 - accuracy: 0.9322 - val_loss: 4.4601 - val_accuracy: 0.3553
Epoch 993/1000
177/177 [==============================] - 0s 408us/step - loss: 0.1526 - accuracy: 0.9266 - val_loss: 4.4141 - val_accuracy: 0.3553
Epoch 994/1000
177/177 [==============================] - 0s 387us/step - loss: 0.1454 - accuracy: 0.9322 - val_loss: 4.5048 - val_accuracy: 0.3553
Epoch 995/1000
177/177 [==============================] - 0s 379us/step - loss: 0.1481 - accuracy: 0.9322 - val_loss: 4.4849 - val_accuracy: 0.3684
Epoch 996/1000
177/177 [==============================] - 0s 379us/step - loss: 0.1486 - accuracy: 0.9322 - val_loss: 4.4726 - val_accuracy: 0.3816
Epoch 997/1000
177/177 [==============================] - 0s 495us/step - loss: 0.1466 - accuracy: 0.9322 - val_loss: 4.4434 - val_accuracy: 0.3553
Epoch 998/1000
177/177 [==============================] - 0s 379us/step - loss: 0.1528 - accuracy: 0.9040 - val_loss: 4.4678 - 

In [162]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 162us/step
test accuracy: 47.37%


In [60]:
pred4 = model4.predict_classes(X_test)
pred4

array([2, 2, 1, 1, 1, 2, 0, 0, 0, 1, 1, 1, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2,
       1, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 1, 2, 1, 1, 0, 0, 0, 2, 0,
       1, 2, 0, 1, 1, 2, 2, 0, 0, 2, 0, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 2,
       0, 2, 2, 2, 0, 2, 2, 1, 2, 0])

In [61]:
dat4['pred'] = pred4
dat4

,id,test,pred
236,SR1129,2,2
31,CA105,2,2
155,NRS175,1,1
92,EUH25,2,1
122,NRS070,2,1
...,...,...,...
235,SR1065,0,2
24,BCH-SA-09,0,2
104,Grady1,0,1
213,NRS254,2,2


In [62]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,4.821690e-07,1.244127e-06,9.999983e-01
1,2.652370e-03,4.865103e-05,9.972990e-01
2,1.198157e-02,9.880184e-01,5.232074e-09
3,1.388957e-01,7.359540e-01,1.251503e-01
4,7.231966e-03,8.103169e-01,1.824512e-01
...,...,...,...
71,1.106059e-06,1.008290e-11,9.999989e-01
72,2.477362e-06,1.787481e-03,9.982101e-01
73,3.486694e-01,6.511054e-01,2.251759e-04
74,4.260638e-01,5.925720e-03,5.680104e-01


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p17.csv", index = False,
         header=None)

In [166]:
hist4 = model4.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 414us/step - loss: 0.1426 - accuracy: 0.9266 - val_loss: 6.7324 - val_accuracy: 0.4868
Epoch 2/1000
177/177 [==============================] - 0s 432us/step - loss: 0.1462 - accuracy: 0.9209 - val_loss: 6.9526 - val_accuracy: 0.4605
Epoch 3/1000
177/177 [==============================] - 0s 364us/step - loss: 0.1503 - accuracy: 0.9322 - val_loss: 6.8268 - val_accuracy: 0.4474
Epoch 4/1000
177/177 [==============================] - 0s 399us/step - loss: 0.1440 - accuracy: 0.9266 - val_loss: 6.6719 - val_accuracy: 0.4605
Epoch 5/1000
177/177 [==============================] - 0s 390us/step - loss: 0.1365 - accuracy: 0.9209 - val_loss: 6.7087 - val_accuracy: 0.4605
Epoch 6/1000
177/177 [==============================] - 0s 411us/step - loss: 0.1394 - accuracy: 0.9266 - val_loss: 6.7510 - val_accuracy: 0.4342
Epoch 7/1000
177/177 [==============================] - 0s 377us/step - loss: 0

177/177 [==============================] - 0s 463us/step - loss: 0.1380 - accuracy: 0.9266 - val_loss: 7.1956 - val_accuracy: 0.4474
Epoch 112/1000
177/177 [==============================] - 0s 452us/step - loss: 0.1362 - accuracy: 0.9266 - val_loss: 7.1289 - val_accuracy: 0.4474
Epoch 113/1000
177/177 [==============================] - 0s 448us/step - loss: 0.1349 - accuracy: 0.9266 - val_loss: 7.1699 - val_accuracy: 0.4342
Epoch 114/1000
177/177 [==============================] - 0s 377us/step - loss: 0.1339 - accuracy: 0.9322 - val_loss: 7.2680 - val_accuracy: 0.4605
Epoch 115/1000
177/177 [==============================] - 0s 384us/step - loss: 0.1377 - accuracy: 0.9266 - val_loss: 7.2250 - val_accuracy: 0.4342
Epoch 116/1000
177/177 [==============================] - 0s 377us/step - loss: 0.1392 - accuracy: 0.9322 - val_loss: 7.1988 - val_accuracy: 0.4342
Epoch 117/1000
177/177 [==============================] - 0s 358us/step - loss: 0.1351 - accuracy: 0.9266 - val_loss: 7.1662 - 

177/177 [==============================] - 0s 346us/step - loss: 0.1336 - accuracy: 0.9322 - val_loss: 6.5071 - val_accuracy: 0.4342
Epoch 222/1000
177/177 [==============================] - 0s 384us/step - loss: 0.1364 - accuracy: 0.9322 - val_loss: 6.6512 - val_accuracy: 0.4079
Epoch 223/1000
177/177 [==============================] - 0s 506us/step - loss: 0.1375 - accuracy: 0.9322 - val_loss: 6.6991 - val_accuracy: 0.3947
Epoch 224/1000
177/177 [==============================] - 0s 362us/step - loss: 0.1386 - accuracy: 0.9266 - val_loss: 6.5653 - val_accuracy: 0.4342
Epoch 225/1000
177/177 [==============================] - 0s 398us/step - loss: 0.1363 - accuracy: 0.9322 - val_loss: 6.6314 - val_accuracy: 0.4211
Epoch 226/1000
177/177 [==============================] - 0s 449us/step - loss: 0.1464 - accuracy: 0.9266 - val_loss: 6.9169 - val_accuracy: 0.4079
Epoch 227/1000
177/177 [==============================] - 0s 449us/step - loss: 0.1406 - accuracy: 0.9266 - val_loss: 6.6328 - 

177/177 [==============================] - 0s 422us/step - loss: 0.1408 - accuracy: 0.9266 - val_loss: 6.9875 - val_accuracy: 0.4342
Epoch 332/1000
177/177 [==============================] - 0s 581us/step - loss: 0.1337 - accuracy: 0.9266 - val_loss: 6.9409 - val_accuracy: 0.4211
Epoch 333/1000
177/177 [==============================] - 0s 391us/step - loss: 0.1437 - accuracy: 0.9322 - val_loss: 6.9134 - val_accuracy: 0.4211
Epoch 334/1000
177/177 [==============================] - 0s 422us/step - loss: 0.1366 - accuracy: 0.9266 - val_loss: 6.7921 - val_accuracy: 0.4474
Epoch 335/1000
177/177 [==============================] - 0s 377us/step - loss: 0.1375 - accuracy: 0.9322 - val_loss: 6.7703 - val_accuracy: 0.4211
Epoch 336/1000
177/177 [==============================] - 0s 380us/step - loss: 0.1534 - accuracy: 0.9322 - val_loss: 7.0066 - val_accuracy: 0.4342
Epoch 337/1000
177/177 [==============================] - 0s 362us/step - loss: 0.1342 - accuracy: 0.9209 - val_loss: 6.7387 - 

177/177 [==============================] - 0s 410us/step - loss: 0.1393 - accuracy: 0.9435 - val_loss: 7.3359 - val_accuracy: 0.4342
Epoch 442/1000
177/177 [==============================] - 0s 372us/step - loss: 0.1361 - accuracy: 0.9379 - val_loss: 7.1494 - val_accuracy: 0.4342
Epoch 443/1000
177/177 [==============================] - 0s 395us/step - loss: 0.1347 - accuracy: 0.9322 - val_loss: 7.0169 - val_accuracy: 0.4342
Epoch 444/1000
177/177 [==============================] - 0s 396us/step - loss: 0.1343 - accuracy: 0.9266 - val_loss: 7.1310 - val_accuracy: 0.4342
Epoch 445/1000
177/177 [==============================] - 0s 501us/step - loss: 0.1545 - accuracy: 0.9266 - val_loss: 7.2024 - val_accuracy: 0.4342
Epoch 446/1000
177/177 [==============================] - ETA: 0s - loss: 0.1312 - accuracy: 0.93 - 0s 593us/step - loss: 0.1347 - accuracy: 0.9322 - val_loss: 7.1669 - val_accuracy: 0.4342
Epoch 447/1000
177/177 [==============================] - 0s 444us/step - loss: 0.144

177/177 [==============================] - 0s 665us/step - loss: 0.1323 - accuracy: 0.9379 - val_loss: 7.3170 - val_accuracy: 0.4474
Epoch 552/1000
177/177 [==============================] - 0s 605us/step - loss: 0.1352 - accuracy: 0.9379 - val_loss: 7.2959 - val_accuracy: 0.4474
Epoch 553/1000
177/177 [==============================] - 0s 399us/step - loss: 0.1304 - accuracy: 0.9379 - val_loss: 7.4102 - val_accuracy: 0.4342
Epoch 554/1000
177/177 [==============================] - 0s 382us/step - loss: 0.1347 - accuracy: 0.9322 - val_loss: 7.4905 - val_accuracy: 0.4474
Epoch 555/1000
177/177 [==============================] - 0s 395us/step - loss: 0.1327 - accuracy: 0.9322 - val_loss: 7.4298 - val_accuracy: 0.4342
Epoch 556/1000
177/177 [==============================] - 0s 481us/step - loss: 0.1292 - accuracy: 0.9379 - val_loss: 7.3849 - val_accuracy: 0.4474
Epoch 557/1000
177/177 [==============================] - 0s 428us/step - loss: 0.1284 - accuracy: 0.9379 - val_loss: 7.5283 - 

177/177 [==============================] - 0s 395us/step - loss: 0.1244 - accuracy: 0.9322 - val_loss: 7.5669 - val_accuracy: 0.4342
Epoch 662/1000
177/177 [==============================] - 0s 379us/step - loss: 0.1211 - accuracy: 0.9435 - val_loss: 7.5473 - val_accuracy: 0.4342
Epoch 663/1000
177/177 [==============================] - 0s 365us/step - loss: 0.1241 - accuracy: 0.9435 - val_loss: 7.6378 - val_accuracy: 0.4342
Epoch 664/1000
177/177 [==============================] - 0s 361us/step - loss: 0.1248 - accuracy: 0.9379 - val_loss: 7.7251 - val_accuracy: 0.4342
Epoch 665/1000
177/177 [==============================] - 0s 351us/step - loss: 0.1218 - accuracy: 0.9435 - val_loss: 7.6565 - val_accuracy: 0.4211
Epoch 666/1000
177/177 [==============================] - 0s 357us/step - loss: 0.1345 - accuracy: 0.9209 - val_loss: 7.4345 - val_accuracy: 0.4605
Epoch 667/1000
177/177 [==============================] - 0s 357us/step - loss: 0.1328 - accuracy: 0.9266 - val_loss: 7.5033 - 

177/177 [==============================] - 0s 349us/step - loss: 0.1199 - accuracy: 0.9435 - val_loss: 7.8463 - val_accuracy: 0.4605
Epoch 772/1000
177/177 [==============================] - 0s 351us/step - loss: 0.1195 - accuracy: 0.9435 - val_loss: 7.8393 - val_accuracy: 0.4737
Epoch 773/1000
177/177 [==============================] - 0s 337us/step - loss: 0.1203 - accuracy: 0.9379 - val_loss: 7.8535 - val_accuracy: 0.4868
Epoch 774/1000
177/177 [==============================] - 0s 351us/step - loss: 0.1202 - accuracy: 0.9322 - val_loss: 7.7512 - val_accuracy: 0.4737
Epoch 775/1000
177/177 [==============================] - 0s 772us/step - loss: 0.1237 - accuracy: 0.9435 - val_loss: 7.8256 - val_accuracy: 0.4737
Epoch 776/1000
177/177 [==============================] - 0s 433us/step - loss: 0.1285 - accuracy: 0.9266 - val_loss: 8.0086 - val_accuracy: 0.4605
Epoch 777/1000
177/177 [==============================] - 0s 409us/step - loss: 0.1255 - accuracy: 0.9379 - val_loss: 7.9127 - 

Epoch 826/1000
177/177 [==============================] - 0s 427us/step - loss: 0.1168 - accuracy: 0.9379 - val_loss: 8.0603 - val_accuracy: 0.4737
Epoch 827/1000
177/177 [==============================] - 0s 359us/step - loss: 0.1237 - accuracy: 0.9435 - val_loss: 8.2508 - val_accuracy: 0.4605
Epoch 828/1000
177/177 [==============================] - 0s 360us/step - loss: 0.1181 - accuracy: 0.9435 - val_loss: 8.0182 - val_accuracy: 0.4737
Epoch 829/1000
177/177 [==============================] - 0s 363us/step - loss: 0.1211 - accuracy: 0.9379 - val_loss: 7.9561 - val_accuracy: 0.4605
Epoch 830/1000
177/177 [==============================] - 0s 412us/step - loss: 0.1183 - accuracy: 0.9435 - val_loss: 8.0695 - val_accuracy: 0.4737
Epoch 831/1000
177/177 [==============================] - 0s 415us/step - loss: 0.1172 - accuracy: 0.9322 - val_loss: 7.9869 - val_accuracy: 0.4868
Epoch 832/1000
177/177 [==============================] - 0s 419us/step - loss: 0.1181 - accuracy: 0.9266 - val_

177/177 [==============================] - 0s 400us/step - loss: 0.1133 - accuracy: 0.9435 - val_loss: 7.8123 - val_accuracy: 0.4079
Epoch 937/1000
177/177 [==============================] - 0s 405us/step - loss: 0.1155 - accuracy: 0.9266 - val_loss: 7.8110 - val_accuracy: 0.4079
Epoch 938/1000
177/177 [==============================] - 0s 416us/step - loss: 0.1134 - accuracy: 0.9435 - val_loss: 7.6956 - val_accuracy: 0.4474
Epoch 939/1000
177/177 [==============================] - 0s 576us/step - loss: 0.1168 - accuracy: 0.9435 - val_loss: 7.6659 - val_accuracy: 0.4342
Epoch 940/1000
177/177 [==============================] - 0s 536us/step - loss: 0.1147 - accuracy: 0.9435 - val_loss: 7.8410 - val_accuracy: 0.4079
Epoch 941/1000
177/177 [==============================] - 0s 471us/step - loss: 0.1146 - accuracy: 0.9435 - val_loss: 7.9123 - val_accuracy: 0.4211
Epoch 942/1000
177/177 [==============================] - 0s 406us/step - loss: 0.1166 - accuracy: 0.9492 - val_loss: 7.8101 - 

In [167]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 93.33%


In [52]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [53]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [54]:
y_prob4 = df_proba4[df_proba4['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[4.82168960e-07, 1.24412740e-06, 9.99998330e-01],
       [2.65237040e-03, 4.86510300e-05, 9.97299000e-01],
       [1.19815710e-02, 9.88018400e-01, 5.23207400e-09],
       [1.38895660e-01, 7.35954050e-01, 1.25150340e-01],
       [7.23196600e-03, 8.10316900e-01, 1.82451170e-01],
       [8.41680000e-06, 9.83987560e-05, 9.99893200e-01],
       [9.61113450e-01, 3.84657350e-02, 4.20822380e-04],
       [9.99839540e-01, 1.23355630e-08, 1.60395850e-04],
       [6.71746500e-01, 3.28201320e-01, 5.21121000e-05],
       [1.57147500e-08, 1.00000000e+00, 4.58735900e-29],
       [5.82733600e-03, 9.94172630e-01, 4.24900700e-08],
       [2.21876710e-06, 9.99997740e-01, 5.39959800e-08],
       [7.25406400e-02, 6.02039060e-02, 8.67255450e-01],
       [2.04181760e-02, 8.62998250e-01, 1.16583600e-01],
       [9.89948150e-01, 1.00517800e-02, 1.46672080e-10],
       [1.10553450e-02, 9.84351040e-01, 4.59361900e-03],
       [3.28627920e-07, 9.99934430e-01, 6.52619800e-05],
       [9.23419240e-01, 1.05150

In [55]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.5169953503286837

In [56]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.5169953503286837

In [57]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.5596361861242813

In [58]:
np.std(ovos)

0.02686820052511635

In [59]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.5596361861242813

In [60]:
np.std(ovrs)

0.02686820052511635

In [168]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [169]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 43.42%


In [170]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.04055536011726605


In [171]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [172]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 93.70%


In [173]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.008062731


In [108]:
############ Feature selection using lasso ##########

In [61]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [62]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [63]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [64]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [65]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7078
selected features: 495


In [66]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  18,   38,   39,   40,   55,   58,   79,   84,  107,  109,  130,
         159,  161,  164,  168,  182,  211,  213,  227,  267,  273,  276,
         297,  310,  313,  323,  338,  347,  357,  394,  395,  422,  430,
         459,  487,  498,  519,  533,  535,  537,  540,  541,  551,  556,
         586,  590,  601,  626,  632,  637,  639,  742,  745,  759,  761,
         762,  779,  785,  821,  822,  833,  838,  850,  866,  874,  881,
         885,  921,  922,  953,  982,  983, 1027, 1031, 1047, 1064, 1088,
        1109, 1114, 1135, 1141, 1162, 1181, 1184, 1193, 1198, 1201, 1202,
        1208, 1221, 1225, 1255, 1273, 1274, 1278, 1281, 1283, 1287, 1288,
        1321, 1322, 1328, 1330, 1365, 1375, 1391, 1400, 1403, 1407, 1414,
        1427, 1442, 1482, 1484, 1501, 1503, 1508, 1514, 1518, 1519, 1521,
        1559, 1562, 1572, 1600, 1618, 1619, 1625, 1635, 1653, 1654, 1700,
        1734, 1738, 1739, 1746, 1785, 1801, 1851, 1853, 1885, 1902, 1908,
        1921, 1946, 1993, 1995, 2026, 

In [67]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTTGCCAATTTTGAGGAGGGAACACATGAAAACACGTTGTTACGATGGTAAAAAATGGCAATATGAGTTTAAGCATGAAGGAAAAAGATACCGTAAG',
       'TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG',
       'TTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAA',
       'TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA',
       'TTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCC',
       'TTTTTGCCGTGGGTAGAAAGTGCTTTTTTATTAATTTTAAAAAAAGCACCAAAAATTTAAATGGAGGTGTCTGAATGTCTGCTATCATTGAAGCTAAAAA',
       'TTTTTCATTTTCATTT',
       'TTTTTATTTTCTACACTTCTGTGTTTTACTTTTGTTAAAATATATAGGATTTAAATTATTTATTTATTTAT',
       'TTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCGA',
       'TTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAAT',
   

In [68]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [69]:
df_sel['strain'] = X.iloc[:,0]

In [70]:
df_sel

,TTTTTTGCCAATTTTGAGGAGGGAACACATGAAAACACGTTGTTACGATGGTAAAAAATGGCAATATGAGTTTAAGCATGAAGGAAAAAGATACCGTAAG,TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG,TTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAA,TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA,TTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCC,TTTTTGCCGTGGGTAGAAAGTGCTTTTTTATTAATTTTAAAAAAAGCACCAAAAATTTAAATGGAGGTGTCTGAATGTCTGCTATCATTGAAGCTAAAAA,TTTTTCATTTTCATTT,TTTTTATTTTCTACACTTCTGTGTTTTACTTTTGTTAAAATATATAGGATTTAAATTATTTATTTATTTAT,TTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCGA,TTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAAT,...,AATGTAATAGACGACCCGTTAATATTCAATACAACGATGGCTACGACTTAATGATAGACCAGCGTTTTATTGAAATGACGCTTGAAAGACATCCGCATTT,AATAAGCGGTT,AAGTATTTTGCCTACAAACCATAATTGAATTAAATTCCAAATTAACTGCACAGTACCTTTCAGTATCATTACAATGCCGTCCCAAACGCCTCGCCAGTTT,AAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTT,AAGCCATTTGGTACCTTTACCTATTTGATCTTTTGTCCAGTTAAATGCCGATGATGCACCGGATTTAATATCTTTCCACATAGTACCTATGCTAAATCTT,AACGTTAGGGCAATAAATGAAAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTT,AAATTTCGGGTAGCTCGCCTACCCTTATTATTTTTT,AAAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTT,pheno,strain
0,1,1,1,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,2,107
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,109
2,0,1,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,2,115
3,0,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,2,120335
4,0,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1,1,0,0,0,0,1,1,1,0,...,1,1,1,1,1,1,1,1,2,SR4152
249,1,1,0,0,0,0,1,1,1,0,...,1,1,1,1,1,1,1,1,1,SR4153
250,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,SR4155
251,1,1,0,0,0,0,1,1,1,0,...,1,1,1,1,1,1,1,1,2,SR4156


In [71]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 496) (253,) (253, 497)


In [72]:
df_sel['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [73]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [74]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [75]:
dat5

,strain,test
231,NY360,2
139,NRS113,0
4,120337,2
59,CFBREBSa127,1
96,GA27,2
...,...,...
236,SR1129,2
145,NRS152,2
86,CFBRSa66A,0
28,BCH-SA-13,1


In [76]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [82]:
#### neural network on over-sampling data
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [83]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [84]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 873us/step - loss: 1.0410 - accuracy: 0.4407 - val_loss: 1.2245 - val_accuracy: 0.4211
Epoch 2/1000
177/177 [==============================] - 0s 203us/step - loss: 0.9645 - accuracy: 0.4915 - val_loss: 1.2435 - val_accuracy: 0.4868
Epoch 3/1000
177/177 [==============================] - 0s 238us/step - loss: 0.9278 - accuracy: 0.5593 - val_loss: 1.2549 - val_accuracy: 0.4605
Epoch 4/1000
177/177 [==============================] - 0s 243us/step - loss: 0.9075 - accuracy: 0.5989 - val_loss: 1.2693 - val_accuracy: 0.5263
Epoch 5/1000
177/177 [==============================] - 0s 205us/step - loss: 0.8815 - accuracy: 0.5932 - val_loss: 1.2791 - val_accuracy: 0.5395
Epoch 6/1000
177/177 [==============================] - 0s 162us/step - loss: 0.8625 - accuracy: 0.6158 - val_loss: 1.2700 - val_accuracy: 0.5395
Epoch 7/1000
177/177 [==============================] - 0s 155us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 110us/step - loss: 0.3204 - accuracy: 0.8757 - val_loss: 1.7817 - val_accuracy: 0.4605
Epoch 113/1000
177/177 [==============================] - 0s 108us/step - loss: 0.2926 - accuracy: 0.8927 - val_loss: 1.7761 - val_accuracy: 0.4474
Epoch 114/1000
177/177 [==============================] - 0s 147us/step - loss: 0.2988 - accuracy: 0.8701 - val_loss: 1.7485 - val_accuracy: 0.4342
Epoch 115/1000
177/177 [==============================] - 0s 248us/step - loss: 0.3085 - accuracy: 0.8701 - val_loss: 1.7542 - val_accuracy: 0.4474
Epoch 116/1000
177/177 [==============================] - 0s 166us/step - loss: 0.2866 - accuracy: 0.8983 - val_loss: 1.7617 - val_accuracy: 0.4474
Epoch 117/1000
177/177 [==============================] - 0s 157us/step - loss: 0.2970 - accuracy: 0.9153 - val_loss: 1.7772 - val_accuracy: 0.4474
Epoch 118/1000
177/177 [==============================] - 0s 124us/step - loss: 0.2847 - accuracy: 0.8983 - val_

Epoch 167/1000
177/177 [==============================] - 0s 107us/step - loss: 0.2321 - accuracy: 0.9096 - val_loss: 1.9320 - val_accuracy: 0.4605
Epoch 168/1000
177/177 [==============================] - 0s 115us/step - loss: 0.2327 - accuracy: 0.9096 - val_loss: 1.9375 - val_accuracy: 0.4737
Epoch 169/1000
177/177 [==============================] - 0s 153us/step - loss: 0.2285 - accuracy: 0.9096 - val_loss: 1.9467 - val_accuracy: 0.4474
Epoch 170/1000
177/177 [==============================] - 0s 122us/step - loss: 0.2256 - accuracy: 0.9040 - val_loss: 1.9970 - val_accuracy: 0.4342
Epoch 171/1000
177/177 [==============================] - 0s 116us/step - loss: 0.2357 - accuracy: 0.8983 - val_loss: 1.9960 - val_accuracy: 0.4605
Epoch 172/1000
177/177 [==============================] - 0s 100us/step - loss: 0.2322 - accuracy: 0.9096 - val_loss: 1.9788 - val_accuracy: 0.4474
Epoch 173/1000
177/177 [==============================] - 0s 105us/step - loss: 0.2466 - accuracy: 0.8983 - val_

177/177 [==============================] - 0s 94us/step - loss: 0.1967 - accuracy: 0.8983 - val_loss: 2.2953 - val_accuracy: 0.4605
Epoch 278/1000
177/177 [==============================] - 0s 87us/step - loss: 0.1858 - accuracy: 0.9209 - val_loss: 2.2490 - val_accuracy: 0.4474
Epoch 279/1000
177/177 [==============================] - 0s 90us/step - loss: 0.1917 - accuracy: 0.9209 - val_loss: 2.2664 - val_accuracy: 0.4605
Epoch 280/1000
177/177 [==============================] - 0s 89us/step - loss: 0.1827 - accuracy: 0.9209 - val_loss: 2.2226 - val_accuracy: 0.4342
Epoch 281/1000
177/177 [==============================] - 0s 108us/step - loss: 0.1882 - accuracy: 0.9040 - val_loss: 2.2386 - val_accuracy: 0.4605
Epoch 282/1000
177/177 [==============================] - 0s 163us/step - loss: 0.1896 - accuracy: 0.8983 - val_loss: 2.2648 - val_accuracy: 0.4474
Epoch 283/1000
177/177 [==============================] - 0s 141us/step - loss: 0.1827 - accuracy: 0.9209 - val_loss: 2.2842 - val_

Epoch 388/1000
177/177 [==============================] - 0s 75us/step - loss: 0.1743 - accuracy: 0.9266 - val_loss: 2.4402 - val_accuracy: 0.4605
Epoch 389/1000
177/177 [==============================] - 0s 72us/step - loss: 0.1684 - accuracy: 0.9209 - val_loss: 2.4982 - val_accuracy: 0.4737
Epoch 390/1000
177/177 [==============================] - 0s 75us/step - loss: 0.1726 - accuracy: 0.9153 - val_loss: 2.5139 - val_accuracy: 0.4868
Epoch 391/1000
177/177 [==============================] - 0s 74us/step - loss: 0.1915 - accuracy: 0.9153 - val_loss: 2.5132 - val_accuracy: 0.4605
Epoch 392/1000
177/177 [==============================] - 0s 67us/step - loss: 0.1727 - accuracy: 0.9266 - val_loss: 2.5491 - val_accuracy: 0.4342
Epoch 393/1000
177/177 [==============================] - 0s 71us/step - loss: 0.1669 - accuracy: 0.9266 - val_loss: 2.4865 - val_accuracy: 0.4211
Epoch 394/1000
177/177 [==============================] - 0s 74us/step - loss: 0.1961 - accuracy: 0.9096 - val_loss: 2

177/177 [==============================] - 0s 127us/step - loss: 0.1692 - accuracy: 0.9266 - val_loss: 2.6385 - val_accuracy: 0.4474
Epoch 499/1000
177/177 [==============================] - 0s 139us/step - loss: 0.1566 - accuracy: 0.9266 - val_loss: 2.6663 - val_accuracy: 0.4474
Epoch 500/1000
177/177 [==============================] - 0s 132us/step - loss: 0.1704 - accuracy: 0.9153 - val_loss: 2.7183 - val_accuracy: 0.4605
Epoch 501/1000
177/177 [==============================] - 0s 173us/step - loss: 0.1638 - accuracy: 0.9040 - val_loss: 2.6764 - val_accuracy: 0.4737
Epoch 502/1000
177/177 [==============================] - 0s 126us/step - loss: 0.1605 - accuracy: 0.9209 - val_loss: 2.6761 - val_accuracy: 0.4737
Epoch 503/1000
177/177 [==============================] - 0s 166us/step - loss: 0.1575 - accuracy: 0.9266 - val_loss: 2.6468 - val_accuracy: 0.4474
Epoch 504/1000
177/177 [==============================] - 0s 144us/step - loss: 0.1596 - accuracy: 0.9209 - val_loss: 2.7122 - 

177/177 [==============================] - 0s 112us/step - loss: 0.1561 - accuracy: 0.9209 - val_loss: 2.8192 - val_accuracy: 0.4737
Epoch 609/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1549 - accuracy: 0.9209 - val_loss: 2.8458 - val_accuracy: 0.4474
Epoch 610/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1558 - accuracy: 0.9266 - val_loss: 2.8627 - val_accuracy: 0.4474
Epoch 611/1000
177/177 [==============================] - 0s 120us/step - loss: 0.1569 - accuracy: 0.9040 - val_loss: 2.9422 - val_accuracy: 0.4737
Epoch 612/1000
177/177 [==============================] - 0s 123us/step - loss: 0.1539 - accuracy: 0.9153 - val_loss: 2.8784 - val_accuracy: 0.4474
Epoch 613/1000
177/177 [==============================] - 0s 111us/step - loss: 0.1534 - accuracy: 0.9266 - val_loss: 2.8709 - val_accuracy: 0.4474
Epoch 614/1000
177/177 [==============================] - 0s 97us/step - loss: 0.1513 - accuracy: 0.9153 - val_loss: 2.8898 - v

177/177 [==============================] - 0s 139us/step - loss: 0.1572 - accuracy: 0.9209 - val_loss: 2.9616 - val_accuracy: 0.4605
Epoch 719/1000
177/177 [==============================] - 0s 156us/step - loss: 0.1517 - accuracy: 0.9266 - val_loss: 2.9254 - val_accuracy: 0.4342
Epoch 720/1000
177/177 [==============================] - 0s 134us/step - loss: 0.1693 - accuracy: 0.9266 - val_loss: 2.9838 - val_accuracy: 0.4868
Epoch 721/1000
177/177 [==============================] - 0s 122us/step - loss: 0.1650 - accuracy: 0.9153 - val_loss: 2.9559 - val_accuracy: 0.4737
Epoch 722/1000
177/177 [==============================] - 0s 147us/step - loss: 0.1517 - accuracy: 0.9153 - val_loss: 2.9495 - val_accuracy: 0.4605
Epoch 723/1000
177/177 [==============================] - 0s 114us/step - loss: 0.1508 - accuracy: 0.9266 - val_loss: 2.9922 - val_accuracy: 0.4605
Epoch 724/1000
177/177 [==============================] - 0s 124us/step - loss: 0.1567 - accuracy: 0.9266 - val_loss: 3.0326 - 

177/177 [==============================] - 0s 181us/step - loss: 0.1872 - accuracy: 0.8927 - val_loss: 3.0391 - val_accuracy: 0.4474
Epoch 829/1000
177/177 [==============================] - 0s 187us/step - loss: 0.1765 - accuracy: 0.9266 - val_loss: 3.1072 - val_accuracy: 0.4737
Epoch 830/1000
177/177 [==============================] - 0s 155us/step - loss: 0.1860 - accuracy: 0.9266 - val_loss: 2.9893 - val_accuracy: 0.4474
Epoch 831/1000
177/177 [==============================] - 0s 232us/step - loss: 0.1570 - accuracy: 0.9266 - val_loss: 3.0611 - val_accuracy: 0.4474
Epoch 832/1000
177/177 [==============================] - 0s 178us/step - loss: 0.1601 - accuracy: 0.9153 - val_loss: 3.1452 - val_accuracy: 0.4868
Epoch 833/1000
177/177 [==============================] - 0s 220us/step - loss: 0.1480 - accuracy: 0.9266 - val_loss: 3.0593 - val_accuracy: 0.4474
Epoch 834/1000
177/177 [==============================] - 0s 192us/step - loss: 0.1597 - accuracy: 0.9209 - val_loss: 3.0711 - 

177/177 [==============================] - 0s 296us/step - loss: 0.1565 - accuracy: 0.9040 - val_loss: 3.1367 - val_accuracy: 0.4605
Epoch 939/1000
177/177 [==============================] - 0s 237us/step - loss: 0.1471 - accuracy: 0.9266 - val_loss: 3.1202 - val_accuracy: 0.4474
Epoch 940/1000
177/177 [==============================] - 0s 345us/step - loss: 0.1533 - accuracy: 0.9266 - val_loss: 3.1772 - val_accuracy: 0.4605
Epoch 941/1000
177/177 [==============================] - 0s 206us/step - loss: 0.1611 - accuracy: 0.9153 - val_loss: 3.2478 - val_accuracy: 0.4737
Epoch 942/1000
177/177 [==============================] - 0s 239us/step - loss: 0.1528 - accuracy: 0.8983 - val_loss: 3.2070 - val_accuracy: 0.4605
Epoch 943/1000
177/177 [==============================] - 0s 285us/step - loss: 0.1448 - accuracy: 0.9209 - val_loss: 3.1612 - val_accuracy: 0.4474
Epoch 944/1000
177/177 [==============================] - 0s 163us/step - loss: 0.1445 - accuracy: 0.9153 - val_loss: 3.1597 - 

In [223]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 87us/step
over-sampling test accuracy: 46.05%


In [85]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([1, 0, 2, 0, 2, 0, 1, 1, 2, 0, 1, 2, 1, 0, 2, 1, 2, 0, 0, 0, 0, 2,
       2, 2, 0, 2, 1, 0, 0, 1, 1, 2, 1, 0, 1, 1, 0, 2, 2, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 2, 2, 2, 2, 0, 2, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 2, 1,
       0, 0, 1, 0, 2, 2, 2, 0, 1, 2])

In [86]:
dat5['pred'] = pred5
dat5

,strain,test,pred
231,NY360,2,1
139,NRS113,0,0
4,120337,2,2
59,CFBREBSa127,1,0
96,GA27,2,2
...,...,...,...
236,SR1129,2,2
145,NRS152,2,2
86,CFBRSa66A,0,0
28,BCH-SA-13,1,1


In [87]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [88]:
dat_proba5

,0,1,2
0,3.848032e-03,9.960963e-01,0.000056
1,8.441349e-01,1.475804e-05,0.155850
2,1.466081e-02,5.713178e-03,0.979626
3,9.985392e-01,3.499175e-04,0.001111
4,1.665667e-08,4.836321e-06,0.999995
...,...,...,...
71,5.429790e-05,2.165729e-02,0.978288
72,5.817226e-02,2.659354e-03,0.939168
73,6.594160e-01,1.464654e-01,0.194119
74,4.252871e-01,5.690199e-01,0.005693


In [89]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [90]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p17.csv", index = False,
         header=None)

In [230]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 116us/step - loss: 0.1607 - accuracy: 0.9266 - val_loss: 2.9458 - val_accuracy: 0.4474
Epoch 2/1000
177/177 [==============================] - 0s 109us/step - loss: 0.1430 - accuracy: 0.9266 - val_loss: 2.9907 - val_accuracy: 0.4868
Epoch 3/1000
177/177 [==============================] - 0s 171us/step - loss: 0.1867 - accuracy: 0.9266 - val_loss: 2.9866 - val_accuracy: 0.4737
Epoch 4/1000
177/177 [==============================] - 0s 204us/step - loss: 0.1671 - accuracy: 0.9322 - val_loss: 2.9284 - val_accuracy: 0.4605
Epoch 5/1000
177/177 [==============================] - 0s 174us/step - loss: 0.1817 - accuracy: 0.9209 - val_loss: 2.9470 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 158us/step - loss: 0.1760 - accuracy: 0.9040 - val_loss: 2.9936 - val_accuracy: 0.4605
Epoch 7/1000
177/177 [==============================] - 0s 142us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 144us/step - loss: 0.1615 - accuracy: 0.9209 - val_loss: 3.0273 - val_accuracy: 0.4868
Epoch 113/1000
177/177 [==============================] - 0s 147us/step - loss: 0.1434 - accuracy: 0.9266 - val_loss: 3.0084 - val_accuracy: 0.4342
Epoch 114/1000
177/177 [==============================] - 0s 964us/step - loss: 0.1528 - accuracy: 0.9153 - val_loss: 3.0388 - val_accuracy: 0.4474
Epoch 115/1000
177/177 [==============================] - 0s 154us/step - loss: 0.1485 - accuracy: 0.9153 - val_loss: 3.0228 - val_accuracy: 0.4868
Epoch 116/1000
177/177 [==============================] - 0s 159us/step - loss: 0.1473 - accuracy: 0.9209 - val_loss: 3.0203 - val_accuracy: 0.4737
Epoch 117/1000
177/177 [==============================] - 0s 343us/step - loss: 0.1464 - accuracy: 0.9322 - val_loss: 3.0135 - val_accuracy: 0.4868
Epoch 118/1000
177/177 [==============================] - 0s 166us/step - loss: 0.1517 - accuracy: 0.9266 - val_

177/177 [==============================] - 0s 203us/step - loss: 0.1513 - accuracy: 0.9153 - val_loss: 3.1334 - val_accuracy: 0.4605
Epoch 223/1000
177/177 [==============================] - 0s 253us/step - loss: 0.1590 - accuracy: 0.8983 - val_loss: 3.1073 - val_accuracy: 0.4605
Epoch 224/1000
177/177 [==============================] - 0s 119us/step - loss: 0.1457 - accuracy: 0.9266 - val_loss: 3.1193 - val_accuracy: 0.4737
Epoch 225/1000
177/177 [==============================] - 0s 115us/step - loss: 0.1465 - accuracy: 0.9040 - val_loss: 3.1591 - val_accuracy: 0.4605
Epoch 226/1000
177/177 [==============================] - 0s 344us/step - loss: 0.1515 - accuracy: 0.9209 - val_loss: 3.1606 - val_accuracy: 0.4605
Epoch 227/1000
177/177 [==============================] - 0s 349us/step - loss: 0.1484 - accuracy: 0.9209 - val_loss: 3.1610 - val_accuracy: 0.4737
Epoch 228/1000
177/177 [==============================] - 0s 190us/step - loss: 0.1429 - accuracy: 0.9266 - val_loss: 3.1514 - 

177/177 [==============================] - 0s 178us/step - loss: 0.1437 - accuracy: 0.9153 - val_loss: 3.1896 - val_accuracy: 0.4737
Epoch 333/1000
177/177 [==============================] - 0s 139us/step - loss: 0.1439 - accuracy: 0.9153 - val_loss: 3.2228 - val_accuracy: 0.4737
Epoch 334/1000
177/177 [==============================] - 0s 156us/step - loss: 0.1476 - accuracy: 0.9266 - val_loss: 3.2054 - val_accuracy: 0.4474
Epoch 335/1000
177/177 [==============================] - 0s 177us/step - loss: 0.1493 - accuracy: 0.9266 - val_loss: 3.1836 - val_accuracy: 0.4342
Epoch 336/1000
177/177 [==============================] - 0s 163us/step - loss: 0.1610 - accuracy: 0.9209 - val_loss: 3.2232 - val_accuracy: 0.4737
Epoch 337/1000
177/177 [==============================] - 0s 203us/step - loss: 0.1509 - accuracy: 0.9209 - val_loss: 3.1482 - val_accuracy: 0.4474
Epoch 338/1000
177/177 [==============================] - 0s 228us/step - loss: 0.1595 - accuracy: 0.9266 - val_loss: 3.1589 - 

177/177 [==============================] - 0s 109us/step - loss: 0.1504 - accuracy: 0.9040 - val_loss: 3.2532 - val_accuracy: 0.4605
Epoch 443/1000
177/177 [==============================] - 0s 95us/step - loss: 0.1472 - accuracy: 0.9153 - val_loss: 3.2711 - val_accuracy: 0.4737
Epoch 444/1000
177/177 [==============================] - 0s 102us/step - loss: 0.1591 - accuracy: 0.9209 - val_loss: 3.2798 - val_accuracy: 0.4605
Epoch 445/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1555 - accuracy: 0.9209 - val_loss: 3.2313 - val_accuracy: 0.4474
Epoch 446/1000
177/177 [==============================] - 0s 99us/step - loss: 0.1595 - accuracy: 0.9153 - val_loss: 3.2627 - val_accuracy: 0.4342
Epoch 447/1000
177/177 [==============================] - 0s 114us/step - loss: 0.1478 - accuracy: 0.9153 - val_loss: 3.2889 - val_accuracy: 0.4868
Epoch 448/1000
177/177 [==============================] - 0s 160us/step - loss: 0.1797 - accuracy: 0.9096 - val_loss: 3.3169 - va

177/177 [==============================] - 0s 129us/step - loss: 0.1471 - accuracy: 0.9209 - val_loss: 3.3004 - val_accuracy: 0.4605
Epoch 553/1000
177/177 [==============================] - 0s 170us/step - loss: 0.1460 - accuracy: 0.9096 - val_loss: 3.3180 - val_accuracy: 0.4737
Epoch 554/1000
177/177 [==============================] - 0s 121us/step - loss: 0.1447 - accuracy: 0.9209 - val_loss: 3.3244 - val_accuracy: 0.4474
Epoch 555/1000
177/177 [==============================] - 0s 135us/step - loss: 0.1516 - accuracy: 0.9153 - val_loss: 3.3560 - val_accuracy: 0.4474
Epoch 556/1000
177/177 [==============================] - 0s 117us/step - loss: 0.1439 - accuracy: 0.9209 - val_loss: 3.3382 - val_accuracy: 0.4605
Epoch 557/1000
177/177 [==============================] - 0s 285us/step - loss: 0.1451 - accuracy: 0.9153 - val_loss: 3.2986 - val_accuracy: 0.4737
Epoch 558/1000
177/177 [==============================] - 0s 237us/step - loss: 0.1404 - accuracy: 0.9266 - val_loss: 3.2513 - 

177/177 [==============================] - 0s 128us/step - loss: 0.1528 - accuracy: 0.9209 - val_loss: 3.3719 - val_accuracy: 0.4342
Epoch 663/1000
177/177 [==============================] - 0s 136us/step - loss: 0.1512 - accuracy: 0.9153 - val_loss: 3.4018 - val_accuracy: 0.4737
Epoch 664/1000
177/177 [==============================] - 0s 130us/step - loss: 0.1525 - accuracy: 0.9153 - val_loss: 3.4470 - val_accuracy: 0.4605
Epoch 665/1000
177/177 [==============================] - 0s 175us/step - loss: 0.1418 - accuracy: 0.9266 - val_loss: 3.4354 - val_accuracy: 0.4079
Epoch 666/1000
177/177 [==============================] - 0s 176us/step - loss: 0.1453 - accuracy: 0.9153 - val_loss: 3.4451 - val_accuracy: 0.4342
Epoch 667/1000
177/177 [==============================] - 0s 143us/step - loss: 0.1597 - accuracy: 0.9209 - val_loss: 3.4330 - val_accuracy: 0.4605
Epoch 668/1000
177/177 [==============================] - 0s 240us/step - loss: 0.1507 - accuracy: 0.9153 - val_loss: 3.4735 - 

177/177 [==============================] - 0s 126us/step - loss: 0.1455 - accuracy: 0.9096 - val_loss: 3.4534 - val_accuracy: 0.4474
Epoch 773/1000
177/177 [==============================] - 0s 128us/step - loss: 0.1466 - accuracy: 0.9209 - val_loss: 3.4960 - val_accuracy: 0.4342
Epoch 774/1000
177/177 [==============================] - 0s 217us/step - loss: 0.1465 - accuracy: 0.9096 - val_loss: 3.5261 - val_accuracy: 0.4474
Epoch 775/1000
177/177 [==============================] - 0s 132us/step - loss: 0.1457 - accuracy: 0.9153 - val_loss: 3.4906 - val_accuracy: 0.4211
Epoch 776/1000
177/177 [==============================] - 0s 185us/step - loss: 0.1447 - accuracy: 0.9153 - val_loss: 3.4695 - val_accuracy: 0.4605
Epoch 777/1000
177/177 [==============================] - 0s 179us/step - loss: 0.1425 - accuracy: 0.9209 - val_loss: 3.4517 - val_accuracy: 0.4474
Epoch 778/1000
177/177 [==============================] - 0s 194us/step - loss: 0.1486 - accuracy: 0.9209 - val_loss: 3.4634 - 

177/177 [==============================] - 0s 63us/step - loss: 0.1440 - accuracy: 0.9209 - val_loss: 3.5665 - val_accuracy: 0.4474
Epoch 883/1000
177/177 [==============================] - 0s 71us/step - loss: 0.1495 - accuracy: 0.9096 - val_loss: 3.5341 - val_accuracy: 0.4605
Epoch 884/1000
177/177 [==============================] - 0s 76us/step - loss: 0.1448 - accuracy: 0.9209 - val_loss: 3.5873 - val_accuracy: 0.4474
Epoch 885/1000
177/177 [==============================] - 0s 74us/step - loss: 0.1457 - accuracy: 0.9266 - val_loss: 3.5579 - val_accuracy: 0.4474
Epoch 886/1000
177/177 [==============================] - 0s 62us/step - loss: 0.1502 - accuracy: 0.9096 - val_loss: 3.5511 - val_accuracy: 0.4342
Epoch 887/1000
177/177 [==============================] - 0s 62us/step - loss: 0.1467 - accuracy: 0.8983 - val_loss: 3.6104 - val_accuracy: 0.4605
Epoch 888/1000
177/177 [==============================] - 0s 62us/step - loss: 0.1537 - accuracy: 0.9153 - val_loss: 3.5880 - val_acc

177/177 [==============================] - 0s 154us/step - loss: 0.1481 - accuracy: 0.9209 - val_loss: 3.6615 - val_accuracy: 0.4342
Epoch 993/1000
177/177 [==============================] - 0s 166us/step - loss: 0.1475 - accuracy: 0.9153 - val_loss: 3.6815 - val_accuracy: 0.4211
Epoch 994/1000
177/177 [==============================] - 0s 172us/step - loss: 0.1399 - accuracy: 0.9209 - val_loss: 3.6606 - val_accuracy: 0.4211
Epoch 995/1000
177/177 [==============================] - 0s 165us/step - loss: 0.1422 - accuracy: 0.9153 - val_loss: 3.6969 - val_accuracy: 0.4342
Epoch 996/1000
177/177 [==============================] - 0s 208us/step - loss: 0.1418 - accuracy: 0.9096 - val_loss: 3.6788 - val_accuracy: 0.4342
Epoch 997/1000
177/177 [==============================] - 0s 188us/step - loss: 0.1407 - accuracy: 0.9266 - val_loss: 3.6620 - val_accuracy: 0.4342
Epoch 998/1000
177/177 [==============================] - 0s 296us/step - loss: 0.1448 - accuracy: 0.9266 - val_loss: 3.6799 - 

In [231]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 91.94%


In [77]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [78]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [79]:
y_prob5 = df_proba5[df_proba5['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[3.8480320e-03, 9.9609630e-01, 5.5734155e-05],
       [8.4413490e-01, 1.4758036e-05, 1.5585022e-01],
       [1.4660809e-02, 5.7131783e-03, 9.7962600e-01],
       [9.9853920e-01, 3.4991750e-04, 1.1108240e-03],
       [1.6656674e-08, 4.8363213e-06, 9.9999510e-01],
       [9.9998460e-01, 3.9074894e-09, 1.5377289e-05],
       [3.7608084e-01, 6.1999214e-01, 3.9269626e-03],
       [4.9021927e-01, 5.0957215e-01, 2.0860568e-04],
       [3.6773793e-03, 1.8459606e-07, 9.9632240e-01],
       [5.2091360e-01, 2.7700220e-01, 2.0208417e-01],
       [1.3920697e-02, 8.9583340e-01, 9.0245850e-02],
       [3.2280846e-03, 2.2900406e-06, 9.9676967e-01],
       [2.9963735e-01, 6.6937000e-01, 3.0992580e-02],
       [9.9858640e-01, 7.5085065e-04, 6.6268880e-04],
       [1.4540682e-03, 3.8387874e-04, 9.9816210e-01],
       [3.8480320e-03, 9.9609630e-01, 5.5734155e-05],
       [4.2552370e-01, 3.9815083e-03, 5.7049480e-01],
       [9.9999905e-01, 1.4746193e-08, 9.0687140e-07],
       [6.2883400e-01, 2.549

In [81]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.6030972995258709

In [82]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.6030972995258709

In [83]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [84]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [85]:
dat6

,strain,test
44,CFBREBSa104,1
170,NRS199,2
197,NRS233,1
238,SR1746,0
172,NRS202,2
...,...,...
92,EUH25,2
217,NRS260,0
75,CFBRSa25,0
156,NRS177,1


In [86]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [95]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [96]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [97]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 726us/step - loss: 1.1133 - accuracy: 0.3559 - val_loss: 1.0662 - val_accuracy: 0.3816
Epoch 2/1000
177/177 [==============================] - 0s 76us/step - loss: 1.0502 - accuracy: 0.4068 - val_loss: 1.0332 - val_accuracy: 0.4474
Epoch 3/1000
177/177 [==============================] - 0s 115us/step - loss: 1.0141 - accuracy: 0.4915 - val_loss: 1.0225 - val_accuracy: 0.4737
Epoch 4/1000
177/177 [==============================] - 0s 122us/step - loss: 0.9877 - accuracy: 0.5198 - val_loss: 1.0150 - val_accuracy: 0.4868
Epoch 5/1000
177/177 [==============================] - 0s 122us/step - loss: 0.9674 - accuracy: 0.5367 - val_loss: 1.0129 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 108us/step - loss: 0.9480 - accuracy: 0.5537 - val_loss: 1.0101 - val_accuracy: 0.4868
Epoch 7/1000
177/177 [==============================] - 0s 115us/step - loss: 0.

Epoch 112/1000
177/177 [==============================] - 0s 128us/step - loss: 0.2519 - accuracy: 0.9096 - val_loss: 1.5637 - val_accuracy: 0.4605
Epoch 113/1000
177/177 [==============================] - 0s 142us/step - loss: 0.2460 - accuracy: 0.9096 - val_loss: 1.5790 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 146us/step - loss: 0.2489 - accuracy: 0.9096 - val_loss: 1.4939 - val_accuracy: 0.5000
Epoch 115/1000
177/177 [==============================] - 0s 128us/step - loss: 0.2501 - accuracy: 0.9040 - val_loss: 1.4772 - val_accuracy: 0.4605
Epoch 116/1000
177/177 [==============================] - 0s 176us/step - loss: 0.2418 - accuracy: 0.9153 - val_loss: 1.5509 - val_accuracy: 0.4868
Epoch 117/1000
177/177 [==============================] - 0s 193us/step - loss: 0.2442 - accuracy: 0.9209 - val_loss: 1.5742 - val_accuracy: 0.4868
Epoch 118/1000
177/177 [==============================] - 0s 103us/step - loss: 0.2377 - accuracy: 0.9322 - val_

177/177 [==============================] - 0s 56us/step - loss: 0.1719 - accuracy: 0.9322 - val_loss: 2.0512 - val_accuracy: 0.5000
Epoch 223/1000
177/177 [==============================] - 0s 67us/step - loss: 0.1708 - accuracy: 0.9266 - val_loss: 2.0188 - val_accuracy: 0.5000
Epoch 224/1000
177/177 [==============================] - ETA: 0s - loss: 0.1405 - accuracy: 0.95 - 0s 66us/step - loss: 0.1693 - accuracy: 0.9266 - val_loss: 2.0111 - val_accuracy: 0.5132
Epoch 225/1000
177/177 [==============================] - 0s 75us/step - loss: 0.1721 - accuracy: 0.9209 - val_loss: 2.0310 - val_accuracy: 0.5263
Epoch 226/1000
177/177 [==============================] - 0s 69us/step - loss: 0.1692 - accuracy: 0.9322 - val_loss: 2.0365 - val_accuracy: 0.5132
Epoch 227/1000
177/177 [==============================] - 0s 78us/step - loss: 0.1674 - accuracy: 0.9322 - val_loss: 2.0464 - val_accuracy: 0.5132
Epoch 228/1000
177/177 [==============================] - 0s 67us/step - loss: 0.1689 - acc

177/177 [==============================] - 0s 84us/step - loss: 0.1550 - accuracy: 0.9266 - val_loss: 2.3074 - val_accuracy: 0.5132
Epoch 333/1000
177/177 [==============================] - 0s 92us/step - loss: 0.1540 - accuracy: 0.9322 - val_loss: 2.3231 - val_accuracy: 0.4868
Epoch 334/1000
177/177 [==============================] - 0s 123us/step - loss: 0.1523 - accuracy: 0.9322 - val_loss: 2.3385 - val_accuracy: 0.5263
Epoch 335/1000
177/177 [==============================] - 0s 119us/step - loss: 0.1593 - accuracy: 0.9322 - val_loss: 2.3482 - val_accuracy: 0.5395
Epoch 336/1000
177/177 [==============================] - 0s 130us/step - loss: 0.1588 - accuracy: 0.9266 - val_loss: 2.3454 - val_accuracy: 0.5263
Epoch 337/1000
177/177 [==============================] - 0s 78us/step - loss: 0.1629 - accuracy: 0.9266 - val_loss: 2.3360 - val_accuracy: 0.5132
Epoch 338/1000
177/177 [==============================] - 0s 96us/step - loss: 0.1490 - accuracy: 0.9266 - val_loss: 2.3512 - val_

177/177 [==============================] - 0s 79us/step - loss: 0.1468 - accuracy: 0.9322 - val_loss: 2.4102 - val_accuracy: 0.5263
Epoch 388/1000
177/177 [==============================] - 0s 81us/step - loss: 0.1531 - accuracy: 0.9322 - val_loss: 2.4266 - val_accuracy: 0.5395
Epoch 389/1000
177/177 [==============================] - 0s 70us/step - loss: 0.1522 - accuracy: 0.9153 - val_loss: 2.4235 - val_accuracy: 0.5132
Epoch 390/1000
177/177 [==============================] - 0s 77us/step - loss: 0.1507 - accuracy: 0.9266 - val_loss: 2.4282 - val_accuracy: 0.5132
Epoch 391/1000
177/177 [==============================] - 0s 69us/step - loss: 0.1583 - accuracy: 0.9209 - val_loss: 2.4697 - val_accuracy: 0.5263
Epoch 392/1000
177/177 [==============================] - 0s 92us/step - loss: 0.1558 - accuracy: 0.9322 - val_loss: 2.4457 - val_accuracy: 0.5132
Epoch 393/1000
177/177 [==============================] - 0s 77us/step - loss: 0.1471 - accuracy: 0.9379 - val_loss: 2.4483 - val_acc

...

Epoch 610/1000
177/177 [==============================] - 0s 74us/step - loss: 0.1445 - accuracy: 0.9322 - val_loss: 2.6910 - val_accuracy: 0.5000
Epoch 611/1000
177/177 [==============================] - 0s 80us/step - loss: 0.1412 - accuracy: 0.9322 - val_loss: 2.7050 - val_accuracy: 0.4868
Epoch 612/1000
177/177 [==============================] - 0s 90us/step - loss: 0.1432 - accuracy: 0.9322 - val_loss: 2.7120 - val_accuracy: 0.5000
Epoch 613/1000
177/177 [==============================] - 0s 82us/step - loss: 0.1414 - accuracy: 0.9266 - val_loss: 2.7219 - val_accuracy: 0.5000
Epoch 614/1000
177/177 [==============================] - 0s 66us/step - loss: 0.1399 - accuracy: 0.9266 - val_loss: 2.7347 - val_accuracy: 0.5395
Epoch 615/1000
177/177 [==============================] - 0s 61us/step - loss: 0.1453 - accuracy: 0.9266 - val_loss: 2.7578 - val_accuracy: 0.5132
Epoch 616/1000
177/177 [==============================] - 0s 77us/step - loss: 0.1420 - accuracy: 0.9266 - val_loss: 2

177/177 [==============================] - 0s 94us/step - loss: 0.1414 - accuracy: 0.9209 - val_loss: 2.8394 - val_accuracy: 0.5000
Epoch 721/1000
177/177 [==============================] - 0s 95us/step - loss: 0.1389 - accuracy: 0.9266 - val_loss: 2.8260 - val_accuracy: 0.5000
Epoch 722/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1399 - accuracy: 0.9266 - val_loss: 2.8445 - val_accuracy: 0.5000
Epoch 723/1000
177/177 [==============================] - 0s 87us/step - loss: 0.1405 - accuracy: 0.9322 - val_loss: 2.8187 - val_accuracy: 0.4868
Epoch 724/1000
177/177 [==============================] - 0s 111us/step - loss: 0.1360 - accuracy: 0.9322 - val_loss: 2.8323 - val_accuracy: 0.4737
Epoch 725/1000
177/177 [==============================] - 0s 91us/step - loss: 0.1356 - accuracy: 0.9322 - val_loss: 2.8346 - val_accuracy: 0.5000
Epoch 726/1000
177/177 [==============================] - 0s 100us/step - loss: 0.1384 - accuracy: 0.9266 - val_loss: 2.8345 - val_a

177/177 [==============================] - 0s 87us/step - loss: 0.1354 - accuracy: 0.9322 - val_loss: 2.9167 - val_accuracy: 0.4868
Epoch 831/1000
177/177 [==============================] - 0s 114us/step - loss: 0.1353 - accuracy: 0.9322 - val_loss: 2.9315 - val_accuracy: 0.4868
Epoch 832/1000
177/177 [==============================] - 0s 109us/step - loss: 0.1361 - accuracy: 0.9322 - val_loss: 2.9323 - val_accuracy: 0.4737
Epoch 833/1000
177/177 [==============================] - 0s 127us/step - loss: 0.1399 - accuracy: 0.9322 - val_loss: 2.9345 - val_accuracy: 0.4737
Epoch 834/1000
177/177 [==============================] - 0s 90us/step - loss: 0.1379 - accuracy: 0.9322 - val_loss: 2.9433 - val_accuracy: 0.5000
Epoch 835/1000
177/177 [==============================] - 0s 93us/step - loss: 0.1362 - accuracy: 0.9266 - val_loss: 2.9460 - val_accuracy: 0.4868
Epoch 836/1000
177/177 [==============================] - 0s 121us/step - loss: 0.1339 - accuracy: 0.9322 - val_loss: 2.9208 - val

177/177 [==============================] - 0s 67us/step - loss: 0.1392 - accuracy: 0.9266 - val_loss: 2.9808 - val_accuracy: 0.4868
Epoch 886/1000
177/177 [==============================] - 0s 73us/step - loss: 0.1380 - accuracy: 0.9266 - val_loss: 2.9496 - val_accuracy: 0.5132
Epoch 887/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1360 - accuracy: 0.9266 - val_loss: 2.9210 - val_accuracy: 0.4868
Epoch 888/1000
177/177 [==============================] - 0s 116us/step - loss: 0.1388 - accuracy: 0.9322 - val_loss: 2.9256 - val_accuracy: 0.5000
Epoch 889/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1362 - accuracy: 0.9209 - val_loss: 2.9374 - val_accuracy: 0.4868
Epoch 890/1000
177/177 [==============================] - 0s 140us/step - loss: 0.1354 - accuracy: 0.9322 - val_loss: 2.9423 - val_accuracy: 0.5000
Epoch 891/1000
177/177 [==============================] - 0s 138us/step - loss: 0.1360 - accuracy: 0.9266 - val_loss: 2.9471 - va

177/177 [==============================] - 0s 116us/step - loss: 0.1344 - accuracy: 0.9322 - val_loss: 3.0430 - val_accuracy: 0.4737
Epoch 996/1000
177/177 [==============================] - 0s 123us/step - loss: 0.1349 - accuracy: 0.9209 - val_loss: 3.0361 - val_accuracy: 0.4737
Epoch 997/1000
177/177 [==============================] - 0s 135us/step - loss: 0.1338 - accuracy: 0.9266 - val_loss: 3.0249 - val_accuracy: 0.5000
Epoch 998/1000
177/177 [==============================] - 0s 127us/step - loss: 0.1372 - accuracy: 0.9266 - val_loss: 3.0371 - val_accuracy: 0.4737
Epoch 999/1000
177/177 [==============================] - 0s 127us/step - loss: 0.1378 - accuracy: 0.9322 - val_loss: 3.0578 - val_accuracy: 0.5000
Epoch 1000/1000
177/177 [==============================] - 0s 92us/step - loss: 0.1368 - accuracy: 0.9209 - val_loss: 3.0274 - val_accuracy: 0.4868


In [265]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 100us/step
test accuracy: 56.58%


In [98]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 2, 1, 0, 0, 2, 1, 0, 0, 0,
       0, 2, 2, 0, 0, 1, 2, 0, 0, 2, 1, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 2, 1, 0, 0, 2, 1, 0, 2, 0, 1, 0, 1, 2, 2, 1, 1,
       1, 0, 0, 2, 2, 0, 0, 2, 1, 2])

In [99]:
dat6['pred'] = pred6
dat6

,strain,test,pred
44,CFBREBSa104,1,2
170,NRS199,2,2
197,NRS233,1,0
238,SR1746,0,2
172,NRS202,2,2
...,...,...,...
92,EUH25,2,0
217,NRS260,0,0
75,CFBRSa25,0,2
156,NRS177,1,1


In [100]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [101]:
dat_proba6

,0,1,2
0,1.290039e-09,1.567630e-07,9.999999e-01
1,2.856966e-05,2.843749e-03,9.971277e-01
2,9.999446e-01,4.541289e-06,5.090974e-05
3,6.282367e-02,7.075194e-04,9.364688e-01
4,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...
71,8.950077e-01,2.174225e-02,8.325008e-02
72,9.952662e-01,4.733360e-03,4.967864e-07
73,1.997957e-01,2.891268e-01,5.110775e-01
74,1.604341e-08,1.000000e+00,4.449736e-09


In [102]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [103]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p17.csv", index = False,
         header=None)

In [269]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 98us/step - loss: 0.1433 - accuracy: 0.9153 - val_loss: 2.8802 - val_accuracy: 0.5789
Epoch 2/1000
177/177 [==============================] - 0s 82us/step - loss: 0.1407 - accuracy: 0.9266 - val_loss: 2.8574 - val_accuracy: 0.5395
Epoch 3/1000
177/177 [==============================] - 0s 75us/step - loss: 0.1396 - accuracy: 0.9266 - val_loss: 2.8628 - val_accuracy: 0.5263
Epoch 4/1000
177/177 [==============================] - 0s 126us/step - loss: 0.1403 - accuracy: 0.9322 - val_loss: 2.8849 - val_accuracy: 0.5263
Epoch 5/1000
177/177 [==============================] - 0s 118us/step - loss: 0.1388 - accuracy: 0.9266 - val_loss: 2.8959 - val_accuracy: 0.5395
Epoch 6/1000
177/177 [==============================] - 0s 104us/step - loss: 0.1395 - accuracy: 0.9153 - val_loss: 2.8872 - val_accuracy: 0.5526
Epoch 7/1000
177/177 [==============================] - 0s 123us/step - loss: 0.14

Epoch 57/1000
177/177 [==============================] - 0s 87us/step - loss: 0.1403 - accuracy: 0.9266 - val_loss: 2.9145 - val_accuracy: 0.5263
Epoch 58/1000
177/177 [==============================] - 0s 82us/step - loss: 0.1460 - accuracy: 0.9266 - val_loss: 2.9246 - val_accuracy: 0.5658
Epoch 59/1000
177/177 [==============================] - 0s 101us/step - loss: 0.1433 - accuracy: 0.9322 - val_loss: 2.9342 - val_accuracy: 0.5526
Epoch 60/1000
177/177 [==============================] - 0s 86us/step - loss: 0.1368 - accuracy: 0.9322 - val_loss: 2.9653 - val_accuracy: 0.5263
Epoch 61/1000
177/177 [==============================] - 0s 109us/step - loss: 0.1476 - accuracy: 0.9266 - val_loss: 2.9106 - val_accuracy: 0.5526
Epoch 62/1000
177/177 [==============================] - 0s 87us/step - loss: 0.1397 - accuracy: 0.9266 - val_loss: 2.9152 - val_accuracy: 0.5526
Epoch 63/1000
177/177 [==============================] - 0s 84us/step - loss: 0.1441 - accuracy: 0.9322 - val_loss: 2.9473

Epoch 113/1000
177/177 [==============================] - 0s 67us/step - loss: 0.1425 - accuracy: 0.9322 - val_loss: 2.9728 - val_accuracy: 0.5395
Epoch 114/1000
177/177 [==============================] - 0s 62us/step - loss: 0.1384 - accuracy: 0.9209 - val_loss: 2.9435 - val_accuracy: 0.5658
Epoch 115/1000
177/177 [==============================] - 0s 339us/step - loss: 0.1372 - accuracy: 0.9266 - val_loss: 2.9683 - val_accuracy: 0.5526
Epoch 116/1000
177/177 [==============================] - 0s 56us/step - loss: 0.1436 - accuracy: 0.9322 - val_loss: 2.9905 - val_accuracy: 0.5658
Epoch 117/1000
177/177 [==============================] - 0s 53us/step - loss: 0.1342 - accuracy: 0.9322 - val_loss: 2.9721 - val_accuracy: 0.5395
Epoch 118/1000
177/177 [==============================] - 0s 101us/step - loss: 0.1429 - accuracy: 0.9266 - val_loss: 2.9641 - val_accuracy: 0.5658
Epoch 119/1000
177/177 [==============================] - 0s 219us/step - loss: 0.1353 - accuracy: 0.9322 - val_loss

Epoch 224/1000
177/177 [==============================] - 0s 84us/step - loss: 0.1371 - accuracy: 0.9322 - val_loss: 2.9861 - val_accuracy: 0.5395
Epoch 225/1000
177/177 [==============================] - 0s 86us/step - loss: 0.1428 - accuracy: 0.9209 - val_loss: 2.9843 - val_accuracy: 0.5395
Epoch 226/1000
177/177 [==============================] - 0s 75us/step - loss: 0.1301 - accuracy: 0.9322 - val_loss: 3.0063 - val_accuracy: 0.5395
Epoch 227/1000
177/177 [==============================] - 0s 87us/step - loss: 0.1588 - accuracy: 0.9266 - val_loss: 3.0461 - val_accuracy: 0.5526
Epoch 228/1000
177/177 [==============================] - 0s 81us/step - loss: 0.1479 - accuracy: 0.9322 - val_loss: 3.0298 - val_accuracy: 0.5526
Epoch 229/1000
177/177 [==============================] - 0s 200us/step - loss: 0.1528 - accuracy: 0.9209 - val_loss: 3.0186 - val_accuracy: 0.5658
Epoch 230/1000
177/177 [==============================] - 0s 268us/step - loss: 0.1285 - accuracy: 0.9435 - val_loss:

Epoch 335/1000
177/177 [==============================] - 0s 91us/step - loss: 0.1348 - accuracy: 0.9322 - val_loss: 3.0002 - val_accuracy: 0.5658
Epoch 336/1000
177/177 [==============================] - 0s 94us/step - loss: 0.1377 - accuracy: 0.9209 - val_loss: 3.0027 - val_accuracy: 0.5526
Epoch 337/1000
177/177 [==============================] - 0s 75us/step - loss: 0.1449 - accuracy: 0.9153 - val_loss: 3.0165 - val_accuracy: 0.5263
Epoch 338/1000
177/177 [==============================] - 0s 83us/step - loss: 0.1410 - accuracy: 0.9322 - val_loss: 3.0508 - val_accuracy: 0.5526
Epoch 339/1000
177/177 [==============================] - 0s 74us/step - loss: 0.1383 - accuracy: 0.9209 - val_loss: 3.0354 - val_accuracy: 0.5395
Epoch 340/1000
177/177 [==============================] - 0s 72us/step - loss: 0.1361 - accuracy: 0.9322 - val_loss: 3.0238 - val_accuracy: 0.5658
Epoch 341/1000
177/177 [==============================] - 0s 80us/step - loss: 0.1358 - accuracy: 0.9266 - val_loss: 3

...

177/177 [==============================] - 0s 70us/step - loss: 0.1383 - accuracy: 0.9266 - val_loss: 3.1641 - val_accuracy: 0.5526
Epoch 668/1000
177/177 [==============================] - 0s 93us/step - loss: 0.1391 - accuracy: 0.9322 - val_loss: 3.1622 - val_accuracy: 0.5526
Epoch 669/1000
177/177 [==============================] - 0s 86us/step - loss: 0.1378 - accuracy: 0.9209 - val_loss: 3.1948 - val_accuracy: 0.5526
Epoch 670/1000
177/177 [==============================] - 0s 118us/step - loss: 0.1379 - accuracy: 0.9322 - val_loss: 3.2461 - val_accuracy: 0.5395
Epoch 671/1000
177/177 [==============================] - 0s 125us/step - loss: 0.1413 - accuracy: 0.9322 - val_loss: 3.2193 - val_accuracy: 0.5395
Epoch 672/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1334 - accuracy: 0.9322 - val_loss: 3.1703 - val_accuracy: 0.5395
Epoch 673/1000
177/177 [==============================] - 0s 116us/step - loss: 0.1477 - accuracy: 0.9096 - val_loss: 3.1597 - val

Epoch 723/1000
177/177 [==============================] - 0s 74us/step - loss: 0.1367 - accuracy: 0.9266 - val_loss: 3.1963 - val_accuracy: 0.5395
Epoch 724/1000
177/177 [==============================] - 0s 95us/step - loss: 0.1353 - accuracy: 0.9322 - val_loss: 3.1835 - val_accuracy: 0.5658
Epoch 725/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1440 - accuracy: 0.9322 - val_loss: 3.1924 - val_accuracy: 0.5395
Epoch 726/1000
177/177 [==============================] - 0s 110us/step - loss: 0.1331 - accuracy: 0.9322 - val_loss: 3.2053 - val_accuracy: 0.5658
Epoch 727/1000
177/177 [==============================] - 0s 76us/step - loss: 0.1404 - accuracy: 0.9322 - val_loss: 3.2105 - val_accuracy: 0.5395
Epoch 728/1000
177/177 [==============================] - 0s 80us/step - loss: 0.1398 - accuracy: 0.9322 - val_loss: 3.1888 - val_accuracy: 0.5395
Epoch 729/1000
177/177 [==============================] - 0s 91us/step - loss: 0.1310 - accuracy: 0.9209 - val_loss: 

Epoch 778/1000
177/177 [==============================] - 0s 97us/step - loss: 0.1414 - accuracy: 0.9266 - val_loss: 3.1771 - val_accuracy: 0.5658
Epoch 779/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1344 - accuracy: 0.9322 - val_loss: 3.1994 - val_accuracy: 0.5263
Epoch 780/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1478 - accuracy: 0.9096 - val_loss: 3.2199 - val_accuracy: 0.5658
Epoch 781/1000
177/177 [==============================] - 0s 149us/step - loss: 0.1385 - accuracy: 0.9322 - val_loss: 3.1855 - val_accuracy: 0.5526
Epoch 782/1000
177/177 [==============================] - 0s 112us/step - loss: 0.1367 - accuracy: 0.9322 - val_loss: 3.1720 - val_accuracy: 0.5526
Epoch 783/1000
177/177 [==============================] - 0s 93us/step - loss: 0.1427 - accuracy: 0.9266 - val_loss: 3.1739 - val_accuracy: 0.5395
Epoch 784/1000
177/177 [==============================] - 0s 139us/step - loss: 0.1337 - accuracy: 0.9322 - val_los

177/177 [==============================] - 0s 118us/step - loss: 0.1426 - accuracy: 0.9209 - val_loss: 3.2827 - val_accuracy: 0.5395
Epoch 889/1000
177/177 [==============================] - 0s 90us/step - loss: 0.1399 - accuracy: 0.9266 - val_loss: 3.3187 - val_accuracy: 0.5658
Epoch 890/1000
177/177 [==============================] - 0s 101us/step - loss: 0.1395 - accuracy: 0.9322 - val_loss: 3.3198 - val_accuracy: 0.5658
Epoch 891/1000
177/177 [==============================] - 0s 104us/step - loss: 0.1376 - accuracy: 0.9322 - val_loss: 3.2974 - val_accuracy: 0.5658
Epoch 892/1000
177/177 [==============================] - 0s 116us/step - loss: 0.1381 - accuracy: 0.9209 - val_loss: 3.2163 - val_accuracy: 0.5526
Epoch 893/1000
177/177 [==============================] - 0s 102us/step - loss: 0.1377 - accuracy: 0.9379 - val_loss: 3.1699 - val_accuracy: 0.5658
Epoch 894/1000
177/177 [==============================] - 0s 108us/step - loss: 0.1379 - accuracy: 0.9322 - val_loss: 3.1246 - v

177/177 [==============================] - 0s 83us/step - loss: 0.1324 - accuracy: 0.9322 - val_loss: 3.3091 - val_accuracy: 0.5658
Epoch 999/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1375 - accuracy: 0.9322 - val_loss: 3.3185 - val_accuracy: 0.5395
Epoch 1000/1000
177/177 [==============================] - 0s 100us/step - loss: 0.1358 - accuracy: 0.9266 - val_loss: 3.3006 - val_accuracy: 0.5526


In [270]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 92.76%


In [87]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [88]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [89]:
y_prob6 = df_proba6[df_proba6['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.2900386e-09, 1.5676302e-07, 9.9999990e-01],
       [2.8569655e-05, 2.8437492e-03, 9.9712770e-01],
       [9.9994457e-01, 4.5412890e-06, 5.0909744e-05],
       [6.2823670e-02, 7.0751936e-04, 9.3646884e-01],
       [8.2295890e-03, 2.1639080e-05, 9.9174880e-01],
       [9.2230283e-07, 7.9983440e-06, 9.9999106e-01],
       [1.2991899e-09, 7.9475770e-10, 1.0000000e+00],
       [6.4736270e-05, 9.9138110e-01, 8.5541240e-03],
       [2.6940496e-08, 4.0668684e-01, 5.9331316e-01],
       [4.8442390e-01, 5.1162860e-01, 3.9474520e-03],
       [6.7524440e-04, 2.1309265e-01, 7.8623220e-01],
       [2.8351959e-09, 4.3906280e-06, 9.9999560e-01],
       [9.3648255e-01, 6.3293840e-02, 2.2363788e-04],
       [2.2122312e-07, 6.7032292e-06, 9.9999310e-01],
       [1.7241009e-06, 9.9889860e-01, 1.0996391e-03],
       [5.4844330e-01, 4.4461897e-01, 6.9377710e-03],
       [5.7867450e-01, 4.2132310e-01, 2.4238445e-06],
       [2.0358592e-04, 6.0059372e-02, 9.3973710e-01],
       [4.6933986e-09, 9.999

In [90]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.693313520694473

In [91]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.693313520694473

In [92]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [93]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [94]:
dat7

,strain,test
222,NRS271,1
242,SR2852,2
34,CA39,2
221,NRS266,1
230,NY356,2
...,...,...
105,MN055,2
67,CFBRSa04,0
235,SR1065,0
190,NRS224,1


In [95]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [96]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [97]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [110]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 663us/step - loss: 1.1034 - accuracy: 0.4294 - val_loss: 1.1676 - val_accuracy: 0.3553
Epoch 2/1000
177/177 [==============================] - 0s 107us/step - loss: 1.0217 - accuracy: 0.5141 - val_loss: 1.1859 - val_accuracy: 0.3421
Epoch 3/1000
177/177 [==============================] - 0s 145us/step - loss: 0.9712 - accuracy: 0.5367 - val_loss: 1.1259 - val_accuracy: 0.4474
Epoch 4/1000
177/177 [==============================] - 0s 129us/step - loss: 0.9216 - accuracy: 0.5480 - val_loss: 1.0906 - val_accuracy: 0.4474
Epoch 5/1000
177/177 [==============================] - 0s 156us/step - loss: 0.8974 - accuracy: 0.5819 - val_loss: 1.0800 - val_accuracy: 0.4474
Epoch 6/1000
177/177 [==============================] - 0s 176us/step - loss: 0.8731 - accuracy: 0.5932 - val_loss: 1.0802 - val_accuracy: 0.3947
Epoch 7/1000
177/177 [==============================] - 0s 816us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 160us/step - loss: 0.3110 - accuracy: 0.8927 - val_loss: 1.6731 - val_accuracy: 0.4079
Epoch 113/1000
177/177 [==============================] - 0s 231us/step - loss: 0.3013 - accuracy: 0.8983 - val_loss: 1.6079 - val_accuracy: 0.3947
Epoch 114/1000
177/177 [==============================] - 0s 550us/step - loss: 0.3017 - accuracy: 0.8983 - val_loss: 1.6619 - val_accuracy: 0.4342
Epoch 115/1000
177/177 [==============================] - 0s 118us/step - loss: 0.3017 - accuracy: 0.8983 - val_loss: 1.6836 - val_accuracy: 0.4211
Epoch 116/1000
177/177 [==============================] - 0s 111us/step - loss: 0.2931 - accuracy: 0.9040 - val_loss: 1.6568 - val_accuracy: 0.4079
Epoch 117/1000
177/177 [==============================] - 0s 128us/step - loss: 0.2978 - accuracy: 0.8983 - val_loss: 1.6785 - val_accuracy: 0.3947
Epoch 118/1000
177/177 [==============================] - 0s 144us/step - loss: 0.2912 - accuracy: 0.9096 - val_

177/177 [==============================] - 0s 140us/step - loss: 0.2134 - accuracy: 0.9153 - val_loss: 2.0976 - val_accuracy: 0.4342
Epoch 223/1000
177/177 [==============================] - 0s 328us/step - loss: 0.2155 - accuracy: 0.8814 - val_loss: 2.0810 - val_accuracy: 0.3816
Epoch 224/1000
177/177 [==============================] - 0s 287us/step - loss: 0.2119 - accuracy: 0.9153 - val_loss: 2.1159 - val_accuracy: 0.4211
Epoch 225/1000
177/177 [==============================] - 0s 116us/step - loss: 0.2135 - accuracy: 0.9040 - val_loss: 2.0925 - val_accuracy: 0.3816
Epoch 226/1000
177/177 [==============================] - 0s 111us/step - loss: 0.2122 - accuracy: 0.8983 - val_loss: 2.0982 - val_accuracy: 0.3947
Epoch 227/1000
177/177 [==============================] - 0s 136us/step - loss: 0.2126 - accuracy: 0.9096 - val_loss: 2.0540 - val_accuracy: 0.4342
Epoch 228/1000
177/177 [==============================] - 0s 136us/step - loss: 0.2091 - accuracy: 0.9096 - val_loss: 2.1538 - 

177/177 [==============================] - 0s 153us/step - loss: 0.1908 - accuracy: 0.9096 - val_loss: 2.4431 - val_accuracy: 0.4079
Epoch 333/1000
177/177 [==============================] - 0s 183us/step - loss: 0.2002 - accuracy: 0.9096 - val_loss: 2.3157 - val_accuracy: 0.4079
Epoch 334/1000
177/177 [==============================] - 0s 138us/step - loss: 0.2051 - accuracy: 0.9209 - val_loss: 2.4507 - val_accuracy: 0.3947
Epoch 335/1000
177/177 [==============================] - 0s 141us/step - loss: 0.1869 - accuracy: 0.9266 - val_loss: 2.5626 - val_accuracy: 0.4211
Epoch 336/1000
177/177 [==============================] - 0s 157us/step - loss: 0.1923 - accuracy: 0.9209 - val_loss: 2.5271 - val_accuracy: 0.4079
Epoch 337/1000
177/177 [==============================] - 0s 213us/step - loss: 0.1843 - accuracy: 0.9266 - val_loss: 2.4792 - val_accuracy: 0.3947
Epoch 338/1000
177/177 [==============================] - 0s 141us/step - loss: 0.1858 - accuracy: 0.9096 - val_loss: 2.4458 - 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.113321). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


...

177/177 [==============================] - 0s 182us/step - loss: 0.1718 - accuracy: 0.9209 - val_loss: 2.7671 - val_accuracy: 0.3947
Epoch 454/1000
177/177 [==============================] - 0s 139us/step - loss: 0.1729 - accuracy: 0.9209 - val_loss: 2.7282 - val_accuracy: 0.3947
Epoch 455/1000
177/177 [==============================] - 0s 119us/step - loss: 0.1718 - accuracy: 0.9040 - val_loss: 2.7419 - val_accuracy: 0.4079
Epoch 456/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1704 - accuracy: 0.9153 - val_loss: 2.7320 - val_accuracy: 0.3947
Epoch 457/1000
177/177 [==============================] - 0s 125us/step - loss: 0.1627 - accuracy: 0.9322 - val_loss: 2.7589 - val_accuracy: 0.4079
Epoch 458/1000
177/177 [==============================] - 0s 98us/step - loss: 0.1710 - accuracy: 0.9266 - val_loss: 2.7225 - val_accuracy: 0.3947
Epoch 459/1000
177/177 [==============================] - 0s 110us/step - loss: 0.1628 - accuracy: 0.9266 - val_loss: 2.8149 - v

177/177 [==============================] - 0s 119us/step - loss: 0.1595 - accuracy: 0.9322 - val_loss: 3.0346 - val_accuracy: 0.3947
Epoch 564/1000
177/177 [==============================] - 0s 151us/step - loss: 0.1600 - accuracy: 0.9153 - val_loss: 3.0209 - val_accuracy: 0.4079
Epoch 565/1000
177/177 [==============================] - 0s 114us/step - loss: 0.1585 - accuracy: 0.9266 - val_loss: 3.0044 - val_accuracy: 0.3816
Epoch 566/1000
177/177 [==============================] - 0s 201us/step - loss: 0.1590 - accuracy: 0.9209 - val_loss: 3.0652 - val_accuracy: 0.4079
Epoch 567/1000
177/177 [==============================] - 0s 132us/step - loss: 0.1574 - accuracy: 0.9266 - val_loss: 3.0687 - val_accuracy: 0.3947
Epoch 568/1000
177/177 [==============================] - 0s 185us/step - loss: 0.1751 - accuracy: 0.9153 - val_loss: 3.0728 - val_accuracy: 0.3947
Epoch 569/1000
177/177 [==============================] - 0s 139us/step - loss: 0.1556 - accuracy: 0.9322 - val_loss: 3.0217 - 

177/177 [==============================] - 0s 148us/step - loss: 0.1588 - accuracy: 0.9096 - val_loss: 3.2657 - val_accuracy: 0.4079
Epoch 674/1000
177/177 [==============================] - 0s 133us/step - loss: 0.1549 - accuracy: 0.9096 - val_loss: 3.2432 - val_accuracy: 0.4079
Epoch 675/1000
177/177 [==============================] - 0s 119us/step - loss: 0.1562 - accuracy: 0.9153 - val_loss: 3.2537 - val_accuracy: 0.3947
Epoch 676/1000
177/177 [==============================] - 0s 169us/step - loss: 0.1536 - accuracy: 0.9209 - val_loss: 3.2344 - val_accuracy: 0.4079
Epoch 677/1000
177/177 [==============================] - 0s 138us/step - loss: 0.1597 - accuracy: 0.9096 - val_loss: 3.2061 - val_accuracy: 0.3947
Epoch 678/1000
177/177 [==============================] - 0s 188us/step - loss: 0.1555 - accuracy: 0.9040 - val_loss: 3.2612 - val_accuracy: 0.4079
Epoch 679/1000
177/177 [==============================] - 0s 142us/step - loss: 0.1536 - accuracy: 0.9322 - val_loss: 3.2773 - 

177/177 [==============================] - 0s 113us/step - loss: 0.1732 - accuracy: 0.9209 - val_loss: 3.5676 - val_accuracy: 0.4079
Epoch 784/1000
177/177 [==============================] - 0s 122us/step - loss: 0.1629 - accuracy: 0.9266 - val_loss: 3.4472 - val_accuracy: 0.3553
Epoch 785/1000
177/177 [==============================] - 0s 122us/step - loss: 0.1582 - accuracy: 0.9266 - val_loss: 3.3585 - val_accuracy: 0.3553
Epoch 786/1000
177/177 [==============================] - 0s 138us/step - loss: 0.1523 - accuracy: 0.9153 - val_loss: 3.5194 - val_accuracy: 0.4079
Epoch 787/1000
177/177 [==============================] - 0s 349us/step - loss: 0.1639 - accuracy: 0.9096 - val_loss: 3.4523 - val_accuracy: 0.3947
Epoch 788/1000
177/177 [==============================] - 0s 583us/step - loss: 0.1499 - accuracy: 0.9322 - val_loss: 3.3877 - val_accuracy: 0.3947
Epoch 789/1000
177/177 [==============================] - 0s 167us/step - loss: 0.1598 - accuracy: 0.9096 - val_loss: 3.3647 - 

177/177 [==============================] - 0s 143us/step - loss: 0.1522 - accuracy: 0.9096 - val_loss: 3.6769 - val_accuracy: 0.4079
Epoch 894/1000
177/177 [==============================] - 0s 139us/step - loss: 0.1589 - accuracy: 0.9266 - val_loss: 3.5395 - val_accuracy: 0.3947
Epoch 895/1000
177/177 [==============================] - 0s 322us/step - loss: 0.1553 - accuracy: 0.9266 - val_loss: 3.5297 - val_accuracy: 0.3947
Epoch 896/1000
177/177 [==============================] - 0s 204us/step - loss: 0.1497 - accuracy: 0.9096 - val_loss: 3.6457 - val_accuracy: 0.3684
Epoch 897/1000
177/177 [==============================] - 0s 162us/step - loss: 0.1610 - accuracy: 0.9266 - val_loss: 3.6194 - val_accuracy: 0.3816
Epoch 898/1000
177/177 [==============================] - 0s 129us/step - loss: 0.1649 - accuracy: 0.9153 - val_loss: 3.5673 - val_accuracy: 0.3816
Epoch 899/1000
177/177 [==============================] - 0s 145us/step - loss: 0.1491 - accuracy: 0.9209 - val_loss: 3.5689 - 

In [298]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 75us/step
test accuracy: 48.68%


In [111]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([0, 2, 0, 2, 2, 0, 0, 2, 1, 0, 2, 1, 2, 0, 1, 0, 2, 0, 2, 2, 2, 0,
       0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 0,
       0, 1, 0, 0, 0, 2, 0, 2, 2, 0, 1, 0, 2, 0, 2, 2, 0, 2, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 2, 0, 2, 2, 2])

In [112]:
dat7['pred'] = pred7
dat7

,strain,test,pred
222,NRS271,1,0
242,SR2852,2,2
34,CA39,2,0
221,NRS266,1,2
230,NY356,2,2
...,...,...,...
105,MN055,2,2
67,CFBRSa04,0,0
235,SR1065,0,2
190,NRS224,1,2


In [113]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [114]:
dat_proba7

,0,1,2
0,0.854929,1.449402e-01,0.000130
1,0.000001,7.516648e-05,0.999923
2,0.959992,1.340566e-02,0.026602
3,0.000018,2.691879e-06,0.999980
4,0.009156,4.119915e-01,0.578852
...,...,...,...
71,0.193116,3.407584e-01,0.466125
72,0.928794,6.980573e-02,0.001400
73,0.000002,1.625966e-09,0.999998
74,0.156228,2.335873e-04,0.843538


In [115]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [116]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p17.csv", index = False,
         header=None)

In [302]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 130us/step - loss: 0.1720 - accuracy: 0.9096 - val_loss: 4.1006 - val_accuracy: 0.4605
Epoch 2/1000
177/177 [==============================] - 0s 113us/step - loss: 0.1691 - accuracy: 0.9096 - val_loss: 4.2049 - val_accuracy: 0.4342
Epoch 3/1000
177/177 [==============================] - 0s 155us/step - loss: 0.1510 - accuracy: 0.9153 - val_loss: 4.4592 - val_accuracy: 0.4737
Epoch 4/1000
177/177 [==============================] - 0s 156us/step - loss: 0.1999 - accuracy: 0.9096 - val_loss: 4.1915 - val_accuracy: 0.4342
Epoch 5/1000
177/177 [==============================] - 0s 160us/step - loss: 0.1699 - accuracy: 0.9266 - val_loss: 3.8094 - val_accuracy: 0.4079
Epoch 6/1000
177/177 [==============================] - 0s 143us/step - loss: 0.1537 - accuracy: 0.9266 - val_loss: 4.1998 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 139us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 98us/step - loss: 0.1769 - accuracy: 0.9096 - val_loss: 4.2668 - val_accuracy: 0.4342
Epoch 113/1000
177/177 [==============================] - 0s 122us/step - loss: 0.1622 - accuracy: 0.9209 - val_loss: 4.2180 - val_accuracy: 0.4342
Epoch 114/1000
177/177 [==============================] - 0s 125us/step - loss: 0.1489 - accuracy: 0.9209 - val_loss: 4.3662 - val_accuracy: 0.4737
Epoch 115/1000
177/177 [==============================] - 0s 123us/step - loss: 0.1633 - accuracy: 0.9153 - val_loss: 4.1732 - val_accuracy: 0.4474
Epoch 116/1000
177/177 [==============================] - 0s 156us/step - loss: 0.1523 - accuracy: 0.9153 - val_loss: 4.1757 - val_accuracy: 0.4079
Epoch 117/1000
177/177 [==============================] - 0s 128us/step - loss: 0.1467 - accuracy: 0.9209 - val_loss: 4.3608 - val_accuracy: 0.4474
Epoch 118/1000
177/177 [==============================] - 0s 165us/step - loss: 0.1522 - accuracy: 0.9266 - val_l

Epoch 222/1000
177/177 [==============================] - 0s 128us/step - loss: 0.1639 - accuracy: 0.9266 - val_loss: 4.4613 - val_accuracy: 0.4211
Epoch 223/1000
177/177 [==============================] - 0s 128us/step - loss: 0.1587 - accuracy: 0.9209 - val_loss: 4.6797 - val_accuracy: 0.4737
Epoch 224/1000
177/177 [==============================] - 0s 183us/step - loss: 0.1594 - accuracy: 0.9153 - val_loss: 4.6308 - val_accuracy: 0.4737
Epoch 225/1000
177/177 [==============================] - 0s 180us/step - loss: 0.1574 - accuracy: 0.9209 - val_loss: 4.5129 - val_accuracy: 0.4474
Epoch 226/1000
177/177 [==============================] - 0s 151us/step - loss: 0.1525 - accuracy: 0.9209 - val_loss: 4.4798 - val_accuracy: 0.4342
Epoch 227/1000
177/177 [==============================] - 0s 295us/step - loss: 0.1574 - accuracy: 0.9209 - val_loss: 4.4565 - val_accuracy: 0.4079
Epoch 228/1000
177/177 [==============================] - 0s 351us/step - loss: 0.1667 - accuracy: 0.9266 - val_

177/177 [==============================] - 0s 159us/step - loss: 0.1545 - accuracy: 0.9266 - val_loss: 4.7800 - val_accuracy: 0.4474
Epoch 333/1000
177/177 [==============================] - 0s 129us/step - loss: 0.1650 - accuracy: 0.9096 - val_loss: 4.7425 - val_accuracy: 0.4474
Epoch 334/1000
177/177 [==============================] - 0s 186us/step - loss: 0.1603 - accuracy: 0.9209 - val_loss: 4.6073 - val_accuracy: 0.4211
Epoch 335/1000
177/177 [==============================] - 0s 142us/step - loss: 0.1596 - accuracy: 0.9096 - val_loss: 4.6946 - val_accuracy: 0.4737
Epoch 336/1000
177/177 [==============================] - 0s 139us/step - loss: 0.1584 - accuracy: 0.9266 - val_loss: 4.7022 - val_accuracy: 0.4605
Epoch 337/1000
177/177 [==============================] - 0s 112us/step - loss: 0.1418 - accuracy: 0.9322 - val_loss: 4.6331 - val_accuracy: 0.4079
Epoch 338/1000
177/177 [==============================] - 0s 136us/step - loss: 0.1533 - accuracy: 0.9266 - val_loss: 4.5975 - 

177/177 [==============================] - 0s 331us/step - loss: 0.1533 - accuracy: 0.9153 - val_loss: 4.8606 - val_accuracy: 0.4342
Epoch 443/1000
177/177 [==============================] - 0s 541us/step - loss: 0.1444 - accuracy: 0.9322 - val_loss: 4.7589 - val_accuracy: 0.4211
Epoch 444/1000
177/177 [==============================] - 0s 269us/step - loss: 0.1565 - accuracy: 0.9266 - val_loss: 4.7555 - val_accuracy: 0.4211
Epoch 445/1000
177/177 [==============================] - 0s 222us/step - loss: 0.1469 - accuracy: 0.9209 - val_loss: 4.9061 - val_accuracy: 0.4474
Epoch 446/1000
177/177 [==============================] - 0s 316us/step - loss: 0.1640 - accuracy: 0.9266 - val_loss: 4.8687 - val_accuracy: 0.4211
Epoch 447/1000
177/177 [==============================] - 0s 175us/step - loss: 0.1584 - accuracy: 0.9209 - val_loss: 4.6706 - val_accuracy: 0.4079
Epoch 448/1000
177/177 [==============================] - 0s 108us/step - loss: 0.1547 - accuracy: 0.9153 - val_loss: 4.8331 - 

177/177 [==============================] - 0s 284us/step - loss: 0.1431 - accuracy: 0.9266 - val_loss: 5.0985 - val_accuracy: 0.4868
Epoch 553/1000
177/177 [==============================] - 0s 151us/step - loss: 0.1634 - accuracy: 0.9153 - val_loss: 5.0471 - val_accuracy: 0.4605
Epoch 554/1000
177/177 [==============================] - 0s 121us/step - loss: 0.1587 - accuracy: 0.9209 - val_loss: 4.9174 - val_accuracy: 0.4079
Epoch 555/1000
177/177 [==============================] - 0s 113us/step - loss: 0.1478 - accuracy: 0.9153 - val_loss: 4.9117 - val_accuracy: 0.4079
Epoch 556/1000
177/177 [==============================] - 0s 164us/step - loss: 0.1452 - accuracy: 0.9266 - val_loss: 4.9148 - val_accuracy: 0.4079
Epoch 557/1000
177/177 [==============================] - 0s 117us/step - loss: 0.1525 - accuracy: 0.9153 - val_loss: 4.9423 - val_accuracy: 0.4211
Epoch 558/1000
177/177 [==============================] - 0s 127us/step - loss: 0.1465 - accuracy: 0.9266 - val_loss: 4.9933 - 

177/177 [==============================] - 0s 165us/step - loss: 0.1599 - accuracy: 0.9209 - val_loss: 5.0824 - val_accuracy: 0.4211
Epoch 663/1000
177/177 [==============================] - 0s 169us/step - loss: 0.1550 - accuracy: 0.8983 - val_loss: 5.1341 - val_accuracy: 0.4211
Epoch 664/1000
177/177 [==============================] - 0s 266us/step - loss: 0.1450 - accuracy: 0.9266 - val_loss: 5.1516 - val_accuracy: 0.4211
Epoch 665/1000
177/177 [==============================] - 0s 179us/step - loss: 0.1474 - accuracy: 0.9209 - val_loss: 5.1379 - val_accuracy: 0.4079
Epoch 666/1000
177/177 [==============================] - 0s 170us/step - loss: 0.1523 - accuracy: 0.9209 - val_loss: 5.1779 - val_accuracy: 0.4211
Epoch 667/1000
177/177 [==============================] - 0s 165us/step - loss: 0.1444 - accuracy: 0.9266 - val_loss: 5.0816 - val_accuracy: 0.4079
Epoch 668/1000
177/177 [==============================] - 0s 137us/step - loss: 0.1618 - accuracy: 0.9266 - val_loss: 5.0957 - 

177/177 [==============================] - 0s 154us/step - loss: 0.1475 - accuracy: 0.9096 - val_loss: 5.1939 - val_accuracy: 0.4079
Epoch 773/1000
177/177 [==============================] - 0s 129us/step - loss: 0.1517 - accuracy: 0.9096 - val_loss: 5.2689 - val_accuracy: 0.4211
Epoch 774/1000
177/177 [==============================] - 0s 130us/step - loss: 0.1505 - accuracy: 0.9209 - val_loss: 5.1920 - val_accuracy: 0.4079
Epoch 775/1000
177/177 [==============================] - 0s 147us/step - loss: 0.1475 - accuracy: 0.9153 - val_loss: 5.2517 - val_accuracy: 0.4079
Epoch 776/1000
177/177 [==============================] - 0s 193us/step - loss: 0.1435 - accuracy: 0.9266 - val_loss: 5.3748 - val_accuracy: 0.4079
Epoch 777/1000
177/177 [==============================] - 0s 168us/step - loss: 0.1504 - accuracy: 0.9153 - val_loss: 5.2890 - val_accuracy: 0.4079
Epoch 778/1000
177/177 [==============================] - 0s 175us/step - loss: 0.1509 - accuracy: 0.9209 - val_loss: 5.1799 - 

177/177 [==============================] - 0s 134us/step - loss: 0.1476 - accuracy: 0.9153 - val_loss: 5.3425 - val_accuracy: 0.4342
Epoch 883/1000
177/177 [==============================] - 0s 121us/step - loss: 0.1459 - accuracy: 0.9153 - val_loss: 5.3350 - val_accuracy: 0.4079
Epoch 884/1000
177/177 [==============================] - 0s 117us/step - loss: 0.1452 - accuracy: 0.9266 - val_loss: 5.3368 - val_accuracy: 0.4079
Epoch 885/1000
177/177 [==============================] - 0s 134us/step - loss: 0.1472 - accuracy: 0.9153 - val_loss: 5.3264 - val_accuracy: 0.3947
Epoch 886/1000
177/177 [==============================] - 0s 138us/step - loss: 0.1439 - accuracy: 0.9209 - val_loss: 5.3591 - val_accuracy: 0.4211
Epoch 887/1000
177/177 [==============================] - 0s 130us/step - loss: 0.1478 - accuracy: 0.9322 - val_loss: 5.3838 - val_accuracy: 0.4079
Epoch 888/1000
177/177 [==============================] - 0s 157us/step - loss: 0.1442 - accuracy: 0.9209 - val_loss: 5.3302 - 

177/177 [==============================] - 0s 151us/step - loss: 0.1547 - accuracy: 0.9209 - val_loss: 5.5536 - val_accuracy: 0.4211
Epoch 993/1000
177/177 [==============================] - 0s 143us/step - loss: 0.1499 - accuracy: 0.9209 - val_loss: 5.5530 - val_accuracy: 0.4211
Epoch 994/1000
177/177 [==============================] - 0s 158us/step - loss: 0.1495 - accuracy: 0.9209 - val_loss: 5.5217 - val_accuracy: 0.3947
Epoch 995/1000
177/177 [==============================] - 0s 177us/step - loss: 0.1522 - accuracy: 0.9209 - val_loss: 5.5284 - val_accuracy: 0.3947
Epoch 996/1000
177/177 [==============================] - 0s 140us/step - loss: 0.1509 - accuracy: 0.9266 - val_loss: 5.7149 - val_accuracy: 0.4211
Epoch 997/1000
177/177 [==============================] - 0s 169us/step - loss: 0.1534 - accuracy: 0.9266 - val_loss: 5.6161 - val_accuracy: 0.4079
Epoch 998/1000
177/177 [==============================] - 0s 155us/step - loss: 0.1523 - accuracy: 0.9266 - val_loss: 5.5539 - 

In [303]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 91.93%


In [98]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [99]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [100]:
y_prob7 = df_proba7[df_proba7['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[8.54929400e-01, 1.44940150e-01, 1.30449070e-04],
       [1.49560550e-06, 7.51664850e-05, 9.99923350e-01],
       [9.59992230e-01, 1.34056640e-02, 2.66021860e-02],
       [1.75608950e-05, 2.69187940e-06, 9.99979730e-01],
       [9.15641150e-03, 4.11991480e-01, 5.78852060e-01],
       [9.91080100e-01, 1.66856120e-04, 8.75307200e-03],
       [4.37400700e-01, 1.60891030e-01, 4.01708280e-01],
       [5.71027440e-07, 1.73184160e-05, 9.99982100e-01],
       [1.19961925e-01, 8.37380200e-01, 4.26579380e-02],
       [8.03629500e-01, 1.92813770e-01, 3.55665600e-03],
       [2.16953570e-07, 8.91369700e-08, 9.99999640e-01],
       [1.11442500e-08, 9.99982360e-01, 1.76459540e-05],
       [9.46645200e-03, 5.83571830e-02, 9.32176400e-01],
       [7.93485640e-01, 6.47594060e-06, 2.06507880e-01],
       [2.71189900e-05, 9.99818400e-01, 1.54520440e-04],
       [6.00475500e-01, 2.31899500e-01, 1.67625010e-01],
       [9.71765100e-07, 4.49885260e-05, 9.99954000e-01],
       [4.16675960e-01, 4.11534

In [101]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.5490105132962276

In [102]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.5490105132962276

In [103]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [104]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [105]:
dat8

,strain,test
131,NRS104,0
123,NRS071,0
124,NRS072,1
27,BCH-SA-12,0
57,CFBREBSa125,2
...,...,...
170,NRS199,2
32,CA11,2
212,NRS253,1
141,NRS119,2


In [106]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [107]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [108]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [123]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 967us/step - loss: 1.1118 - accuracy: 0.3842 - val_loss: 1.0559 - val_accuracy: 0.4211
Epoch 2/1000
177/177 [==============================] - 0s 303us/step - loss: 1.0452 - accuracy: 0.4689 - val_loss: 1.0576 - val_accuracy: 0.3816
Epoch 3/1000
177/177 [==============================] - 0s 504us/step - loss: 1.0178 - accuracy: 0.5028 - val_loss: 1.0573 - val_accuracy: 0.3816
Epoch 4/1000
177/177 [==============================] - 0s 620us/step - loss: 0.9838 - accuracy: 0.5537 - val_loss: 1.0359 - val_accuracy: 0.4474
Epoch 5/1000
177/177 [==============================] - 0s 93us/step - loss: 0.9636 - accuracy: 0.5706 - val_loss: 1.0183 - val_accuracy: 0.4868
Epoch 6/1000
177/177 [==============================] - 0s 537us/step - loss: 0.9388 - accuracy: 0.5706 - val_loss: 1.0049 - val_accuracy: 0.5000
Epoch 7/1000
177/177 [==============================] - 0s 311us/step - loss: 0.

Epoch 112/1000
177/177 [==============================] - 0s 96us/step - loss: 0.2947 - accuracy: 0.8814 - val_loss: 1.4168 - val_accuracy: 0.5000
Epoch 113/1000
177/177 [==============================] - 0s 107us/step - loss: 0.2937 - accuracy: 0.8814 - val_loss: 1.4266 - val_accuracy: 0.5395
Epoch 114/1000
177/177 [==============================] - 0s 90us/step - loss: 0.2903 - accuracy: 0.8757 - val_loss: 1.4344 - val_accuracy: 0.5000
Epoch 115/1000
177/177 [==============================] - 0s 101us/step - loss: 0.2945 - accuracy: 0.8870 - val_loss: 1.4503 - val_accuracy: 0.5132
Epoch 116/1000
177/177 [==============================] - 0s 100us/step - loss: 0.2932 - accuracy: 0.8757 - val_loss: 1.4800 - val_accuracy: 0.5395
Epoch 117/1000
177/177 [==============================] - 0s 198us/step - loss: 0.2966 - accuracy: 0.8701 - val_loss: 1.4811 - val_accuracy: 0.5000
Epoch 118/1000
177/177 [==============================] - 0s 351us/step - loss: 0.2843 - accuracy: 0.8870 - val_lo

177/177 [==============================] - 0s 109us/step - loss: 0.2217 - accuracy: 0.8870 - val_loss: 1.8814 - val_accuracy: 0.4868
Epoch 223/1000
177/177 [==============================] - 0s 91us/step - loss: 0.2302 - accuracy: 0.8757 - val_loss: 1.8831 - val_accuracy: 0.5000
Epoch 224/1000
177/177 [==============================] - 0s 84us/step - loss: 0.2207 - accuracy: 0.8927 - val_loss: 1.8871 - val_accuracy: 0.4868
Epoch 225/1000
177/177 [==============================] - 0s 122us/step - loss: 0.2248 - accuracy: 0.8927 - val_loss: 1.9043 - val_accuracy: 0.4737
Epoch 226/1000
177/177 [==============================] - 0s 99us/step - loss: 0.2238 - accuracy: 0.8870 - val_loss: 1.9213 - val_accuracy: 0.4737
Epoch 227/1000
177/177 [==============================] - 0s 69us/step - loss: 0.2202 - accuracy: 0.8757 - val_loss: 1.9310 - val_accuracy: 0.4868
Epoch 228/1000
177/177 [==============================] - 0s 90us/step - loss: 0.2256 - accuracy: 0.8701 - val_loss: 1.9261 - val_a

Epoch 278/1000
177/177 [==============================] - 0s 712us/step - loss: 0.2165 - accuracy: 0.8983 - val_loss: 2.0781 - val_accuracy: 0.4868
Epoch 279/1000
177/177 [==============================] - 0s 93us/step - loss: 0.2210 - accuracy: 0.8870 - val_loss: 2.0638 - val_accuracy: 0.4737
Epoch 280/1000
177/177 [==============================] - 0s 99us/step - loss: 0.2194 - accuracy: 0.8870 - val_loss: 2.0536 - val_accuracy: 0.4605
Epoch 281/1000
177/177 [==============================] - 0s 98us/step - loss: 0.2217 - accuracy: 0.8927 - val_loss: 2.0656 - val_accuracy: 0.4737
Epoch 282/1000
177/177 [==============================] - 0s 102us/step - loss: 0.2216 - accuracy: 0.8927 - val_loss: 2.0859 - val_accuracy: 0.4868
Epoch 283/1000
177/177 [==============================] - 0s 89us/step - loss: 0.2090 - accuracy: 0.8927 - val_loss: 2.1025 - val_accuracy: 0.4868
Epoch 284/1000
177/177 [==============================] - 0s 84us/step - loss: 0.2174 - accuracy: 0.8814 - val_loss:

177/177 [==============================] - 0s 117us/step - loss: 0.2016 - accuracy: 0.8983 - val_loss: 2.2577 - val_accuracy: 0.4605
Epoch 389/1000
177/177 [==============================] - 0s 88us/step - loss: 0.2025 - accuracy: 0.8870 - val_loss: 2.2881 - val_accuracy: 0.4737
Epoch 390/1000
177/177 [==============================] - 0s 86us/step - loss: 0.2014 - accuracy: 0.8927 - val_loss: 2.2834 - val_accuracy: 0.4868
Epoch 391/1000
177/177 [==============================] - 0s 98us/step - loss: 0.2241 - accuracy: 0.8644 - val_loss: 2.2839 - val_accuracy: 0.4605
Epoch 392/1000
177/177 [==============================] - 0s 124us/step - loss: 0.2086 - accuracy: 0.8927 - val_loss: 2.3299 - val_accuracy: 0.4868
Epoch 393/1000
177/177 [==============================] - 0s 91us/step - loss: 0.2152 - accuracy: 0.8927 - val_loss: 2.3094 - val_accuracy: 0.4868
Epoch 394/1000
177/177 [==============================] - 0s 477us/step - loss: 0.2122 - accuracy: 0.8814 - val_loss: 2.2910 - val_

...

177/177 [==============================] - 0s 109us/step - loss: 0.2006 - accuracy: 0.8983 - val_loss: 2.5803 - val_accuracy: 0.5000
Epoch 609/1000
177/177 [==============================] - 0s 75us/step - loss: 0.2023 - accuracy: 0.8757 - val_loss: 2.5474 - val_accuracy: 0.4868
Epoch 610/1000
177/177 [==============================] - 0s 86us/step - loss: 0.2041 - accuracy: 0.8870 - val_loss: 2.5725 - val_accuracy: 0.4737
Epoch 611/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1988 - accuracy: 0.8983 - val_loss: 2.6025 - val_accuracy: 0.4868
Epoch 612/1000
177/177 [==============================] - 0s 86us/step - loss: 0.2075 - accuracy: 0.8927 - val_loss: 2.6035 - val_accuracy: 0.4737
Epoch 613/1000
177/177 [==============================] - 0s 195us/step - loss: 0.2002 - accuracy: 0.8927 - val_loss: 2.5847 - val_accuracy: 0.4605
Epoch 614/1000
177/177 [==============================] - 0s 256us/step - loss: 0.2073 - accuracy: 0.8870 - val_loss: 2.5740 - val

177/177 [==============================] - 0s 80us/step - loss: 0.1940 - accuracy: 0.8983 - val_loss: 2.6608 - val_accuracy: 0.5000
Epoch 719/1000
177/177 [==============================] - 0s 94us/step - loss: 0.1944 - accuracy: 0.8983 - val_loss: 2.6719 - val_accuracy: 0.4868
Epoch 720/1000
177/177 [==============================] - 0s 141us/step - loss: 0.1988 - accuracy: 0.8870 - val_loss: 2.6870 - val_accuracy: 0.4737
Epoch 721/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1964 - accuracy: 0.8870 - val_loss: 2.7184 - val_accuracy: 0.4868
Epoch 722/1000
177/177 [==============================] - 0s 95us/step - loss: 0.2029 - accuracy: 0.8927 - val_loss: 2.7182 - val_accuracy: 0.4737
Epoch 723/1000
177/177 [==============================] - 0s 145us/step - loss: 0.1987 - accuracy: 0.8927 - val_loss: 2.7139 - val_accuracy: 0.4474
Epoch 724/1000
177/177 [==============================] - 0s 196us/step - loss: 0.1998 - accuracy: 0.8983 - val_loss: 2.7080 - val_

177/177 [==============================] - 0s 80us/step - loss: 0.1994 - accuracy: 0.8870 - val_loss: 2.7627 - val_accuracy: 0.4737
Epoch 829/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2017 - accuracy: 0.8927 - val_loss: 2.7804 - val_accuracy: 0.4605
Epoch 830/1000
177/177 [==============================] - 0s 79us/step - loss: 0.2026 - accuracy: 0.8927 - val_loss: 2.7799 - val_accuracy: 0.4474
Epoch 831/1000
177/177 [==============================] - 0s 76us/step - loss: 0.1983 - accuracy: 0.8927 - val_loss: 2.8133 - val_accuracy: 0.5000
Epoch 832/1000
177/177 [==============================] - 0s 82us/step - loss: 0.2104 - accuracy: 0.8927 - val_loss: 2.7930 - val_accuracy: 0.5000
Epoch 833/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2144 - accuracy: 0.8644 - val_loss: 2.7482 - val_accuracy: 0.4474
Epoch 834/1000
177/177 [==============================] - 0s 68us/step - loss: 0.2140 - accuracy: 0.8983 - val_loss: 2.7226 - val_acc

Epoch 884/1000
177/177 [==============================] - 0s 87us/step - loss: 0.2045 - accuracy: 0.8814 - val_loss: 2.8533 - val_accuracy: 0.5000
Epoch 885/1000
177/177 [==============================] - 0s 101us/step - loss: 0.1969 - accuracy: 0.8814 - val_loss: 2.8506 - val_accuracy: 0.4737
Epoch 886/1000
177/177 [==============================] - 0s 82us/step - loss: 0.1956 - accuracy: 0.8983 - val_loss: 2.8403 - val_accuracy: 0.4737
Epoch 887/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1975 - accuracy: 0.8927 - val_loss: 2.8638 - val_accuracy: 0.4605
Epoch 888/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1933 - accuracy: 0.8983 - val_loss: 2.8488 - val_accuracy: 0.4605
Epoch 889/1000
177/177 [==============================] - 0s 85us/step - loss: 0.2005 - accuracy: 0.8644 - val_loss: 2.8396 - val_accuracy: 0.4868
Epoch 890/1000
177/177 [==============================] - 0s 96us/step - loss: 0.1966 - accuracy: 0.8757 - val_loss: 

177/177 [==============================] - 0s 236us/step - loss: 0.1906 - accuracy: 0.8983 - val_loss: 2.9437 - val_accuracy: 0.4737
Epoch 995/1000
177/177 [==============================] - 0s 92us/step - loss: 0.1947 - accuracy: 0.8870 - val_loss: 2.9357 - val_accuracy: 0.4868
Epoch 996/1000
177/177 [==============================] - 0s 98us/step - loss: 0.1909 - accuracy: 0.8927 - val_loss: 2.9295 - val_accuracy: 0.4605
Epoch 997/1000
177/177 [==============================] - 0s 98us/step - loss: 0.2059 - accuracy: 0.8814 - val_loss: 2.9284 - val_accuracy: 0.4605
Epoch 998/1000
177/177 [==============================] - 0s 102us/step - loss: 0.1956 - accuracy: 0.8927 - val_loss: 2.9679 - val_accuracy: 0.4868
Epoch 999/1000
177/177 [==============================] - 0s 85us/step - loss: 0.2086 - accuracy: 0.8927 - val_loss: 2.9939 - val_accuracy: 0.5000
Epoch 1000/1000
177/177 [==============================] - 0s 96us/step - loss: 0.2045 - accuracy: 0.8927 - val_loss: 2.9881 - val_

In [328]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 75us/step
over-sampling test accuracy: 50.00%


In [124]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([1, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0, 1, 2, 1, 0, 2, 1, 2, 0, 1, 2, 2,
       2, 1, 0, 2, 2, 1, 2, 2, 2, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 0, 2, 0,
       1, 2, 2, 0, 1, 2, 2, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1,
       1, 2, 2, 1, 1, 1, 1, 1, 2, 0])

In [125]:
dat8['pred'] = pred8
dat8

,strain,test,pred
131,NRS104,0,1
123,NRS071,0,2
124,NRS072,1,2
27,BCH-SA-12,0,2
57,CFBREBSa125,2,0
...,...,...,...
170,NRS199,2,1
32,CA11,2,1
212,NRS253,1,1
141,NRS119,2,2


In [126]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [127]:
dat_proba8

,0,1,2
0,8.210638e-02,0.917809,0.000085
1,1.015229e-02,0.255327,0.734520
2,3.642946e-03,0.000002,0.996355
3,2.809318e-03,0.099188,0.898003
4,4.875667e-01,0.088657,0.423776
...,...,...,...
71,2.406253e-01,0.757694,0.001680
72,7.546038e-02,0.922721,0.001818
73,2.170159e-08,0.999959,0.000041
74,1.087519e-09,0.000335,0.999665


In [128]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [129]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p17.csv", index = False,
         header=None)

In [332]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 99us/step - loss: 0.1938 - accuracy: 0.8983 - val_loss: 3.0909 - val_accuracy: 0.5000
Epoch 2/1000
177/177 [==============================] - 0s 78us/step - loss: 0.1950 - accuracy: 0.8870 - val_loss: 3.0930 - val_accuracy: 0.4868
Epoch 3/1000
177/177 [==============================] - 0s 77us/step - loss: 0.1986 - accuracy: 0.8983 - val_loss: 3.1016 - val_accuracy: 0.4474
Epoch 4/1000
177/177 [==============================] - 0s 112us/step - loss: 0.1959 - accuracy: 0.8983 - val_loss: 3.1034 - val_accuracy: 0.4605
Epoch 5/1000
177/177 [==============================] - 0s 127us/step - loss: 0.1915 - accuracy: 0.8927 - val_loss: 3.1144 - val_accuracy: 0.4868
Epoch 6/1000
177/177 [==============================] - 0s 115us/step - loss: 0.1937 - accuracy: 0.8983 - val_loss: 3.1155 - val_accuracy: 0.4868
Epoch 7/1000
177/177 [==============================] - 0s 113us/step - loss: 0.19

Epoch 57/1000
177/177 [==============================] - 0s 89us/step - loss: 0.1967 - accuracy: 0.8983 - val_loss: 3.2025 - val_accuracy: 0.4868
Epoch 58/1000
177/177 [==============================] - 0s 109us/step - loss: 0.1935 - accuracy: 0.8983 - val_loss: 3.1380 - val_accuracy: 0.4868
Epoch 59/1000
177/177 [==============================] - 0s 96us/step - loss: 0.2076 - accuracy: 0.8927 - val_loss: 3.1276 - val_accuracy: 0.5000
Epoch 60/1000
177/177 [==============================] - 0s 106us/step - loss: 0.2044 - accuracy: 0.8870 - val_loss: 3.1567 - val_accuracy: 0.5000
Epoch 61/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1959 - accuracy: 0.8983 - val_loss: 3.2415 - val_accuracy: 0.4737
Epoch 62/1000
177/177 [==============================] - 0s 94us/step - loss: 0.2011 - accuracy: 0.8983 - val_loss: 3.1791 - val_accuracy: 0.4737
Epoch 63/1000
177/177 [==============================] - 0s 107us/step - loss: 0.1905 - accuracy: 0.8927 - val_loss: 3.08

Epoch 113/1000
177/177 [==============================] - 0s 86us/step - loss: 0.1958 - accuracy: 0.8927 - val_loss: 3.1593 - val_accuracy: 0.4605
Epoch 114/1000
177/177 [==============================] - 0s 80us/step - loss: 0.1945 - accuracy: 0.8983 - val_loss: 3.1761 - val_accuracy: 0.4605
Epoch 115/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1943 - accuracy: 0.8983 - val_loss: 3.2006 - val_accuracy: 0.4605
Epoch 116/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1949 - accuracy: 0.8927 - val_loss: 3.2198 - val_accuracy: 0.4868
Epoch 117/1000
177/177 [==============================] - 0s 78us/step - loss: 0.1910 - accuracy: 0.9040 - val_loss: 3.2173 - val_accuracy: 0.4868
Epoch 118/1000
177/177 [==============================] - 0s 84us/step - loss: 0.1975 - accuracy: 0.8870 - val_loss: 3.1989 - val_accuracy: 0.5132
Epoch 119/1000
177/177 [==============================] - 0s 96us/step - loss: 0.2000 - accuracy: 0.8870 - val_loss: 3

Epoch 224/1000
177/177 [==============================] - 0s 150us/step - loss: 0.1955 - accuracy: 0.8927 - val_loss: 3.1238 - val_accuracy: 0.4868
Epoch 225/1000
177/177 [==============================] - 0s 114us/step - loss: 0.1937 - accuracy: 0.8757 - val_loss: 3.1327 - val_accuracy: 0.5263
Epoch 226/1000
177/177 [==============================] - 0s 109us/step - loss: 0.1943 - accuracy: 0.8757 - val_loss: 3.1361 - val_accuracy: 0.5132
Epoch 227/1000
177/177 [==============================] - 0s 95us/step - loss: 0.1934 - accuracy: 0.8870 - val_loss: 3.1278 - val_accuracy: 0.5000
Epoch 228/1000
177/177 [==============================] - 0s 136us/step - loss: 0.1939 - accuracy: 0.8983 - val_loss: 3.1272 - val_accuracy: 0.5000
Epoch 229/1000
177/177 [==============================] - 0s 107us/step - loss: 0.1972 - accuracy: 0.8870 - val_loss: 3.1244 - val_accuracy: 0.4868
Epoch 230/1000
177/177 [==============================] - 0s 114us/step - loss: 0.1912 - accuracy: 0.8983 - val_l

177/177 [==============================] - 0s 322us/step - loss: 0.1944 - accuracy: 0.8983 - val_loss: 3.2225 - val_accuracy: 0.5000
Epoch 335/1000
177/177 [==============================] - 0s 155us/step - loss: 0.1913 - accuracy: 0.8870 - val_loss: 3.2541 - val_accuracy: 0.5000
Epoch 336/1000
177/177 [==============================] - 0s 294us/step - loss: 0.2025 - accuracy: 0.8927 - val_loss: 3.2898 - val_accuracy: 0.4868
Epoch 337/1000
177/177 [==============================] - 0s 126us/step - loss: 0.2009 - accuracy: 0.8870 - val_loss: 3.2515 - val_accuracy: 0.4605
Epoch 338/1000
177/177 [==============================] - 0s 107us/step - loss: 0.1969 - accuracy: 0.8870 - val_loss: 3.2265 - val_accuracy: 0.5000
Epoch 339/1000
177/177 [==============================] - 0s 110us/step - loss: 0.1942 - accuracy: 0.8927 - val_loss: 3.2352 - val_accuracy: 0.5000
Epoch 340/1000
177/177 [==============================] - 0s 92us/step - loss: 0.1963 - accuracy: 0.8814 - val_loss: 3.2795 - v

...

Epoch 556/1000
177/177 [==============================] - 0s 76us/step - loss: 0.1933 - accuracy: 0.8927 - val_loss: 3.4068 - val_accuracy: 0.4605
Epoch 557/1000
177/177 [==============================] - 0s 81us/step - loss: 0.1967 - accuracy: 0.8927 - val_loss: 3.3801 - val_accuracy: 0.4737
Epoch 558/1000
177/177 [==============================] - 0s 91us/step - loss: 0.1979 - accuracy: 0.8870 - val_loss: 3.3553 - val_accuracy: 0.4342
Epoch 559/1000
177/177 [==============================] - 0s 81us/step - loss: 0.1941 - accuracy: 0.8870 - val_loss: 3.3771 - val_accuracy: 0.4737
Epoch 560/1000
177/177 [==============================] - 0s 228us/step - loss: 0.1982 - accuracy: 0.8870 - val_loss: 3.4019 - val_accuracy: 0.5000
Epoch 561/1000
177/177 [==============================] - 0s 124us/step - loss: 0.2012 - accuracy: 0.8927 - val_loss: 3.3598 - val_accuracy: 0.4737
Epoch 562/1000
177/177 [==============================] - 0s 101us/step - loss: 0.1938 - accuracy: 0.8983 - val_loss

Epoch 667/1000
177/177 [==============================] - 0s 266us/step - loss: 0.1916 - accuracy: 0.8983 - val_loss: 3.6705 - val_accuracy: 0.4474
Epoch 668/1000
177/177 [==============================] - 0s 496us/step - loss: 0.2005 - accuracy: 0.8814 - val_loss: 3.7194 - val_accuracy: 0.4868
Epoch 669/1000
177/177 [==============================] - 0s 680us/step - loss: 0.1973 - accuracy: 0.8870 - val_loss: 3.6632 - val_accuracy: 0.4737
Epoch 670/1000
177/177 [==============================] - 0s 252us/step - loss: 0.1917 - accuracy: 0.8983 - val_loss: 3.6122 - val_accuracy: 0.4737
Epoch 671/1000
177/177 [==============================] - 0s 194us/step - loss: 0.1924 - accuracy: 0.8983 - val_loss: 3.5840 - val_accuracy: 0.4605
Epoch 672/1000
177/177 [==============================] - 0s 106us/step - loss: 0.1958 - accuracy: 0.8927 - val_loss: 3.5591 - val_accuracy: 0.4605
Epoch 673/1000
177/177 [==============================] - 0s 81us/step - loss: 0.1894 - accuracy: 0.8927 - val_l

177/177 [==============================] - 0s 141us/step - loss: 0.1963 - accuracy: 0.8757 - val_loss: 3.5611 - val_accuracy: 0.4868
Epoch 778/1000
177/177 [==============================] - 0s 202us/step - loss: 0.2040 - accuracy: 0.8870 - val_loss: 3.6025 - val_accuracy: 0.4868
Epoch 779/1000
177/177 [==============================] - 0s 164us/step - loss: 0.1984 - accuracy: 0.8927 - val_loss: 3.6211 - val_accuracy: 0.4474
Epoch 780/1000
177/177 [==============================] - 0s 232us/step - loss: 0.1897 - accuracy: 0.8983 - val_loss: 3.6247 - val_accuracy: 0.5000
Epoch 781/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1986 - accuracy: 0.8927 - val_loss: 3.6587 - val_accuracy: 0.4868
Epoch 782/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1933 - accuracy: 0.8814 - val_loss: 3.6907 - val_accuracy: 0.4737
Epoch 783/1000
177/177 [==============================] - 0s 78us/step - loss: 0.1953 - accuracy: 0.8983 - val_loss: 3.6999 - val

177/177 [==============================] - 0s 118us/step - loss: 0.1917 - accuracy: 0.8814 - val_loss: 3.6557 - val_accuracy: 0.4605
Epoch 888/1000
177/177 [==============================] - 0s 113us/step - loss: 0.1922 - accuracy: 0.8983 - val_loss: 3.6334 - val_accuracy: 0.4605
Epoch 889/1000
177/177 [==============================] - 0s 134us/step - loss: 0.1913 - accuracy: 0.8870 - val_loss: 3.6224 - val_accuracy: 0.4737
Epoch 890/1000
177/177 [==============================] - 0s 225us/step - loss: 0.1919 - accuracy: 0.8870 - val_loss: 3.6359 - val_accuracy: 0.4868
Epoch 891/1000
177/177 [==============================] - 0s 86us/step - loss: 0.1930 - accuracy: 0.8983 - val_loss: 3.6311 - val_accuracy: 0.4868
Epoch 892/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1902 - accuracy: 0.8927 - val_loss: 3.6360 - val_accuracy: 0.4737
Epoch 893/1000
177/177 [==============================] - 0s 84us/step - loss: 0.1930 - accuracy: 0.8983 - val_loss: 3.6561 - val

177/177 [==============================] - 0s 73us/step - loss: 0.1914 - accuracy: 0.8983 - val_loss: 3.7601 - val_accuracy: 0.4605
Epoch 998/1000
177/177 [==============================] - 0s 72us/step - loss: 0.1926 - accuracy: 0.8870 - val_loss: 3.7341 - val_accuracy: 0.4474
Epoch 999/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1897 - accuracy: 0.8983 - val_loss: 3.7019 - val_accuracy: 0.4868
Epoch 1000/1000
177/177 [==============================] - 0s 81us/step - loss: 0.1945 - accuracy: 0.8814 - val_loss: 3.6934 - val_accuracy: 0.5000


In [333]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 89.04%


In [109]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [110]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [111]:
y_prob8 = df_proba8[df_proba8['phage']=='p0017kpresabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[8.21063800e-02, 9.17808800e-01, 8.47792250e-05],
       [1.01522890e-02, 2.55327370e-01, 7.34520260e-01],
       [3.64294650e-03, 2.34740170e-06, 9.96354700e-01],
       [2.80931820e-03, 9.91877900e-02, 8.98002900e-01],
       [4.87566740e-01, 8.86572300e-02, 4.23776060e-01],
       [8.37581160e-01, 1.62388010e-01, 3.08292840e-05],
       [3.15970500e-01, 5.51623200e-01, 1.32406230e-01],
       [5.64665100e-09, 3.97672550e-07, 9.99999640e-01],
       [9.98806360e-01, 5.45188260e-04, 6.48426900e-04],
       [5.19147900e-01, 1.70550230e-01, 3.10301900e-01],
       [8.36943000e-01, 9.05285850e-02, 7.25284600e-02],
       [1.57329770e-06, 9.62578300e-01, 3.74201200e-02],
       [4.13292000e-03, 1.57350900e-01, 8.38516200e-01],
       [6.03742060e-03, 9.93954360e-01, 8.15803300e-06],
       [5.19147900e-01, 1.70550230e-01, 3.10301900e-01],
       [2.03509640e-08, 3.55870060e-04, 9.99644160e-01],
       [4.97381400e-02, 9.45196750e-01, 5.06509800e-03],
       [3.72726800e-01, 2.07393

In [112]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.6786687739068692

In [113]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.6786687739068692

In [114]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.6310225268558602

In [115]:
np.std(ovos2)

0.05842970447680021

In [116]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.6310225268558602

In [117]:
np.std(ovrs2)

0.05842970447680021

In [334]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [335]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 50.33%


In [336]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.03878231828138047


In [337]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [338]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 91.42%


In [339]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.014140104
